<a href="https://colab.research.google.com/github/palanipsb/Chatbot/blob/master/testfiletg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DROP TRIGGER CHECK_DATA_FOR_PPID;

CREATE OR REPLACE TRIGGER CHECK_DATA_FOR_PPID
                    before INSERT ON DATA
                    referencing OLD as OLD NEW as NEW
                    for each row
declare
                    c_Wrk_Id VARCHAR(40):=NULL;
                    v_username VARCHAR2 (128);
                BEGIN
                    c_Wrk_Id := :new.MODIFIED_BY;
                    SELECT USER INTO v_username FROM DUAL;  
                    :new.CREATED_BY_USER :=  v_username ;
                    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
                    :new.MODIFIED_BY_USER :=  v_username;
                    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
                    if (:new.FIELD LIKE 'PPID') then
                        begin
                            update USR_PAT set PP_ID = :new.VALUE, MODIFIED = :new.MODIFIED, MODIFIED_BY = :new.MODIFIED_BY where PAT_ID = :new.PAT_ID;
                        end;
                    end if;
                END;
/


DROP TRIGGER CHECK_DATA_FOR_PPID_MODIF;

CREATE OR REPLACE TRIGGER CHECK_DATA_FOR_PPID_MODIF
                before UPDATE of VALUE ON DATA
                referencing OLD as OLD NEW as NEW
                for each row
declare
                    v_username VARCHAR2 (128);
                BEGIN
                
                    SELECT USER INTO v_username FROM DUAL;  
                    :new.MODIFIED_BY_USER :=  v_username;
                    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
                    
                    if (:new.FIELD LIKE 'PPID') then
                        begin
                            update USR_PAT set PP_ID = :new.VALUE, MODIFIED = :new.MODIFIED, MODIFIED_BY = :new.MODIFIED_BY where PAT_ID = :new.PAT_ID;
                        end;
                    end if;
                END;
/


DROP TRIGGER DELETE_CODE;

CREATE OR REPLACE TRIGGER DELETE_CODE
BEFORE DELETE ON CODES
REFERENCING OLD AS OLD
FOR EACH ROW
begin
 INSERT INTO CODES_AUDIT
  (REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, CODE_ID,
  DICT,  CODE_TYPE, CODE1, CODE2, CODE3, CODE4, CODE5, CODE6,
  DECODE1, DECODE2, DECODE3, DECODE4, DECODE5, DECODE6,
  MODIFIED, MODIFIED_BY, STATUS, LOC,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
 VALUES
  (:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.CODE_ID,
  :old.DICT, :old.CODE_TYPE, :old.CODE1, :old.CODE2, :old.CODE3, :old.CODE4, :old.CODE5, :old.CODE6,
  :old.DECODE1, :old.DECODE2, :old.DECODE3, :old.DECODE4, :old.DECODE5, :old.DECODE6,
  :old.MODIFIED, :old.MODIFIED_BY, :old.STATUS, :old.LOC,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
end;
/


DROP TRIGGER DELETE_COMMENTS;

CREATE OR REPLACE TRIGGER DELETE_COMMENTS
before DELETE ON COMMENTS
referencing OLD as OLD
for each row
BEGIN
	INSERT INTO COMMENTS_AUDIT
		(PAT_ID, FORM, PAGE, MODULE, GROUP_ID, FIELD, VALUE, MODIFIED_BY, MODIFIED, REASON, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.GROUP_ID, :old.FIELD, :old.VALUE, :old.MODIFIED_BY, :old.MODIFIED, :old.REASON,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELETE_DATA;

CREATE OR REPLACE TRIGGER DELETE_DATA
                    before DELETE ON DATA
                    referencing OLD as OLD
                    for each row
declare
                	c_Module VARCHAR2(40):=NULL;
                	n_Criticals NUMBER := 0;
                BEGIN
                	if (LOCKING_PKG.isFrozen(1, :old.PAT_ID, :old.FORM, :old.PAGE, :old.FIELD)) then
                		Raise_application_error(-20200, 'Patient is locked or frozen.');
                	end if;
                
                	if (:old.LOCKED=1) then
                		Raise_application_error(-20210, 'Field '||:old.FIELD||' is locked.');
                	end if;

                
                	update USR_PAT set MODIFIED=SYSDATE, SIGNED=0 where PAT_ID = :old.PAT_ID;

                    INSERT INTO DATA_AUDIT
                        (REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE, SPEC_VALUE, MODIFIED, MODIFIED_BY, COMMENTS, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
                    VALUES
                        (:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE, :old.SPEC_VALUE, :old.MODIFIED, :old.MODIFIED_BY, :old.COMMENTS, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
                
                	UPDATE QUERY SET STATUS=11, ANSWER='Query invalid due to deleted data', MODIFIED_BY=:old.MODIFIED_BY WHERE
                	   STATUS in (0,1,4,21) AND
                	   PAT_ID=:old.PAT_ID AND FIELD=:old.FIELD AND FORM=:old.FORM AND PAGE=:old.PAGE AND COUNTER=:old.COUNTER;
                
                	delete from OPT_DATA where PAT_ID=:old.PAT_ID AND FIELD=:old.FIELD AND FORM=:old.FORM AND PAGE=:old.PAGE AND COUNTER=:old.COUNTER;
                	update CODES set MODIFIED = :old.MODIFIED, MODIFIED_BY = :old.MODIFIED_BY, REASON = 'Data deleted', STATUS=2
                		where PAT_ID=:old.PAT_ID and FORM=:old.FORM and PAGE=:old.PAGE and FIELD=:old.FIELD and COUNTER=:old.COUNTER;
                	delete from CODES where PAT_ID=:old.PAT_ID and FORM=:old.FORM and PAGE=:old.PAGE and FIELD=:old.FIELD and COUNTER=:old.COUNTER;

                	SDV_PKG.onDataUpdate(:old.MODIFIED_BY, :old.PAT_ID, :old.FORM, :old.PAGE, :old.FIELD, :old.COUNTER);

                	begin
                		select MODULE into c_Module from FIELD_DESCR where FORM=:old.FORM and FIELD=:old.FIELD;
                	exception
                		when NO_DATA_FOUND then
                			c_Module:=NULL;
                	end;
                    select count(*) into n_Criticals from FIELD_DESCR where BITAND(FLAGS,16)=16;
                    IF(n_Criticals > 0) THEN   
                       SDV_PKG.deleteCriticalLoopField(:old.PAT_ID, :old.FORM, :old.PAGE, :old.COUNTER, :old.FIELD);
                    END IF;












                		   
                END;
/


DROP TRIGGER DELETE_DATA_VIS;

CREATE OR REPLACE TRIGGER DELETE_DATA_VIS
before DELETE ON DATA
referencing OLD as OLD
for each row
BEGIN
    if (:old.FIELD in ('VISDAY','VISMON','VISYEA') OR :new.FIELD like '%!_DATE' escape '!') then
        insert into VS_QUEUE (PAT_ID,FORM,INSTANCE,COUNTER) values(:old.PAT_ID,:old.FORM,:old.PAGE,:old.COUNTER);
    end if;
END;
/


DROP TRIGGER DELETE_EXT_DATA;

CREATE OR REPLACE TRIGGER DELETE_EXT_DATA
before DELETE ON EXT_DATA
referencing OLD as OLD
for each row
BEGIN
	if (LOCKING_PKG.isFrozen(4, :old.PAT_ID, :old.FORM, :old.PAGE, :old.FIELD)) then
		Raise_application_error(-20200, 'Patient is locked or frozen (ext. data).');
	end if;

	if (:old.LOCKED=1) then
		Raise_application_error(-20212, 'Ext. field '||:old.FIELD||' is locked.');
	end if;

	if (:old.ID = 1) then
		INSERT INTO EXT_DATA_AUDIT
	   		(PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE_TYPE,
			 ID, VALUE, COMMENTS, MODIFIED, MODIFIED_BY,
			 REASON, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
		VALUES
	   		(:old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE_TYPE,
			 :old.ID, :old.VALUE, :old.COMMENTS, :old.MODIFIED, :old.MODIFIED_BY,
			 :old.REASON, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);

		SDV_PKG.onDataUpdate(:old.MODIFIED_BY, :old.PAT_ID, :old.FORM, :old.PAGE, :old.FIELD, :old.COUNTER);	   
	end if;
END;
/


DROP TRIGGER DELETE_LABR_DATA;

CREATE OR REPLACE TRIGGER DELETE_LABR_DATA
before DELETE ON LABR_DATA
referencing OLD as OLD
for each row
BEGIN
	INSERT INTO LABR_DATA_AUDIT
   		(SIT_ID, SET_NAME, SET_COUNTER, PARAM, PARAM_COUNTER, LIMIT1, LIMIT2, LIMIT1_REL, LIMIT2_REL, LIMIT1_UNIT,
   		LIMIT1_UNIT_T, LIMIT2_UNIT, LIMIT2_UNIT_T, SEX, AGE_FROM, AGE_FROM_REL, AGE_TO, AGE_TO_REL, MODIFIED, MODIFIED_BY)
	VALUES
   		(:old.SIT_ID, :old.SET_NAME, :old.SET_COUNTER, :old.PARAM, :old.PARAM_COUNTER, :old.LIMIT1, :old.LIMIT2, :old.LIMIT1_REL, :old.LIMIT2_REL, :old.LIMIT1_UNIT,
   		:old.LIMIT1_UNIT_T, :old.LIMIT2_UNIT, :old.LIMIT2_UNIT_T, :old.SEX, :old.AGE_FROM, :old.AGE_FROM_REL, :old.AGE_TO, :old.AGE_TO_REL, :old.MODIFIED, :old.MODIFIED_BY);
END;
/


DROP TRIGGER DELETE_LABR_SETS;

CREATE OR REPLACE TRIGGER DELETE_LABR_SETS
before DELETE ON LABR_SETS
referencing OLD as OLD
for each row
BEGIN
	INSERT INTO LABR_SETS_AUDIT
   		(SIT_ID, SET_NAME, SET_COUNTER, LAB_NAME, LAB_ID, FROM_DAY, FROM_MON, FROM_YEA, FROM_REL,
   		TO_DAY, TO_MON, TO_YEA, TO_REL, SIGNED, SIGNED_BY, MODIFIED, MODIFIED_BY)
	VALUES
   		(:old.SIT_ID, :old.SET_NAME, :old.SET_COUNTER, :old.LAB_NAME, :old.LAB_ID, :old.FROM_DAY, :old.FROM_MON, :old.FROM_YEA, :old.FROM_REL,
   		:old.TO_DAY, :old.TO_MON, :old.TO_YEA, :old.TO_REL, :old.SIGNED, :old.SIGNED_BY, :old.MODIFIED, :old.MODIFIED_BY);
END;
/


DROP TRIGGER DELETE_LOCK;

CREATE OR REPLACE TRIGGER DELETE_LOCK
before DELETE ON LOCKING
referencing OLD as OLD
for each row
BEGIN
	insert into LOCKING_AUDIT
   		(PAT_ID, FORM, PAGE, MODULE, SET_ID, STATUS, MODIFIED, MODIFIED_BY, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.SET_ID, :old.STATUS, :old.MODIFIED, :old.MODIFIED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
end;
/


DROP TRIGGER DELETE_OPT_CODE;

CREATE OR REPLACE TRIGGER DELETE_OPT_CODE
before DELETE ON OPTCODES_LIST
for each row
BEGIN
  begin
      INSERT INTO OPTCODES_LIST_AUDIT
         (CODE,
         DICT,  CODE1, CODE2, CODE3, CODE4, CODE5, CODE6,
         DECODE1, DECODE2, DECODE3, DECODE4, DECODE5, DECODE6,
        MODIFIED, MODIFIED_BY)
     VALUES
      ( :old.CODE, :old.DICT,
    :old.CODE1, :old.CODE2, :old.CODE3, :old.CODE4, :old.CODE5, :old.CODE6,
      :old.DECODE1, :old.DECODE2, :old.DECODE3, :old.DECODE4, :old.DECODE5, :old.DECODE6,
      :old.MODIFIED, :old.MODIFIED_BY);
    end;
    delete from CODES where CODE1=to_char(:old.code) and decode(code_type, 4, 'med', 3, 'drug')=:old.dict;
END;
/


DROP TRIGGER DELETE_OPT_DATA;

CREATE OR REPLACE TRIGGER DELETE_OPT_DATA
before DELETE ON OPT_DATA
referencing OLD as OLD
for each row
BEGIN
	if (LOCKING_PKG.isFrozen(2, :old.PAT_ID, :old.FORM, :old.PAGE, :old.FIELD)) then
		Raise_application_error(-20200, 'Patient is locked or frozen (opt. data).');
	end if;

	if (:old.LOCKED=1) then
		Raise_application_error(-20211, 'Opt. field '||:old.FIELD||' is locked.');
	end if;

	INSERT INTO OPT_DATA_AUDIT
   		(REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE, MODIFIED, MODIFIED_BY, COMMENTS, ID, STATUS, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE, :old.MODIFIED, :old.MODIFIED_BY, :old.COMMENTS, :old.ID, :old.STATUS, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);

END;
/


DROP TRIGGER DELETE_SCRIPT;

CREATE OR REPLACE TRIGGER DELETE_SCRIPT
before DELETE ON SCRIPTS
referencing OLD as OLD
for each row
BEGIN
	insert into SCRIPTS_AUDIT
   		(NAME, VERSION, SOURCE, FLAGS, ALLOW, DENY, MODIFIED)
	VALUES
   		(:old.NAME, :old.VERSION, :old.SOURCE, :old.FLAGS, :old.ALLOW, :old.DENY, :old.MODIFIED);
end;
/


DROP TRIGGER DELETE_SCRIPT_QUERY;

CREATE OR REPLACE TRIGGER DELETE_SCRIPT_QUERY
before DELETE ON SCRIPT_QUERY
referencing OLD as OLD
for each row
BEGIN
	insert into SCRIPT_QUERY_AUDIT
   		(QUERY_ID, RUN_ID, PAT_ID, FIELD, FORM, PAGE, COUNTER, QUESTION, STATUS, CREATED, CREATED_BY, MODIFIED_BY, MODIFIED, GROUP_ID, ORIG_QUESTION, VALUE, SPEC_VALUE, COMMENTS, REF_ID, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.QUERY_ID, :old.RUN_ID, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.QUESTION, :old.STATUS, :old.CREATED, :old.CREATED_BY, :old.MODIFIED_BY, :old.MODIFIED, :old.GROUP_ID, :old.ORIG_QUESTION, :old.VALUE, :old.SPEC_VALUE, :old.COMMENTS, :old.REF_ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
end;
/


DROP TRIGGER DELETE_SDV;

CREATE OR REPLACE TRIGGER DELETE_SDV
before DELETE ON SDV
referencing OLD as OLD
for each row
BEGIN
	INSERT INTO SDV_AUDIT
   		(PAT_ID, FORM, PAGE, MODULE, FIELD, COUNTER, FLAGS, GROUP_ID, MODIFIED_BY, MODIFIED, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.FIELD, :old.COUNTER, :old.FLAGS, :old.GROUP_ID, :old.MODIFIED_BY, :old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELETE_SDV_ERRORS;

CREATE OR REPLACE TRIGGER DELETE_SDV_ERRORS
before DELETE ON SDV_ERRORS
referencing OLD as OLD
for each row
BEGIN
	INSERT INTO SDV_ERRORS_AUDIT
   		(PAT_ID, FORM, PAGE, MODULE, ERRORS, FIELDS1, FIELDS2, FIELDS3, FIELDS4, GROUP_ID, MODIFIED_BY, MODIFIED, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.ERRORS, :old.FIELDS1, :old.FIELDS2, :old.FIELDS3, :old.FIELDS4, :old.GROUP_ID, :old.MODIFIED_BY, :old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELETE_SETTING;

CREATE OR REPLACE TRIGGER DELETE_SETTING
BEFORE DELETE
ON SETTING
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
BEGIN
	INSERT INTO SETTING_AUDIT
        (TYPE,HIERARCHICAL,HIERARCHY_LEVEL,PARAMETER_GROUP,ACTIVE,REQUIRED,DEFAULT_VALUE,NAME,SETTING_ID,DEFAULT_UNIT,SETTING_GROUP,VALIDATION,SUBTYPE,MODIFIED,MODIFIED_BY,CREATED, CREATED_BY, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
        (:old.TYPE,:old.HIERARCHICAL,:old.HIERARCHY_LEVEL,:old.PARAMETER_GROUP,:old.ACTIVE,:old.REQUIRED,:old.DEFAULT_VALUE,:old.NAME,:old.SETTING_ID,:old.DEFAULT_UNIT,:old.SETTING_GROUP,:old.VALIDATION,:old.SUBTYPE,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELETE_SETTING_PERMISSION;

CREATE OR REPLACE TRIGGER DELETE_SETTING_PERMISSION
BEFORE DELETE
ON SETTING_PERMISSION
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
BEGIN
    INSERT INTO SETTING_PERMISSION_AUDIT
        (ID,SETTING_ID,PERMISSION_ID,WRITE,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY, CREATED_BY_USER, CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
        (:old.ID,:old.SETTING_ID,:old.PERMISSION_ID,:old.WRITE,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELETE_SETTING_VALUES;

CREATE OR REPLACE TRIGGER DELETE_SETTING_VALUES
BEFORE DELETE
ON SETTING_VALUES
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
BEGIN
 INSERT INTO SETTING_VALUES_AUDIT
        (ID,SETTING_ID,LOCATION_ID,PARAMETER_ID,VALUE,UNIT,MODIFIED,MODIFIED_BY,CREATED, CREATED_BY, CREATED_BY_USER, CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
        (:old.ID,:old.SETTING_ID,:old.LOCATION_ID,:old.PARAMETER_ID,:old.VALUE,:old.UNIT,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELETE_SIGNED_MSG;

CREATE OR REPLACE TRIGGER DELETE_SIGNED_MSG
before DELETE ON SIGNED_MSG
referencing OLD as OLD
for each row
BEGIN
	INSERT INTO SIGNED_MSG_AUDIT
   		(MSG_ID, MSG_NAME, PAT_ID, MESSAGE, SIGNATURE, MODIFIED, MODIFIED_BY, KEY_ID, OBSOLETE, REASON, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.MSG_ID, :old.MSG_NAME, :old.PAT_ID, :old.MESSAGE, :old.SIGNATURE,
		:old.MODIFIED, :old.MODIFIED_BY, :old.KEY_ID, :old.OBSOLETE, :old.REASON,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELETE_USR_PAT;

CREATE OR REPLACE TRIGGER DELETE_USR_PAT
            BEFORE DELETE ON USR_PAT
            REFERENCING NEW AS NEW OLD AS OLD
            FOR EACH ROW
BEGIN  
                INSERT INTO USR_PAT_AUDIT
                    ( PAT_ID,SIT_ID,CREATED,CREATED_BY,MODIFIED, MODIFIED_BY, SIGNED,INCLUDED,SCR_NO,PAT_NO,INITIALS,BIRTHDATE,WITHDRAWN_DATE,RANDOM_DATE,STATUS,STATUS_TS,MODIFIED_TIMESTAMP,MODIFIED_BY_USER,CREATED_TIMESTAMP,CREATED_BY_USER,PP_ID)
                VALUES
                    (:old.PAT_ID,:old.SIT_ID,:old.CREATED,:old.CREATED_BY,:old.MODIFIED,:old.MODIFIED_BY,:old.SIGNED,:old.INCLUDED,:old.SCR_NO,:old.PAT_NO,:old.INITIALS,:old.BIRTHDATE,:old.WITHDRAWN_DATE,:old.RANDOM_DATE,:old.STATUS,:old.STATUS_TS,:old.MODIFIED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.CREATED_TIMESTAMP,:old.CREATED_BY_USER,:old.PP_ID);
            END;
/


DROP TRIGGER DELETE_WAIVER;

CREATE OR REPLACE TRIGGER DELETE_WAIVER
BEFORE DELETE
ON WAIVER
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
BEGIN
    INSERT INTO WAIVER_AUDIT (ID,PAT_ID,VISIT_ID,SCHEDULE_ID,CONDITION_ID,REASON,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES (:old.ID,:old.PAT_ID,:old.VISIT_ID,:old.SCHEDULE_ID,:old.CONDITION_ID,:old.REASON,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
    INSERT INTO WAIVER_AUDIT (ID,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES (:old.ID,sysdate,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER DELET_NOTIF_OPT_OUT;

CREATE OR REPLACE TRIGGER DELET_NOTIF_OPT_OUT BEFORE DELETE ON NOTIF_OPT_OUT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO NOTIF_OPT_OUT_AUDIT (AUDIT_ID, ID,NOTIFICATION_ID,USR_ID,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NOTIFICATION_ID,:old.USR_ID,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DELET_NOTIF_ROLES;

CREATE OR REPLACE TRIGGER DELET_NOTIF_ROLES BEFORE DELETE ON NOTIF_ROLES REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO NOTIF_ROLES_AUDIT (AUDIT_ID, ID,NOTIFICATION_ID,ROLE_ID,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NOTIFICATION_ID,:old.ROLE_ID,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DELET_REPORT_PERMISSION;

CREATE OR REPLACE TRIGGER DELET_REPORT_PERMISSION BEFORE DELETE ON REPORT_PERMISSION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO REPORT_PERMISSION_AUDIT (AUDIT_ID, ID,REPORT_OBJECT_PATH,VISIT_ID,ROLE_ID_READ_ACCESS,DISABLED,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.REPORT_OBJECT_PATH,:old.VISIT_ID,:old.ROLE_ID_READ_ACCESS,:old.DISABLED,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_BATCH;

CREATE OR REPLACE TRIGGER DRUG_DELET_BATCH BEFORE DELETE ON DRUG_BATCH REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_BATCH (AUDIT_ID, ID,NAME,BLOCKED,MODIFIED,MODIFIED_AT,WAREHOUSE_BATCH_NAME,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.BLOCKED,:old.MODIFIED,:old.MODIFIED_AT,:old.WAREHOUSE_BATCH_NAME,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_BATCH_COUNTRY;

CREATE OR REPLACE TRIGGER DRUG_DELET_BATCH_COUNTRY BEFORE DELETE ON DRUG_BATCH_COUNTRY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_BATCH_COUNTRY (AUDIT_ID, ID,COUNTRY_ID,BATCH_ID,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.COUNTRY_ID,:old.BATCH_ID,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_COHORT;

CREATE OR REPLACE TRIGGER DRUG_DELET_COHORT BEFORE DELETE ON DRUG_COHORT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_COHORT (AUDIT_ID, ID,MODIFIED,MODIFIED_AT,NAME,DESCRIPTION,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.MODIFIED,:old.MODIFIED_AT,:old.NAME,:old.DESCRIPTION,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_COHORT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_DELET_COHORT_ATTRIB BEFORE DELETE ON DRUG_COHORT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_COHORT_ATTRIB (AUDIT_ID, COHORT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.COHORT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_CONFIGURATION;

CREATE OR REPLACE TRIGGER DRUG_DELET_CONFIGURATION BEFORE DELETE ON DRUG_CONFIGURATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_CONFIGURATION (AUDIT_ID, ID,NAME,VALUE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_COUNTRY_RELEASE;

CREATE OR REPLACE TRIGGER DRUG_DELET_COUNTRY_RELEASE BEFORE DELETE ON DRUG_COUNTRY_RELEASE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_COUNTRY_RELEASE (AUDIT_ID, ID,TRANSACTION_ID,COUNTRY_ID,PREQ_ID,UNBL_KIT_TYPE_ID,KIT_DESCR,MIN_RANGE,MAX_RANGE,DU_COUNT,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.TRANSACTION_ID,:old.COUNTRY_ID,:old.PREQ_ID,:old.UNBL_KIT_TYPE_ID,:old.KIT_DESCR,:old.MIN_RANGE,:old.MAX_RANGE,:old.DU_COUNT,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_COUNTRY_RELEASE_KIT;

CREATE OR REPLACE TRIGGER DRUG_DELET_COUNTRY_RELEASE_KIT BEFORE DELETE ON DRUG_COUNTRY_RELEASE_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_COUNTRY_RELEASE_KIT (AUDIT_ID, ID,COUNTRY_RELEASE_ID,KIT_ID,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.COUNTRY_RELEASE_ID,:old.KIT_ID,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_DATA_FILE;

CREATE OR REPLACE TRIGGER DRUG_DELET_DATA_FILE BEFORE DELETE ON DRUG_DATA_FILE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_DATA_FILE (AUDIT_ID, ID,NAME,SUPER_FOLDER_ID,CONTENT,COMPLETED_AT,COUNTER,CREATION_TYPE,SUBCLASS,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SUPER_FOLDER_ID,:old.CONTENT,:old.COMPLETED_AT,:old.COUNTER,:old.CREATION_TYPE,:old.SUBCLASS,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_DISPENSE_MATRIX;

CREATE OR REPLACE TRIGGER DRUG_DELET_DISPENSE_MATRIX BEFORE DELETE ON DRUG_DISPENSE_MATRIX REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_DISPENSE_MATRIX (AUDIT_ID, ID,VISIT_DESC,TREATMENTARM_ID,DOSE,QUANTITY_SELECTED,UNBL_KIT_TYPE,QUANTITY_TO_DISPENSE,QUANTITY_TO_PROJECT,REGION,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.VISIT_DESC,:old.TREATMENTARM_ID,:old.DOSE,:old.QUANTITY_SELECTED,:old.UNBL_KIT_TYPE,:old.QUANTITY_TO_DISPENSE,:old.QUANTITY_TO_PROJECT,:old.REGION,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_DOSE;

CREATE OR REPLACE TRIGGER DRUG_DELET_DOSE BEFORE DELETE ON DRUG_DOSE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_DOSE (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_DOSE_CUSTOM;

CREATE OR REPLACE TRIGGER DRUG_DELET_DOSE_CUSTOM BEFORE DELETE ON DRUG_DOSE_CUSTOM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_DOSE_CUSTOM (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_ENROLMENT_CAT;

CREATE OR REPLACE TRIGGER DRUG_DELET_ENROLMENT_CAT BEFORE DELETE ON DRUG_ENROLMENT_CAT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_ENROLMENT_CAT (AUDIT_ID, ID,NAME,IS_DEFAULT,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.IS_DEFAULT,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_ENROLMENT_LEVEL;

CREATE OR REPLACE TRIGGER DRUG_DELET_ENROLMENT_LEVEL BEFORE DELETE ON DRUG_ENROLMENT_LEVEL REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_ENROLMENT_LEVEL (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_EXPECTED_QTY;

CREATE OR REPLACE TRIGGER DRUG_DELET_EXPECTED_QTY BEFORE DELETE ON DRUG_EXPECTED_QTY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_EXPECTED_QTY (AUDIT_ID, ID,REQUESTED_QTY,UNBLINDED_KIT_TYPE_ID,SHIPMENT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.REQUESTED_QTY,:old.UNBLINDED_KIT_TYPE_ID,:old.SHIPMENT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_FACTOR;

CREATE OR REPLACE TRIGGER DRUG_DELET_FACTOR BEFORE DELETE ON DRUG_FACTOR REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_FACTOR (AUDIT_ID, ID,NAME,SHORT_NAME,FACTOR_TYPE,DESCRIPTION,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SHORT_NAME,:old.FACTOR_TYPE,:old.DESCRIPTION,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_FACTOR_CONFIG;

CREATE OR REPLACE TRIGGER DRUG_DELET_FACTOR_CONFIG BEFORE DELETE ON DRUG_FACTOR_CONFIG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_FACTOR_CONFIG (AUDIT_ID, NAME,FIELD,FORM,PAGE,MODIFIED,MODIFIED_AT,SHORT_NAME,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.NAME,:old.FIELD,:old.FORM,:old.PAGE,:old.MODIFIED,:old.MODIFIED_AT,:old.SHORT_NAME,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER DRUG_DELET_FACTOR_VALUE BEFORE DELETE ON DRUG_FACTOR_VALUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_FACTOR_VALUE (AUDIT_ID, ID,NAME,VALUE,MODIFIED,MODIFIED_AT,DESCRIPTION,RATIO,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.DESCRIPTION,:old.RATIO,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_KIT;

CREATE OR REPLACE TRIGGER DRUG_DELET_KIT BEFORE DELETE ON DRUG_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_KIT (AUDIT_ID, ID,SHIPMENT_ID,STATUS_ID,EXPIRATION_DATE,DISPENSE_DATE,PAT_ID,LOCATION_ID,BATCH_ID,MODIFIED,MODIFIED_AT,NAME,VERIFICATION_CODE,UNBL_KIT_TYPE_ID,VISIT_ID,KIT_TYPE_ID,QUARANTINED,PREVIOUS_STATUS_ID,VERIFICATION_STATUS_ID,COUNT,RECALL_DATE,SORT_KEY,FILEITEM_ID,MANUAL_DISPENSED,MANUFACTURING_LOT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP,REPLACES_KIT_ID,BLOCK_NUMBER, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SHIPMENT_ID,:old.STATUS_ID,:old.EXPIRATION_DATE,:old.DISPENSE_DATE,:old.PAT_ID,:old.LOCATION_ID,:old.BATCH_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.NAME,:old.VERIFICATION_CODE,:old.UNBL_KIT_TYPE_ID,:old.VISIT_ID,:old.KIT_TYPE_ID,:old.QUARANTINED,:old.PREVIOUS_STATUS_ID,:old.VERIFICATION_STATUS_ID,:old.COUNT,:old.RECALL_DATE,:old.SORT_KEY,:old.FILEITEM_ID,:old.MANUAL_DISPENSED,:old.MANUFACTURING_LOT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP,:old.REPLACES_KIT_ID,:old.BLOCK_NUMBER, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_KIT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_DELET_KIT_ATTRIB BEFORE DELETE ON DRUG_KIT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_KIT_ATTRIB (AUDIT_ID, KIT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.KIT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_KIT_FILEITEM;

CREATE OR REPLACE TRIGGER DRUG_DELET_KIT_FILEITEM BEFORE DELETE ON DRUG_KIT_FILEITEM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_KIT_FILEITEM (AUDIT_ID, ID,NAME,LINES,WORKER_ID,IMPORT_TIMESTAMP,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.LINES,:old.WORKER_ID,:old.IMPORT_TIMESTAMP,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_KIT_FILEITEM_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_DELET_KIT_FILEITEM_ATTRIB BEFORE DELETE ON DRUG_KIT_FILEITEM_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_KIT_FILEITEM_ATTRIB (AUDIT_ID, ID,FILEITEM_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.FILEITEM_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_KIT_PROJECT;

CREATE OR REPLACE TRIGGER DRUG_DELET_KIT_PROJECT BEFORE DELETE ON DRUG_KIT_PROJECT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_KIT_PROJECT (AUDIT_ID, ID,KIT_ID,PROJECT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.KIT_ID,:old.PROJECT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_KIT_TYPE;

CREATE OR REPLACE TRIGGER DRUG_DELET_KIT_TYPE BEFORE DELETE ON DRUG_KIT_TYPE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_KIT_TYPE (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,LABEL_MODE,DOSE,ENROLMENT_LEVEL_ID,SUBCLASS,BLINDED_KIT_TYPE_ID,DESCRIPTION,ADDRESS_TYPE_ID,TEMP_CONTROLLED,RANGE_LOW_TEMP,RANGE_HIGH_TEMP,EXTRA_KIT_LOGIC,NON_DRUG,BLOCK_SHIPMENT,BLOCK_SIZE,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.LABEL_MODE,:old.DOSE,:old.ENROLMENT_LEVEL_ID,:old.SUBCLASS,:old.BLINDED_KIT_TYPE_ID,:old.DESCRIPTION,:old.ADDRESS_TYPE_ID,:old.TEMP_CONTROLLED,:old.RANGE_LOW_TEMP,:old.RANGE_HIGH_TEMP,:old.EXTRA_KIT_LOGIC,:old.NON_DRUG,:old.BLOCK_SHIPMENT,:old.BLOCK_SIZE,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_KIT_TYPE_DESCRS;

CREATE OR REPLACE TRIGGER DRUG_DELET_KIT_TYPE_DESCRS BEFORE DELETE ON DRUG_KIT_TYPE_DESCRS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_KIT_TYPE_DESCRS (AUDIT_ID, ID,UNBL_KIT_TYPE_ID,BLINDED_DESCR,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.UNBL_KIT_TYPE_ID,:old.BLINDED_DESCR,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_LOCATION;

CREATE OR REPLACE TRIGGER DRUG_DELET_LOCATION BEFORE DELETE ON DRUG_LOCATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_LOCATION (AUDIT_ID, ID,STATUS_ID,ENROLMENT_CATEGORY_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP,LOCATION_SEQ_NUMBER, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.STATUS_ID,:old.ENROLMENT_CATEGORY_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP,:old.LOCATION_SEQ_NUMBER, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_LOCATION_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_DELET_LOCATION_ATTRIB BEFORE DELETE ON DRUG_LOCATION_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_LOCATION_ATTRIB (AUDIT_ID, LOCATION_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.LOCATION_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_NOTIFICATION;

CREATE OR REPLACE TRIGGER DRUG_DELET_NOTIFICATION BEFORE DELETE ON DRUG_NOTIFICATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_NOTIFICATION (AUDIT_ID, ID,NAME,CLASS_NAME,TRIGGER_ON_INSERT,TRIGGER_ON_UPDATE,PARAMETER,PARAMETER_ACCESS,NOTIFIER_OBJECT_PATH,LOCATION_QUERY_TYPE,LOCATION_QUERY,REPLACER_QUERY_TYPE,REPLACER_QUERY,TEXT_QUERY_TYPE,TEXT_QUERY,SUBJECT,PREAMBLE,TEXT,INTERVAL,INTERVAL_UNIT,LASTACTION,STARTDATE,DISABLE,IS_BLINDED,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP,ATTACHMENT_URL, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.CLASS_NAME,:old.TRIGGER_ON_INSERT,:old.TRIGGER_ON_UPDATE,:old.PARAMETER,:old.PARAMETER_ACCESS,:old.NOTIFIER_OBJECT_PATH,:old.LOCATION_QUERY_TYPE,:old.LOCATION_QUERY,:old.REPLACER_QUERY_TYPE,:old.REPLACER_QUERY,:old.TEXT_QUERY_TYPE,:old.TEXT_QUERY,:old.SUBJECT,:old.PREAMBLE,:old.TEXT,:old.INTERVAL,:old.INTERVAL_UNIT,:old.LASTACTION,:old.STARTDATE,:old.DISABLE,:old.IS_BLINDED,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP,:old.ATTACHMENT_URL, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_NOTIFICATION_QUEUE;

CREATE OR REPLACE TRIGGER DRUG_DELET_NOTIFICATION_QUEUE BEFORE DELETE ON DRUG_NOTIFICATION_QUEUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_NOTIFICATION_QUEUE (AUDIT_ID, ID,NOTIFICATION,LOCATION_ID,NOTIFIER_OBJECT_ID,LOCATION_QUERY_RESULT,NODE,USR_ID,DELAYED_UNTIL,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NOTIFICATION,:old.LOCATION_ID,:old.NOTIFIER_OBJECT_ID,:old.LOCATION_QUERY_RESULT,:old.NODE,:old.USR_ID,:old.DELAYED_UNTIL,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_PAT_VST_QTY_REQ;

CREATE OR REPLACE TRIGGER DRUG_DELET_PAT_VST_QTY_REQ BEFORE DELETE ON DRUG_PAT_VST_QTY_REQ REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_PAT_VST_QTY_REQ (AUDIT_ID, ID,PAT_ID,UNBL_KIT_TYPE_ID,QUANTITY,VISIT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PAT_ID,:old.UNBL_KIT_TYPE_ID,:old.QUANTITY,:old.VISIT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_PRIORITY_ASSIGNMENT;

CREATE OR REPLACE TRIGGER DRUG_DELET_PRIORITY_ASSIGNMENT BEFORE DELETE ON DRUG_PRIORITY_ASSIGNMENT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_PRIORITY_ASSIGNMENT (AUDIT_ID, ID,PRIORITY,UNBLINDED_KIT_TYPE_ID,QUANTITY,DOSE_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PRIORITY,:old.UNBLINDED_KIT_TYPE_ID,:old.QUANTITY,:old.DOSE_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_PRIORITY_SUPPLY;

CREATE OR REPLACE TRIGGER DRUG_DELET_PRIORITY_SUPPLY BEFORE DELETE ON DRUG_PRIORITY_SUPPLY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_PRIORITY_SUPPLY (AUDIT_ID, ID,PRIORITY,UNBL_KIT_TYPE_ID_FROM,UNBL_KIT_TYPE_ID_TO,QUANTITY,COUNTRY_ID,ENABLED,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PRIORITY,:old.UNBL_KIT_TYPE_ID_FROM,:old.UNBL_KIT_TYPE_ID_TO,:old.QUANTITY,:old.COUNTRY_ID,:old.ENABLED,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RNDNO;

CREATE OR REPLACE TRIGGER DRUG_DELET_RNDNO BEFORE DELETE ON DRUG_RNDNO REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RNDNO (AUDIT_ID, ID,PAT_NO,STATUS_ID,TREATMENTARM_ID,MODIFIED,MODIFIED_AT,PAT_ID,VISIT_ID,UNBLINDED_AT,REASON_UNBLINDING,RANDOM_SOURCE,SORT_KEY,RANDOMIZATION_ORDER,FILEITEM_ID,BLOCK,BLOCK_ASSIGNED,COHORT_ID,RANDOM_WEIGHT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PAT_NO,:old.STATUS_ID,:old.TREATMENTARM_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.PAT_ID,:old.VISIT_ID,:old.UNBLINDED_AT,:old.REASON_UNBLINDING,:old.RANDOM_SOURCE,:old.SORT_KEY,:old.RANDOMIZATION_ORDER,:old.FILEITEM_ID,:old.BLOCK,:old.BLOCK_ASSIGNED,:old.COHORT_ID,:old.RANDOM_WEIGHT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RNDNO_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_DELET_RNDNO_ATTRIB BEFORE DELETE ON DRUG_RNDNO_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RNDNO_ATTRIB (AUDIT_ID, RNDNO_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.RNDNO_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RNDNO_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER DRUG_DELET_RNDNO_FACTOR_VALUE BEFORE DELETE ON DRUG_RNDNO_FACTOR_VALUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RNDNO_FACTOR_VALUE (AUDIT_ID, RNDNO_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.RNDNO_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RNDNO_FILEITEM;

CREATE OR REPLACE TRIGGER DRUG_DELET_RNDNO_FILEITEM BEFORE DELETE ON DRUG_RNDNO_FILEITEM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RNDNO_FILEITEM (AUDIT_ID, ID,NAME,LINES,WORKER_ID,IMPORT_TIMESTAMP,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.LINES,:old.WORKER_ID,:old.IMPORT_TIMESTAMP,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RNDNO_FILEITEM_ATTR;

CREATE OR REPLACE TRIGGER DRUG_DELET_RNDNO_FILEITEM_ATTR BEFORE DELETE ON DRUG_RNDNO_FILEITEM_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RNDNO_FILEITEM_ATTR (AUDIT_ID, ID,FILEITEM_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.FILEITEM_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RND_CONFIG;

CREATE OR REPLACE TRIGGER DRUG_DELET_RND_CONFIG BEFORE DELETE ON DRUG_RND_CONFIG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RND_CONFIG (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,RND_METHOD_ID,VALUE,IS_ACTIVE,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.RND_METHOD_ID,:old.VALUE,:old.IS_ACTIVE,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RND_METHOD;

CREATE OR REPLACE TRIGGER DRUG_DELET_RND_METHOD BEFORE DELETE ON DRUG_RND_METHOD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RND_METHOD (AUDIT_ID, ID,RND_METHOD_NAME,RND_METHOD,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.RND_METHOD_NAME,:old.RND_METHOD,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_RND_NUM_GENERATOR;

CREATE OR REPLACE TRIGGER DRUG_DELET_RND_NUM_GENERATOR BEFORE DELETE ON DRUG_RND_NUM_GENERATOR REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_RND_NUM_GENERATOR (AUDIT_ID, ID,RND_NUM_GENERATOR_NAME,RND_NUM_GENERATOR,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.RND_NUM_GENERATOR_NAME,:old.RND_NUM_GENERATOR,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_SCHEDULE;

CREATE OR REPLACE TRIGGER DRUG_DELET_SCHEDULE BEFORE DELETE ON DRUG_SCHEDULE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_SCHEDULE (AUDIT_ID, ID,SUBCLASS,TREATMENTARM_ID,VISIT_ID,UNBLINDED_KIT_TYPE_ID,DISPENSE_QTY,PROJECTION_QTY,DOSE_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SUBCLASS,:old.TREATMENTARM_ID,:old.VISIT_ID,:old.UNBLINDED_KIT_TYPE_ID,:old.DISPENSE_QTY,:old.PROJECTION_QTY,:old.DOSE_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_SERVICE;

CREATE OR REPLACE TRIGGER DRUG_DELET_SERVICE BEFORE DELETE ON DRUG_SERVICE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_SERVICE (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_SHIPMENT;

CREATE OR REPLACE TRIGGER DRUG_DELET_SHIPMENT BEFORE DELETE ON DRUG_SHIPMENT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_SHIPMENT (AUDIT_ID, ID,STATUS_ID,SENDER,FROM_LOCATION_ID,TO_LOCATION_ID,SERVICE_ID,SHIP_DATE,REQUEST_DATE,ACKNOWLEDGEMENT_DATE,LOCATION_ADDRESS_ID,TEMP_CONTROLLED,TEMP_OUT_OF_RANGE,MODIFIED,MODIFIED_AT,NAME,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.STATUS_ID,:old.SENDER,:old.FROM_LOCATION_ID,:old.TO_LOCATION_ID,:old.SERVICE_ID,:old.SHIP_DATE,:old.REQUEST_DATE,:old.ACKNOWLEDGEMENT_DATE,:old.LOCATION_ADDRESS_ID,:old.TEMP_CONTROLLED,:old.TEMP_OUT_OF_RANGE,:old.MODIFIED,:old.MODIFIED_AT,:old.NAME,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_SHIPMENT_ADDRESS;

CREATE OR REPLACE TRIGGER DRUG_DELET_SHIPMENT_ADDRESS BEFORE DELETE ON DRUG_SHIPMENT_ADDRESS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_SHIPMENT_ADDRESS (AUDIT_ID, ID,SHIPMENT_ID,FNAME,MNAME,LNAME,TITLE,ORGANISATION,STREET,CITY,STATE,ZIP,COUNTRY_ID,PHONE,MPHONE,FAX,EMAIL,NOTES,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SHIPMENT_ID,:old.FNAME,:old.MNAME,:old.LNAME,:old.TITLE,:old.ORGANISATION,:old.STREET,:old.CITY,:old.STATE,:old.ZIP,:old.COUNTRY_ID,:old.PHONE,:old.MPHONE,:old.FAX,:old.EMAIL,:old.NOTES,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_SHIPMENT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_DELET_SHIPMENT_ATTRIB BEFORE DELETE ON DRUG_SHIPMENT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_SHIPMENT_ATTRIB (AUDIT_ID, SHIPMENT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.SHIPMENT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_SHIPMENT_FILE;

CREATE OR REPLACE TRIGGER DRUG_DELET_SHIPMENT_FILE BEFORE DELETE ON DRUG_SHIPMENT_FILE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_SHIPMENT_FILE (AUDIT_ID, ID,NAME,SHIPMENT_ID,DESCRIPTION,MODIFIED_AT,MODIFIED,FILENAME,CONTENT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SHIPMENT_ID,:old.DESCRIPTION,:old.MODIFIED_AT,:old.MODIFIED,:old.FILENAME,:old.CONTENT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_STATUS;

CREATE OR REPLACE TRIGGER DRUG_DELET_STATUS BEFORE DELETE ON DRUG_STATUS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_STATUS (AUDIT_ID, ID,NAME,SUBCLASS,LOCATION_TYPE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SUBCLASS,:old.LOCATION_TYPE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_SUPPLY_CONF;

CREATE OR REPLACE TRIGGER DRUG_DELET_SUPPLY_CONF BEFORE DELETE ON DRUG_SUPPLY_CONF REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_SUPPLY_CONF (AUDIT_ID, ID,ENROLMENT_LEVEL_ID,ENROLMENT_CATEGORY_ID,WAREHOUSE_ID,INITIAL_VALUE,MIN_VALUE,MAX_VALUE,SUBCLASS,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.ENROLMENT_LEVEL_ID,:old.ENROLMENT_CATEGORY_ID,:old.WAREHOUSE_ID,:old.INITIAL_VALUE,:old.MIN_VALUE,:old.MAX_VALUE,:old.SUBCLASS,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_TRACK_STATS_LBLD;

CREATE OR REPLACE TRIGGER DRUG_DELET_TRACK_STATS_LBLD BEFORE DELETE ON DRUG_TRACK_STATS_LBLD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_TRACK_STATS_LBLD (AUDIT_ID, ID,SHIPMENT_ID,KIT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SHIPMENT_ID,:old.KIT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_TRACK_STATS_UNLBLD;

CREATE OR REPLACE TRIGGER DRUG_DELET_TRACK_STATS_UNLBLD BEFORE DELETE ON DRUG_TRACK_STATS_UNLBLD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_TRACK_STATS_UNLBLD (AUDIT_ID, ID,COUNT,KIT_AMOUNT,SHIPMENT_ID,BATCH_ID,KIT_TYPE_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.COUNT,:old.KIT_AMOUNT,:old.SHIPMENT_ID,:old.BATCH_ID,:old.KIT_TYPE_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_TRANSITION;

CREATE OR REPLACE TRIGGER DRUG_DELET_TRANSITION BEFORE DELETE ON DRUG_TRANSITION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_TRANSITION (AUDIT_ID, FROM_STATUS_ID,TO_STATUS_ID,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.FROM_STATUS_ID,:old.TO_STATUS_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_TREATMENTARM;

CREATE OR REPLACE TRIGGER DRUG_DELET_TREATMENTARM BEFORE DELETE ON DRUG_TREATMENTARM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_TREATMENTARM (AUDIT_ID, ID,NAME,DESCRIPTION,MODIFIED,MODIFIED_AT,STATUS_ID,DOSE,TREATMENTARM_TYPE,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.DESCRIPTION,:old.MODIFIED,:old.MODIFIED_AT,:old.STATUS_ID,:old.DOSE,:old.TREATMENTARM_TYPE,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_UNBLINDING_LOG;

CREATE OR REPLACE TRIGGER DRUG_DELET_UNBLINDING_LOG BEFORE DELETE ON DRUG_UNBLINDING_LOG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_UNBLINDING_LOG (AUDIT_ID, ID,WORKER_ID,RNDNO_ID,REASON_UNBLINDING,TYPE,UNBLINDED_AT,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.WORKER_ID,:old.RNDNO_ID,:old.REASON_UNBLINDING,:old.TYPE,:old.UNBLINDED_AT,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_UNPACKAGED_KIT;

CREATE OR REPLACE TRIGGER DRUG_DELET_UNPACKAGED_KIT BEFORE DELETE ON DRUG_UNPACKAGED_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_UNPACKAGED_KIT (AUDIT_ID, ID,NAME,VERIFICATION_CODE,SORT_KEY,COUNT,KIT_TYPE_ID,UNBL_KIT_TYPE_ID,FILEITEM_ID,BLOCK_NUMBER,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.VERIFICATION_CODE,:old.SORT_KEY,:old.COUNT,:old.KIT_TYPE_ID,:old.UNBL_KIT_TYPE_ID,:old.FILEITEM_ID,:old.BLOCK_NUMBER,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_DELET_UNPACKAGED_KIT_ATTR;

CREATE OR REPLACE TRIGGER DRUG_DELET_UNPACKAGED_KIT_ATTR BEFORE DELETE ON DRUG_UNPACKAGED_KIT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
BEGIN  INSERT INTO DRUG_AUDIT_UNPACKAGED_KIT_ATTR (AUDIT_ID, UNPACKAGED_KIT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.UNPACKAGED_KIT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'DELETE', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_INS_BATCH;

CREATE OR REPLACE TRIGGER DRUG_INS_BATCH BEFORE INSERT ON DRUG_BATCH REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_BATCH_COUNTRY;

CREATE OR REPLACE TRIGGER DRUG_INS_BATCH_COUNTRY BEFORE INSERT ON DRUG_BATCH_COUNTRY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_COHORT;

CREATE OR REPLACE TRIGGER DRUG_INS_COHORT BEFORE INSERT ON DRUG_COHORT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_COHORT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_INS_COHORT_ATTRIB BEFORE INSERT ON DRUG_COHORT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_CONFIGURATION;

CREATE OR REPLACE TRIGGER DRUG_INS_CONFIGURATION BEFORE INSERT ON DRUG_CONFIGURATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_COUNTRY_RELEASE;

CREATE OR REPLACE TRIGGER DRUG_INS_COUNTRY_RELEASE BEFORE INSERT ON DRUG_COUNTRY_RELEASE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_COUNTRY_RELEASE_KIT;

CREATE OR REPLACE TRIGGER DRUG_INS_COUNTRY_RELEASE_KIT BEFORE INSERT ON DRUG_COUNTRY_RELEASE_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_DATA_FILE;

CREATE OR REPLACE TRIGGER DRUG_INS_DATA_FILE BEFORE INSERT ON DRUG_DATA_FILE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_DISPENSE_MATRIX;

CREATE OR REPLACE TRIGGER DRUG_INS_DISPENSE_MATRIX BEFORE INSERT ON DRUG_DISPENSE_MATRIX REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_DOSE;

CREATE OR REPLACE TRIGGER DRUG_INS_DOSE BEFORE INSERT ON DRUG_DOSE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_DOSE_CUSTOM;

CREATE OR REPLACE TRIGGER DRUG_INS_DOSE_CUSTOM BEFORE INSERT ON DRUG_DOSE_CUSTOM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_ENROLMENT_CAT;

CREATE OR REPLACE TRIGGER DRUG_INS_ENROLMENT_CAT BEFORE INSERT ON DRUG_ENROLMENT_CAT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_ENROLMENT_LEVEL;

CREATE OR REPLACE TRIGGER DRUG_INS_ENROLMENT_LEVEL BEFORE INSERT ON DRUG_ENROLMENT_LEVEL REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_EXPECTED_QTY;

CREATE OR REPLACE TRIGGER DRUG_INS_EXPECTED_QTY BEFORE INSERT ON DRUG_EXPECTED_QTY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_FACTOR;

CREATE OR REPLACE TRIGGER DRUG_INS_FACTOR BEFORE INSERT ON DRUG_FACTOR REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_FACTOR_CONFIG;

CREATE OR REPLACE TRIGGER DRUG_INS_FACTOR_CONFIG BEFORE INSERT ON DRUG_FACTOR_CONFIG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER DRUG_INS_FACTOR_VALUE BEFORE INSERT ON DRUG_FACTOR_VALUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_KIT;

CREATE OR REPLACE TRIGGER DRUG_INS_KIT BEFORE INSERT ON DRUG_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_KIT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_INS_KIT_ATTRIB BEFORE INSERT ON DRUG_KIT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_KIT_FILEITEM;

CREATE OR REPLACE TRIGGER DRUG_INS_KIT_FILEITEM BEFORE INSERT ON DRUG_KIT_FILEITEM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_KIT_FILEITEM_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_INS_KIT_FILEITEM_ATTRIB BEFORE INSERT ON DRUG_KIT_FILEITEM_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_KIT_PROJECT;

CREATE OR REPLACE TRIGGER DRUG_INS_KIT_PROJECT BEFORE INSERT ON DRUG_KIT_PROJECT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_KIT_TYPE;

CREATE OR REPLACE TRIGGER DRUG_INS_KIT_TYPE BEFORE INSERT ON DRUG_KIT_TYPE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_KIT_TYPE_DESCRS;

CREATE OR REPLACE TRIGGER DRUG_INS_KIT_TYPE_DESCRS BEFORE INSERT ON DRUG_KIT_TYPE_DESCRS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_LOCATION;

CREATE OR REPLACE TRIGGER DRUG_INS_LOCATION BEFORE INSERT ON DRUG_LOCATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_LOCATION_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_INS_LOCATION_ATTRIB BEFORE INSERT ON DRUG_LOCATION_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_NOTIFICATION;

CREATE OR REPLACE TRIGGER DRUG_INS_NOTIFICATION BEFORE INSERT ON DRUG_NOTIFICATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_NOTIFICATION_QUEUE;

CREATE OR REPLACE TRIGGER DRUG_INS_NOTIFICATION_QUEUE BEFORE INSERT ON DRUG_NOTIFICATION_QUEUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_PAT_VST_QTY_REQ;

CREATE OR REPLACE TRIGGER DRUG_INS_PAT_VST_QTY_REQ BEFORE INSERT ON DRUG_PAT_VST_QTY_REQ REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_PRIORITY_ASSIGNMENT;

CREATE OR REPLACE TRIGGER DRUG_INS_PRIORITY_ASSIGNMENT BEFORE INSERT ON DRUG_PRIORITY_ASSIGNMENT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_PRIORITY_SUPPLY;

CREATE OR REPLACE TRIGGER DRUG_INS_PRIORITY_SUPPLY BEFORE INSERT ON DRUG_PRIORITY_SUPPLY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RNDNO;

CREATE OR REPLACE TRIGGER DRUG_INS_RNDNO BEFORE INSERT ON DRUG_RNDNO REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RNDNO_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_INS_RNDNO_ATTRIB BEFORE INSERT ON DRUG_RNDNO_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RNDNO_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER DRUG_INS_RNDNO_FACTOR_VALUE BEFORE INSERT ON DRUG_RNDNO_FACTOR_VALUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RNDNO_FILEITEM;

CREATE OR REPLACE TRIGGER DRUG_INS_RNDNO_FILEITEM BEFORE INSERT ON DRUG_RNDNO_FILEITEM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RNDNO_FILEITEM_ATTR;

CREATE OR REPLACE TRIGGER DRUG_INS_RNDNO_FILEITEM_ATTR BEFORE INSERT ON DRUG_RNDNO_FILEITEM_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RND_CONFIG;

CREATE OR REPLACE TRIGGER DRUG_INS_RND_CONFIG BEFORE INSERT ON DRUG_RND_CONFIG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RND_METHOD;

CREATE OR REPLACE TRIGGER DRUG_INS_RND_METHOD BEFORE INSERT ON DRUG_RND_METHOD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_RND_NUM_GENERATOR;

CREATE OR REPLACE TRIGGER DRUG_INS_RND_NUM_GENERATOR BEFORE INSERT ON DRUG_RND_NUM_GENERATOR REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_SCHEDULE;

CREATE OR REPLACE TRIGGER DRUG_INS_SCHEDULE BEFORE INSERT ON DRUG_SCHEDULE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_SERVICE;

CREATE OR REPLACE TRIGGER DRUG_INS_SERVICE BEFORE INSERT ON DRUG_SERVICE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_SHIPMENT;

CREATE OR REPLACE TRIGGER DRUG_INS_SHIPMENT BEFORE INSERT ON DRUG_SHIPMENT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_SHIPMENT_ADDRESS;

CREATE OR REPLACE TRIGGER DRUG_INS_SHIPMENT_ADDRESS BEFORE INSERT ON DRUG_SHIPMENT_ADDRESS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_SHIPMENT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_INS_SHIPMENT_ATTRIB BEFORE INSERT ON DRUG_SHIPMENT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_SHIPMENT_FILE;

CREATE OR REPLACE TRIGGER DRUG_INS_SHIPMENT_FILE BEFORE INSERT ON DRUG_SHIPMENT_FILE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_STATUS;

CREATE OR REPLACE TRIGGER DRUG_INS_STATUS BEFORE INSERT ON DRUG_STATUS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_SUPPLY_CONF;

CREATE OR REPLACE TRIGGER DRUG_INS_SUPPLY_CONF BEFORE INSERT ON DRUG_SUPPLY_CONF REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_TRACK_STATS_LBLD;

CREATE OR REPLACE TRIGGER DRUG_INS_TRACK_STATS_LBLD BEFORE INSERT ON DRUG_TRACK_STATS_LBLD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_TRACK_STATS_UNLBLD;

CREATE OR REPLACE TRIGGER DRUG_INS_TRACK_STATS_UNLBLD BEFORE INSERT ON DRUG_TRACK_STATS_UNLBLD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_TRANSITION;

CREATE OR REPLACE TRIGGER DRUG_INS_TRANSITION BEFORE INSERT ON DRUG_TRANSITION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_TREATMENTARM;

CREATE OR REPLACE TRIGGER DRUG_INS_TREATMENTARM BEFORE INSERT ON DRUG_TREATMENTARM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_UNBLINDING_LOG;

CREATE OR REPLACE TRIGGER DRUG_INS_UNBLINDING_LOG BEFORE INSERT ON DRUG_UNBLINDING_LOG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_UNPACKAGED_KIT;

CREATE OR REPLACE TRIGGER DRUG_INS_UNPACKAGED_KIT BEFORE INSERT ON DRUG_UNPACKAGED_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_INS_UNPACKAGED_KIT_ATTR;

CREATE OR REPLACE TRIGGER DRUG_INS_UNPACKAGED_KIT_ATTR BEFORE INSERT ON DRUG_UNPACKAGED_KIT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER DRUG_MODIF_BATCH;

CREATE OR REPLACE TRIGGER DRUG_MODIF_BATCH BEFORE UPDATE ON DRUG_BATCH REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_BATCH (AUDIT_ID, ID,NAME,BLOCKED,MODIFIED,MODIFIED_AT,WAREHOUSE_BATCH_NAME,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.BLOCKED,:old.MODIFIED,:old.MODIFIED_AT,:old.WAREHOUSE_BATCH_NAME,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_BATCH_COUNTRY;

CREATE OR REPLACE TRIGGER DRUG_MODIF_BATCH_COUNTRY BEFORE UPDATE ON DRUG_BATCH_COUNTRY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_BATCH_COUNTRY (AUDIT_ID, ID,COUNTRY_ID,BATCH_ID,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.COUNTRY_ID,:old.BATCH_ID,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_COHORT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_COHORT BEFORE UPDATE ON DRUG_COHORT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_COHORT (AUDIT_ID, ID,MODIFIED,MODIFIED_AT,NAME,DESCRIPTION,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.MODIFIED,:old.MODIFIED_AT,:old.NAME,:old.DESCRIPTION,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_COHORT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_MODIF_COHORT_ATTRIB BEFORE UPDATE ON DRUG_COHORT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_COHORT_ATTRIB (AUDIT_ID, COHORT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.COHORT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_CONFIGURATION;

CREATE OR REPLACE TRIGGER DRUG_MODIF_CONFIGURATION BEFORE UPDATE ON DRUG_CONFIGURATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_CONFIGURATION (AUDIT_ID, ID,NAME,VALUE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_COUNTRY_RELEASE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_COUNTRY_RELEASE BEFORE UPDATE ON DRUG_COUNTRY_RELEASE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_COUNTRY_RELEASE (AUDIT_ID, ID,TRANSACTION_ID,COUNTRY_ID,PREQ_ID,UNBL_KIT_TYPE_ID,KIT_DESCR,MIN_RANGE,MAX_RANGE,DU_COUNT,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.TRANSACTION_ID,:old.COUNTRY_ID,:old.PREQ_ID,:old.UNBL_KIT_TYPE_ID,:old.KIT_DESCR,:old.MIN_RANGE,:old.MAX_RANGE,:old.DU_COUNT,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_COUNTRY_RELEASE_KIT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_COUNTRY_RELEASE_KIT BEFORE UPDATE ON DRUG_COUNTRY_RELEASE_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_COUNTRY_RELEASE_KIT (AUDIT_ID, ID,COUNTRY_RELEASE_ID,KIT_ID,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.COUNTRY_RELEASE_ID,:old.KIT_ID,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_DATA_FILE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_DATA_FILE BEFORE UPDATE ON DRUG_DATA_FILE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_DATA_FILE (AUDIT_ID, ID,NAME,SUPER_FOLDER_ID,CONTENT,COMPLETED_AT,COUNTER,CREATION_TYPE,SUBCLASS,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SUPER_FOLDER_ID,:old.CONTENT,:old.COMPLETED_AT,:old.COUNTER,:old.CREATION_TYPE,:old.SUBCLASS,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_DISPENSE_MATRIX;

CREATE OR REPLACE TRIGGER DRUG_MODIF_DISPENSE_MATRIX BEFORE UPDATE ON DRUG_DISPENSE_MATRIX REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_DISPENSE_MATRIX (AUDIT_ID, ID,VISIT_DESC,TREATMENTARM_ID,DOSE,QUANTITY_SELECTED,UNBL_KIT_TYPE,QUANTITY_TO_DISPENSE,QUANTITY_TO_PROJECT,REGION,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.VISIT_DESC,:old.TREATMENTARM_ID,:old.DOSE,:old.QUANTITY_SELECTED,:old.UNBL_KIT_TYPE,:old.QUANTITY_TO_DISPENSE,:old.QUANTITY_TO_PROJECT,:old.REGION,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_DOSE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_DOSE BEFORE UPDATE ON DRUG_DOSE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_DOSE (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_DOSE_CUSTOM;

CREATE OR REPLACE TRIGGER DRUG_MODIF_DOSE_CUSTOM BEFORE UPDATE ON DRUG_DOSE_CUSTOM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_DOSE_CUSTOM (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_ENROLMENT_CAT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_ENROLMENT_CAT BEFORE UPDATE ON DRUG_ENROLMENT_CAT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_ENROLMENT_CAT (AUDIT_ID, ID,NAME,IS_DEFAULT,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.IS_DEFAULT,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_ENROLMENT_LEVEL;

CREATE OR REPLACE TRIGGER DRUG_MODIF_ENROLMENT_LEVEL BEFORE UPDATE ON DRUG_ENROLMENT_LEVEL REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_ENROLMENT_LEVEL (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_EXPECTED_QTY;

CREATE OR REPLACE TRIGGER DRUG_MODIF_EXPECTED_QTY BEFORE UPDATE ON DRUG_EXPECTED_QTY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_EXPECTED_QTY (AUDIT_ID, ID,REQUESTED_QTY,UNBLINDED_KIT_TYPE_ID,SHIPMENT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.REQUESTED_QTY,:old.UNBLINDED_KIT_TYPE_ID,:old.SHIPMENT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_FACTOR;

CREATE OR REPLACE TRIGGER DRUG_MODIF_FACTOR BEFORE UPDATE ON DRUG_FACTOR REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_FACTOR (AUDIT_ID, ID,NAME,SHORT_NAME,FACTOR_TYPE,DESCRIPTION,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SHORT_NAME,:old.FACTOR_TYPE,:old.DESCRIPTION,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_FACTOR_CONFIG;

CREATE OR REPLACE TRIGGER DRUG_MODIF_FACTOR_CONFIG BEFORE UPDATE ON DRUG_FACTOR_CONFIG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_FACTOR_CONFIG (AUDIT_ID, NAME,FIELD,FORM,PAGE,MODIFIED,MODIFIED_AT,SHORT_NAME,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.NAME,:old.FIELD,:old.FORM,:old.PAGE,:old.MODIFIED,:old.MODIFIED_AT,:old.SHORT_NAME,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_FACTOR_VALUE BEFORE UPDATE ON DRUG_FACTOR_VALUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_FACTOR_VALUE (AUDIT_ID, ID,NAME,VALUE,MODIFIED,MODIFIED_AT,DESCRIPTION,RATIO,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.DESCRIPTION,:old.RATIO,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_KIT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_KIT BEFORE UPDATE ON DRUG_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_KIT (AUDIT_ID, ID,SHIPMENT_ID,STATUS_ID,EXPIRATION_DATE,DISPENSE_DATE,PAT_ID,LOCATION_ID,BATCH_ID,MODIFIED,MODIFIED_AT,NAME,VERIFICATION_CODE,UNBL_KIT_TYPE_ID,VISIT_ID,KIT_TYPE_ID,QUARANTINED,PREVIOUS_STATUS_ID,VERIFICATION_STATUS_ID,COUNT,RECALL_DATE,SORT_KEY,FILEITEM_ID,MANUAL_DISPENSED,MANUFACTURING_LOT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP,REPLACES_KIT_ID,BLOCK_NUMBER, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SHIPMENT_ID,:old.STATUS_ID,:old.EXPIRATION_DATE,:old.DISPENSE_DATE,:old.PAT_ID,:old.LOCATION_ID,:old.BATCH_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.NAME,:old.VERIFICATION_CODE,:old.UNBL_KIT_TYPE_ID,:old.VISIT_ID,:old.KIT_TYPE_ID,:old.QUARANTINED,:old.PREVIOUS_STATUS_ID,:old.VERIFICATION_STATUS_ID,:old.COUNT,:old.RECALL_DATE,:old.SORT_KEY,:old.FILEITEM_ID,:old.MANUAL_DISPENSED,:old.MANUFACTURING_LOT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP,:old.REPLACES_KIT_ID,:old.BLOCK_NUMBER, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_KIT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_MODIF_KIT_ATTRIB BEFORE UPDATE ON DRUG_KIT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_KIT_ATTRIB (AUDIT_ID, KIT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.KIT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_KIT_FILEITEM;

CREATE OR REPLACE TRIGGER DRUG_MODIF_KIT_FILEITEM BEFORE UPDATE ON DRUG_KIT_FILEITEM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_KIT_FILEITEM (AUDIT_ID, ID,NAME,LINES,WORKER_ID,IMPORT_TIMESTAMP,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.LINES,:old.WORKER_ID,:old.IMPORT_TIMESTAMP,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_KIT_FILEITEM_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_MODIF_KIT_FILEITEM_ATTRIB BEFORE UPDATE ON DRUG_KIT_FILEITEM_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_KIT_FILEITEM_ATTRIB (AUDIT_ID, ID,FILEITEM_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.FILEITEM_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_KIT_PROJECT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_KIT_PROJECT BEFORE UPDATE ON DRUG_KIT_PROJECT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_KIT_PROJECT (AUDIT_ID, ID,KIT_ID,PROJECT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.KIT_ID,:old.PROJECT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_KIT_TYPE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_KIT_TYPE BEFORE UPDATE ON DRUG_KIT_TYPE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_KIT_TYPE (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,LABEL_MODE,DOSE,ENROLMENT_LEVEL_ID,SUBCLASS,BLINDED_KIT_TYPE_ID,DESCRIPTION,ADDRESS_TYPE_ID,TEMP_CONTROLLED,RANGE_LOW_TEMP,RANGE_HIGH_TEMP,EXTRA_KIT_LOGIC,NON_DRUG,BLOCK_SHIPMENT,BLOCK_SIZE,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.LABEL_MODE,:old.DOSE,:old.ENROLMENT_LEVEL_ID,:old.SUBCLASS,:old.BLINDED_KIT_TYPE_ID,:old.DESCRIPTION,:old.ADDRESS_TYPE_ID,:old.TEMP_CONTROLLED,:old.RANGE_LOW_TEMP,:old.RANGE_HIGH_TEMP,:old.EXTRA_KIT_LOGIC,:old.NON_DRUG,:old.BLOCK_SHIPMENT,:old.BLOCK_SIZE,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_KIT_TYPE_DESCRS;

CREATE OR REPLACE TRIGGER DRUG_MODIF_KIT_TYPE_DESCRS BEFORE UPDATE ON DRUG_KIT_TYPE_DESCRS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_KIT_TYPE_DESCRS (AUDIT_ID, ID,UNBL_KIT_TYPE_ID,BLINDED_DESCR,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.UNBL_KIT_TYPE_ID,:old.BLINDED_DESCR,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_LOCATION;

CREATE OR REPLACE TRIGGER DRUG_MODIF_LOCATION BEFORE UPDATE ON DRUG_LOCATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_LOCATION (AUDIT_ID, ID,STATUS_ID,ENROLMENT_CATEGORY_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP,LOCATION_SEQ_NUMBER, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.STATUS_ID,:old.ENROLMENT_CATEGORY_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP,:old.LOCATION_SEQ_NUMBER, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_LOCATION_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_MODIF_LOCATION_ATTRIB BEFORE UPDATE ON DRUG_LOCATION_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_LOCATION_ATTRIB (AUDIT_ID, LOCATION_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.LOCATION_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_NOTIFICATION;

CREATE OR REPLACE TRIGGER DRUG_MODIF_NOTIFICATION BEFORE UPDATE ON DRUG_NOTIFICATION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_NOTIFICATION (AUDIT_ID, ID,NAME,CLASS_NAME,TRIGGER_ON_INSERT,TRIGGER_ON_UPDATE,PARAMETER,PARAMETER_ACCESS,NOTIFIER_OBJECT_PATH,LOCATION_QUERY_TYPE,LOCATION_QUERY,REPLACER_QUERY_TYPE,REPLACER_QUERY,TEXT_QUERY_TYPE,TEXT_QUERY,SUBJECT,PREAMBLE,TEXT,INTERVAL,INTERVAL_UNIT,LASTACTION,STARTDATE,DISABLE,IS_BLINDED,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP,ATTACHMENT_URL, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.CLASS_NAME,:old.TRIGGER_ON_INSERT,:old.TRIGGER_ON_UPDATE,:old.PARAMETER,:old.PARAMETER_ACCESS,:old.NOTIFIER_OBJECT_PATH,:old.LOCATION_QUERY_TYPE,:old.LOCATION_QUERY,:old.REPLACER_QUERY_TYPE,:old.REPLACER_QUERY,:old.TEXT_QUERY_TYPE,:old.TEXT_QUERY,:old.SUBJECT,:old.PREAMBLE,:old.TEXT,:old.INTERVAL,:old.INTERVAL_UNIT,:old.LASTACTION,:old.STARTDATE,:old.DISABLE,:old.IS_BLINDED,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP,:old.ATTACHMENT_URL, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_NOTIFICATION_QUEUE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_NOTIFICATION_QUEUE BEFORE UPDATE ON DRUG_NOTIFICATION_QUEUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_NOTIFICATION_QUEUE (AUDIT_ID, ID,NOTIFICATION,LOCATION_ID,NOTIFIER_OBJECT_ID,LOCATION_QUERY_RESULT,NODE,USR_ID,DELAYED_UNTIL,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NOTIFICATION,:old.LOCATION_ID,:old.NOTIFIER_OBJECT_ID,:old.LOCATION_QUERY_RESULT,:old.NODE,:old.USR_ID,:old.DELAYED_UNTIL,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_PAT_VST_QTY_REQ;

CREATE OR REPLACE TRIGGER DRUG_MODIF_PAT_VST_QTY_REQ BEFORE UPDATE ON DRUG_PAT_VST_QTY_REQ REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_PAT_VST_QTY_REQ (AUDIT_ID, ID,PAT_ID,UNBL_KIT_TYPE_ID,QUANTITY,VISIT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PAT_ID,:old.UNBL_KIT_TYPE_ID,:old.QUANTITY,:old.VISIT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_PRIORITY_ASSIGNMENT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_PRIORITY_ASSIGNMENT BEFORE UPDATE ON DRUG_PRIORITY_ASSIGNMENT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_PRIORITY_ASSIGNMENT (AUDIT_ID, ID,PRIORITY,UNBLINDED_KIT_TYPE_ID,QUANTITY,DOSE_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PRIORITY,:old.UNBLINDED_KIT_TYPE_ID,:old.QUANTITY,:old.DOSE_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_PRIORITY_SUPPLY;

CREATE OR REPLACE TRIGGER DRUG_MODIF_PRIORITY_SUPPLY BEFORE UPDATE ON DRUG_PRIORITY_SUPPLY REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_PRIORITY_SUPPLY (AUDIT_ID, ID,PRIORITY,UNBL_KIT_TYPE_ID_FROM,UNBL_KIT_TYPE_ID_TO,QUANTITY,COUNTRY_ID,ENABLED,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PRIORITY,:old.UNBL_KIT_TYPE_ID_FROM,:old.UNBL_KIT_TYPE_ID_TO,:old.QUANTITY,:old.COUNTRY_ID,:old.ENABLED,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RNDNO;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RNDNO BEFORE UPDATE ON DRUG_RNDNO REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RNDNO (AUDIT_ID, ID,PAT_NO,STATUS_ID,TREATMENTARM_ID,MODIFIED,MODIFIED_AT,PAT_ID,VISIT_ID,UNBLINDED_AT,REASON_UNBLINDING,RANDOM_SOURCE,SORT_KEY,RANDOMIZATION_ORDER,FILEITEM_ID,BLOCK,BLOCK_ASSIGNED,COHORT_ID,RANDOM_WEIGHT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.PAT_NO,:old.STATUS_ID,:old.TREATMENTARM_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.PAT_ID,:old.VISIT_ID,:old.UNBLINDED_AT,:old.REASON_UNBLINDING,:old.RANDOM_SOURCE,:old.SORT_KEY,:old.RANDOMIZATION_ORDER,:old.FILEITEM_ID,:old.BLOCK,:old.BLOCK_ASSIGNED,:old.COHORT_ID,:old.RANDOM_WEIGHT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RNDNO_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RNDNO_ATTRIB BEFORE UPDATE ON DRUG_RNDNO_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RNDNO_ATTRIB (AUDIT_ID, RNDNO_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.RNDNO_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RNDNO_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RNDNO_FACTOR_VALUE BEFORE UPDATE ON DRUG_RNDNO_FACTOR_VALUE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RNDNO_FACTOR_VALUE (AUDIT_ID, RNDNO_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.RNDNO_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RNDNO_FILEITEM;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RNDNO_FILEITEM BEFORE UPDATE ON DRUG_RNDNO_FILEITEM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RNDNO_FILEITEM (AUDIT_ID, ID,NAME,LINES,WORKER_ID,IMPORT_TIMESTAMP,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.LINES,:old.WORKER_ID,:old.IMPORT_TIMESTAMP,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RNDNO_FILEITEM_ATTR;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RNDNO_FILEITEM_ATTR BEFORE UPDATE ON DRUG_RNDNO_FILEITEM_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RNDNO_FILEITEM_ATTR (AUDIT_ID, ID,FILEITEM_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.FILEITEM_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RND_CONFIG;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RND_CONFIG BEFORE UPDATE ON DRUG_RND_CONFIG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RND_CONFIG (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,RND_METHOD_ID,VALUE,IS_ACTIVE,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.RND_METHOD_ID,:old.VALUE,:old.IS_ACTIVE,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RND_METHOD;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RND_METHOD BEFORE UPDATE ON DRUG_RND_METHOD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RND_METHOD (AUDIT_ID, ID,RND_METHOD_NAME,RND_METHOD,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.RND_METHOD_NAME,:old.RND_METHOD,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_RND_NUM_GENERATOR;

CREATE OR REPLACE TRIGGER DRUG_MODIF_RND_NUM_GENERATOR BEFORE UPDATE ON DRUG_RND_NUM_GENERATOR REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_RND_NUM_GENERATOR (AUDIT_ID, ID,RND_NUM_GENERATOR_NAME,RND_NUM_GENERATOR,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.RND_NUM_GENERATOR_NAME,:old.RND_NUM_GENERATOR,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_SCHEDULE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_SCHEDULE BEFORE UPDATE ON DRUG_SCHEDULE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_SCHEDULE (AUDIT_ID, ID,SUBCLASS,TREATMENTARM_ID,VISIT_ID,UNBLINDED_KIT_TYPE_ID,DISPENSE_QTY,PROJECTION_QTY,DOSE_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SUBCLASS,:old.TREATMENTARM_ID,:old.VISIT_ID,:old.UNBLINDED_KIT_TYPE_ID,:old.DISPENSE_QTY,:old.PROJECTION_QTY,:old.DOSE_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_SERVICE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_SERVICE BEFORE UPDATE ON DRUG_SERVICE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_SERVICE (AUDIT_ID, ID,NAME,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_SHIPMENT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_SHIPMENT BEFORE UPDATE ON DRUG_SHIPMENT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_SHIPMENT (AUDIT_ID, ID,STATUS_ID,SENDER,FROM_LOCATION_ID,TO_LOCATION_ID,SERVICE_ID,SHIP_DATE,REQUEST_DATE,ACKNOWLEDGEMENT_DATE,LOCATION_ADDRESS_ID,TEMP_CONTROLLED,TEMP_OUT_OF_RANGE,MODIFIED,MODIFIED_AT,NAME,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.STATUS_ID,:old.SENDER,:old.FROM_LOCATION_ID,:old.TO_LOCATION_ID,:old.SERVICE_ID,:old.SHIP_DATE,:old.REQUEST_DATE,:old.ACKNOWLEDGEMENT_DATE,:old.LOCATION_ADDRESS_ID,:old.TEMP_CONTROLLED,:old.TEMP_OUT_OF_RANGE,:old.MODIFIED,:old.MODIFIED_AT,:old.NAME,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_SHIPMENT_ADDRESS;

CREATE OR REPLACE TRIGGER DRUG_MODIF_SHIPMENT_ADDRESS BEFORE UPDATE ON DRUG_SHIPMENT_ADDRESS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_SHIPMENT_ADDRESS (AUDIT_ID, ID,SHIPMENT_ID,FNAME,MNAME,LNAME,TITLE,ORGANISATION,STREET,CITY,STATE,ZIP,COUNTRY_ID,PHONE,MPHONE,FAX,EMAIL,NOTES,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SHIPMENT_ID,:old.FNAME,:old.MNAME,:old.LNAME,:old.TITLE,:old.ORGANISATION,:old.STREET,:old.CITY,:old.STATE,:old.ZIP,:old.COUNTRY_ID,:old.PHONE,:old.MPHONE,:old.FAX,:old.EMAIL,:old.NOTES,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_SHIPMENT_ATTRIB;

CREATE OR REPLACE TRIGGER DRUG_MODIF_SHIPMENT_ATTRIB BEFORE UPDATE ON DRUG_SHIPMENT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_SHIPMENT_ATTRIB (AUDIT_ID, SHIPMENT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.SHIPMENT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_SHIPMENT_FILE;

CREATE OR REPLACE TRIGGER DRUG_MODIF_SHIPMENT_FILE BEFORE UPDATE ON DRUG_SHIPMENT_FILE REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_SHIPMENT_FILE (AUDIT_ID, ID,NAME,SHIPMENT_ID,DESCRIPTION,MODIFIED_AT,MODIFIED,FILENAME,CONTENT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SHIPMENT_ID,:old.DESCRIPTION,:old.MODIFIED_AT,:old.MODIFIED,:old.FILENAME,:old.CONTENT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_STATUS;

CREATE OR REPLACE TRIGGER DRUG_MODIF_STATUS BEFORE UPDATE ON DRUG_STATUS REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_STATUS (AUDIT_ID, ID,NAME,SUBCLASS,LOCATION_TYPE,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.SUBCLASS,:old.LOCATION_TYPE,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_SUPPLY_CONF;

CREATE OR REPLACE TRIGGER DRUG_MODIF_SUPPLY_CONF BEFORE UPDATE ON DRUG_SUPPLY_CONF REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_SUPPLY_CONF (AUDIT_ID, ID,ENROLMENT_LEVEL_ID,ENROLMENT_CATEGORY_ID,WAREHOUSE_ID,INITIAL_VALUE,MIN_VALUE,MAX_VALUE,SUBCLASS,MODIFIED_AT,MODIFIED,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.ENROLMENT_LEVEL_ID,:old.ENROLMENT_CATEGORY_ID,:old.WAREHOUSE_ID,:old.INITIAL_VALUE,:old.MIN_VALUE,:old.MAX_VALUE,:old.SUBCLASS,:old.MODIFIED_AT,:old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_TRACK_STATS_LBLD;

CREATE OR REPLACE TRIGGER DRUG_MODIF_TRACK_STATS_LBLD BEFORE UPDATE ON DRUG_TRACK_STATS_LBLD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_TRACK_STATS_LBLD (AUDIT_ID, ID,SHIPMENT_ID,KIT_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.SHIPMENT_ID,:old.KIT_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_TRACK_STATS_UNLBLD;

CREATE OR REPLACE TRIGGER DRUG_MODIF_TRACK_STATS_UNLBLD BEFORE UPDATE ON DRUG_TRACK_STATS_UNLBLD REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_TRACK_STATS_UNLBLD (AUDIT_ID, ID,COUNT,KIT_AMOUNT,SHIPMENT_ID,BATCH_ID,KIT_TYPE_ID,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.COUNT,:old.KIT_AMOUNT,:old.SHIPMENT_ID,:old.BATCH_ID,:old.KIT_TYPE_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_TRANSITION;

CREATE OR REPLACE TRIGGER DRUG_MODIF_TRANSITION BEFORE UPDATE ON DRUG_TRANSITION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_TRANSITION (AUDIT_ID, FROM_STATUS_ID,TO_STATUS_ID,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.FROM_STATUS_ID,:old.TO_STATUS_ID,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_TREATMENTARM;

CREATE OR REPLACE TRIGGER DRUG_MODIF_TREATMENTARM BEFORE UPDATE ON DRUG_TREATMENTARM REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_TREATMENTARM (AUDIT_ID, ID,NAME,DESCRIPTION,MODIFIED,MODIFIED_AT,STATUS_ID,DOSE,TREATMENTARM_TYPE,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.DESCRIPTION,:old.MODIFIED,:old.MODIFIED_AT,:old.STATUS_ID,:old.DOSE,:old.TREATMENTARM_TYPE,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_UNBLINDING_LOG;

CREATE OR REPLACE TRIGGER DRUG_MODIF_UNBLINDING_LOG BEFORE UPDATE ON DRUG_UNBLINDING_LOG REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_UNBLINDING_LOG (AUDIT_ID, ID,WORKER_ID,RNDNO_ID,REASON_UNBLINDING,TYPE,UNBLINDED_AT,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.WORKER_ID,:old.RNDNO_ID,:old.REASON_UNBLINDING,:old.TYPE,:old.UNBLINDED_AT,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_UNPACKAGED_KIT;

CREATE OR REPLACE TRIGGER DRUG_MODIF_UNPACKAGED_KIT BEFORE UPDATE ON DRUG_UNPACKAGED_KIT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_UNPACKAGED_KIT (AUDIT_ID, ID,NAME,VERIFICATION_CODE,SORT_KEY,COUNT,KIT_TYPE_ID,UNBL_KIT_TYPE_ID,FILEITEM_ID,BLOCK_NUMBER,MODIFIED,MODIFIED_AT,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NAME,:old.VERIFICATION_CODE,:old.SORT_KEY,:old.COUNT,:old.KIT_TYPE_ID,:old.UNBL_KIT_TYPE_ID,:old.FILEITEM_ID,:old.BLOCK_NUMBER,:old.MODIFIED,:old.MODIFIED_AT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER DRUG_MODIF_UNPACKAGED_KIT_ATTR;

CREATE OR REPLACE TRIGGER DRUG_MODIF_UNPACKAGED_KIT_ATTR BEFORE UPDATE ON DRUG_UNPACKAGED_KIT_ATTRIB REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO DRUG_AUDIT_UNPACKAGED_KIT_ATTR (AUDIT_ID, UNPACKAGED_KIT_ID,NAME,VALUE,MODIFIED,MODIFIED_AT,ID,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.UNPACKAGED_KIT_ID,:old.NAME,:old.VALUE,:old.MODIFIED,:old.MODIFIED_AT,:old.ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER INS_NOTIF_OPT_OUT_AUDIT;

CREATE OR REPLACE TRIGGER INS_NOTIF_OPT_OUT_AUDIT BEFORE INSERT ON NOTIF_OPT_OUT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER INS_NOTIF_ROLES_AUDIT;

CREATE OR REPLACE TRIGGER INS_NOTIF_ROLES_AUDIT BEFORE INSERT ON NOTIF_ROLES REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER INS_REPORT_PERMISSION_AUDIT;

CREATE OR REPLACE TRIGGER INS_REPORT_PERMISSION_AUDIT BEFORE INSERT ON REPORT_PERMISSION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN  SELECT USER INTO v_username FROM DUAL;  :new.CREATED_BY_USER :=  v_username ; :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);END;
/


DROP TRIGGER MODIFY_SETTING;

CREATE OR REPLACE TRIGGER MODIFY_SETTING
BEFORE UPDATE
OF TYPE,HIERARCHICAL,HIERARCHY_LEVEL,PARAMETER_GROUP,ACTIVE,REQUIRED,DEFAULT_VALUE,NAME,SETTING_ID,DEFAULT_UNIT,SETTING_GROUP,VALIDATION,SUBTYPE,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY
ON SETTING
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
DECLARE v_username VARCHAR2 (128);
BEGIN
	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED := sysdate;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

    INSERT INTO SETTING_AUDIT
        (TYPE,HIERARCHICAL,HIERARCHY_LEVEL,PARAMETER_GROUP,ACTIVE,REQUIRED,DEFAULT_VALUE,NAME,SETTING_ID,DEFAULT_UNIT,SETTING_GROUP,VALIDATION,SUBTYPE,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY, CREATED_BY_USER, CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
        (:old.TYPE,:old.HIERARCHICAL,:old.HIERARCHY_LEVEL,:old.PARAMETER_GROUP,:old.ACTIVE,:old.REQUIRED,:old.DEFAULT_VALUE,:old.NAME,:old.SETTING_ID,:old.DEFAULT_UNIT,:old.SETTING_GROUP,:old.VALIDATION,:old.SUBTYPE,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIFY_SETTING_PERMISSION;

CREATE OR REPLACE TRIGGER MODIFY_SETTING_PERMISSION
BEFORE UPDATE
OF ID,SETTING_ID,PERMISSION_ID,WRITE,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY
ON SETTING_PERMISSION
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
DECLARE v_username VARCHAR2 (128);
BEGIN
	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED := sysdate;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);

   INSERT INTO SETTING_PERMISSION_AUDIT
        (ID,SETTING_ID,PERMISSION_ID,WRITE,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY, CREATED_BY_USER, CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
        (:old.ID,:old.SETTING_ID,:old.PERMISSION_ID,:old.WRITE,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIFY_SETTING_VALUES;

CREATE OR REPLACE TRIGGER MODIFY_SETTING_VALUES
BEFORE UPDATE
OF ID,SETTING_ID,LOCATION_ID,PARAMETER_ID,VALUE,UNIT,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY
ON SETTING_VALUES
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
DECLARE v_username VARCHAR2 (128);
BEGIN
	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED := sysdate;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);

   INSERT INTO SETTING_VALUES_AUDIT
        (ID,SETTING_ID,LOCATION_ID,PARAMETER_ID,VALUE,UNIT,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY, CREATED_BY_USER, CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
        (:old.ID,:old.SETTING_ID,:old.LOCATION_ID,:old.PARAMETER_ID,:old.VALUE,:old.UNIT,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIFY_WAIVER;

CREATE OR REPLACE TRIGGER MODIFY_WAIVER
BEFORE UPDATE
OF PAT_ID,VISIT_ID,SCHEDULE_ID,CONDITION_ID,REASON,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER
ON WAIVER
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
BEGIN
   INSERT INTO WAIVER_AUDIT (ID,PAT_ID,VISIT_ID,SCHEDULE_ID,CONDITION_ID,REASON,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES (:old.ID,:old.PAT_ID,:old.VISIT_ID,:old.SCHEDULE_ID,:old.CONDITION_ID,:old.REASON,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIF_CODE;

CREATE OR REPLACE TRIGGER MODIF_CODE
BEFORE UPDATE ON CODES
REFERENCING OLD AS OLD NEW AS NEW
FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  
BEGIN
   SELECT USER INTO v_username FROM DUAL;
   :new.MODIFIED := SYSDATE;
   :new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

   INSERT INTO CODES_AUDIT
   (REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, CODE_ID,
    DICT, CODE_TYPE, CODE1, CODE2, CODE3, CODE4, CODE5, CODE6,
    DECODE1, DECODE2, DECODE3, DECODE4, DECODE5, DECODE6,
    MODIFIED, MODIFIED_BY, STATUS, LOC, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
    (:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.CODE_ID,
    :old.DICT, :old.CODE_TYPE, :old.CODE1, :old.CODE2, :old.CODE3, :old.CODE4, :old.CODE5, :old.CODE6,
    :old.DECODE1, :old.DECODE2, :old.DECODE3, :old.DECODE4, :old.DECODE5, :old.DECODE6,
    :old.MODIFIED, :old.MODIFIED_BY, :old.STATUS, :old.LOC, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIF_COMMENTS;

CREATE OR REPLACE TRIGGER MODIF_COMMENTS
before UPDATE ON COMMENTS
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED := sysdate;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
    INSERT INTO COMMENTS_AUDIT
        (PAT_ID, FORM, PAGE, MODULE, GROUP_ID, FIELD, VALUE, MODIFIED_BY, MODIFIED, REASON, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
    VALUES
           (:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.GROUP_ID, :old.FIELD, :old.VALUE, :old.MODIFIED_BY, :old.MODIFIED, :old.REASON,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIF_DATA;

CREATE OR REPLACE TRIGGER MODIF_DATA
                before UPDATE of VALUE, COMMENTS ON DATA
                referencing OLD as OLD NEW as NEW
                for each row
declare
                c_Reason VARCHAR(20000):=NULL;
                c_Module VARCHAR2(40):=NULL;
                v_username VARCHAR2 (128);
            BEGIN
                if (LOCKING_PKG.isFrozen(1, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
                    Raise_application_error(-20200, 'Patient is locked or frozen.');
                end if;
            
                if (:old.LOCKED=1) then
                    Raise_application_error(-20210, 'Field '||:new.FIELD||' is locked.');
                end if;
                
                if (:new.VALUE is not NULL and :new.SPEC_VALUE<>0) then
                    Raise_application_error(-20208, :new.FIELD||': Both value and special value are not empty.');
                end if;
                
                if (:new.REASON is NULL) then
                    Raise_application_error(-20201, :new.FIELD||': Reason of modification is empty');
                end if;

                SELECT USER INTO v_username FROM DUAL;
                :new.ID := :old.ID;
                :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
                :new.MODIFIED_BY_USER := v_username;
                :new.MODIFIED := SYSDATE;
            
                INSERT INTO DATA_AUDIT
                       (REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE, SPEC_VALUE, MODIFIED, MODIFIED_BY, COMMENTS, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
                VALUES
                       (:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE, :old.SPEC_VALUE, :old.MODIFIED, :old.MODIFIED_BY, :old.COMMENTS, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);

                
                update USR_PAT set MODIFIED=:new.MODIFIED, MODIFIED_BY = :new.MODIFIED_BY, SIGNED=0 where PAT_ID = :new.PAT_ID;
            
                if (instr(:new.REASON, 'answer to query')>0) then
                    
                    c_Reason := :new.REASON||': ';
                    for cur in (        
                        SELECT QUESTION FROM QUERY WHERE
                            STATUS in (1, 21) and
                            PAT_ID=:new.PAT_ID AND FIELD=:new.FIELD AND FORM=:new.FORM AND PAGE=:new.PAGE AND COUNTER=:new.COUNTER
                    ) loop
                        c_Reason := c_Reason||cur.QUESTION||CHR(13)||CHR(10);
                    end loop;
                    
                    if (LENGTH(c_Reason)>4000) then
                        :new.REASON := SUBSTR(c_Reason, 1, 4000);
                    else
                        :new.REASON := c_Reason;
                    end if;    
                    
                    UPDATE QUERY SET STATUS=10, ANSWER='Modified data', MODIFIED_BY=:new.MODIFIED_BY WHERE
                       STATUS in (1, 21) and
                       PAT_ID=:new.PAT_ID AND FIELD=:new.FIELD AND FORM=:new.FORM AND PAGE=:new.PAGE AND COUNTER=:new.COUNTER;
                       
                end if;
                update OPT_DATA set STATUS=1 where ID=2 and
                    PAT_ID=:new.PAT_ID and FIELD=:new.FIELD AND FORM=:new.FORM AND PAGE=:new.PAGE AND COUNTER=:new.COUNTER;
                update CODES set MODIFIED = :new.MODIFIED, MODIFIED_BY = :new.MODIFIED_BY, REASON = 'Data modified', STATUS=2
                    where PAT_ID=:new.PAT_ID and FORM=:new.FORM and PAGE=:new.PAGE and FIELD=:new.FIELD and COUNTER=:new.COUNTER;
                
                ON_UPDATE_RECORD(:new.MODIFIED_BY, :new.PAT_ID, :new.FORM, :new.PAGE, UPPER(:new.FIELD), :new.COUNTER, :new.VALUE);
            
                SDV_PKG.onDataUpdate(:new.MODIFIED_BY, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD, :new.COUNTER);
                
                begin
                    select MODULE into c_Module from FIELD_DESCR where FORM=:new.FORM and FIELD=:new.FIELD;
                exception
                    when NO_DATA_FOUND then
                        c_Module:=NULL;
                end;












                       
            END;
/


DROP TRIGGER MODIF_DATA_VIS;

CREATE OR REPLACE TRIGGER MODIF_DATA_VIS
before UPDATE of VALUE ON DATA
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
	SELECT USER INTO v_username FROM DUAL;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
    if (:old.FIELD in ('VISDAY','VISMON','VISYEA')) then
        insert into VS_QUEUE (PAT_ID,FORM,INSTANCE,COUNTER) values(:old.PAT_ID,:old.FORM,:old.PAGE,:old.COUNTER);
    end if;
END;
/


DROP TRIGGER MODIF_EXT_DATA;

CREATE OR REPLACE TRIGGER MODIF_EXT_DATA
                before UPDATE of COMMENTS ON EXT_DATA
                referencing OLD as OLD NEW as NEW
                for each row
DECLARE
                v_username VARCHAR2 (128);  
            BEGIN
                if (LOCKING_PKG.isFrozen(4, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
                    Raise_application_error(-20200, 'Patient is locked or frozen (ext. data).');
                end if;
            
                if (:old.LOCKED=1) then
                    Raise_application_error(-20212, 'Ext. field '||:new.FIELD||' is locked.');
                end if;
                
                if (:new.REASON is NULL) then
                    Raise_application_error(-20201, :old.FIELD||': Reason of modification is empty');
                end if;
            
                :new.MODIFIED := SYSDATE;
                
                
                SELECT USER INTO v_username FROM DUAL;  
                :new.MODIFIED_BY_USER :=  v_username;
                :new.MODIFIED_BY := v_username;
                :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
            
                if (:old.ID = 1) then
                    INSERT INTO EXT_DATA_AUDIT
                        (PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE_TYPE,
                        ID, VALUE, COMMENTS, MODIFIED, MODIFIED_BY,
                        REASON, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
                    VALUES
                        (:old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE_TYPE,
                        :old.ID, :old.VALUE, :old.COMMENTS, :old.MODIFIED, :old.MODIFIED_BY,
                        :old.REASON, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
                    ON_UPDATE_RECORD(:new.MODIFIED_BY, :new.PAT_ID, :new.FORM, :new.PAGE, UPPER(:new.FIELD), :new.COUNTER, :new.COMMENTS);
                    
                    SDV_PKG.onDataUpdate(:new.MODIFIED_BY, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD, :new.COUNTER);        
                end if;
                
            END;
/


DROP TRIGGER MODIF_FIELD;

CREATE OR REPLACE TRIGGER MODIF_FIELD
before UPDATE ON FIELD_VALLAB
referencing OLD as OLD NEW as NEW
for each row
BEGIN
    :new.ID := :old.ID;
    :new.CREATED_TIMESTAMP := :old.CREATED_TIMESTAMP;
 	:new.CREATED_BY_USER := :old.CREATED_BY_USER;
	:new.MODIFIED_TIMESTAMP := SYSTIMESTAMP;
 	:new.MODIFIED_BY_USER := USER;
END;
/


DROP TRIGGER MODIF_FIELD_VALLAB;

CREATE OR REPLACE TRIGGER MODIF_FIELD_VALLAB
before UPDATE ON FIELD_VALLAB
referencing OLD as OLD NEW as NEW
for each row
BEGIN
    :new.ID := :old.ID;
    :new.CREATED_TIMESTAMP := :old.CREATED_TIMESTAMP;
 	:new.CREATED_BY_USER := :old.CREATED_BY_USER;
	:new.MODIFIED_TIMESTAMP := SYSTIMESTAMP;
 	:new.MODIFIED_BY_USER := USER;
END;
/


DROP TRIGGER MODIF_LABR_DATA;

CREATE OR REPLACE TRIGGER MODIF_LABR_DATA
before UPDATE of LIMIT1, LIMIT2, LIMIT1_REL, LIMIT2_REL, LIMIT1_UNIT, LIMIT1_UNIT_T, LIMIT2_UNIT, LIMIT2_UNIT_T, SEX, AGE_FROM, AGE_FROM_REL, AGE_TO, AGE_TO_REL ON LABR_DATA
referencing OLD as OLD NEW as NEW
for each row
BEGIN
   	:new.MODIFIED := SYSDATE;
	INSERT INTO LABR_DATA_AUDIT
   		(SIT_ID, SET_NAME, SET_COUNTER, PARAM, PARAM_COUNTER, LIMIT1, LIMIT2, LIMIT1_REL, LIMIT2_REL, LIMIT1_UNIT,
   		LIMIT1_UNIT_T, LIMIT2_UNIT, LIMIT2_UNIT_T, SEX, AGE_FROM, AGE_FROM_REL, AGE_TO, AGE_TO_REL, MODIFIED, MODIFIED_BY)
	VALUES
   		(:old.SIT_ID, :old.SET_NAME, :old.SET_COUNTER, :old.PARAM, :old.PARAM_COUNTER, :old.LIMIT1, :old.LIMIT2, :old.LIMIT1_REL, :old.LIMIT2_REL, :old.LIMIT1_UNIT,
   		:old.LIMIT1_UNIT_T, :old.LIMIT2_UNIT, :old.LIMIT2_UNIT_T, :old.SEX, :old.AGE_FROM, :old.AGE_FROM_REL, :old.AGE_TO, :old.AGE_TO_REL, :old.MODIFIED, :old.MODIFIED_BY);
END;
/


DROP TRIGGER MODIF_LABR_SETS;

CREATE OR REPLACE TRIGGER MODIF_LABR_SETS
before UPDATE of LAB_NAME, LAB_ID, FROM_DAY, FROM_MON, FROM_YEA, FROM_REL, TO_DAY, TO_MON, TO_YEA, TO_REL, SIGNED, SIGNED_BY ON LABR_SETS
referencing OLD as OLD NEW as NEW
for each row
BEGIN
   	:new.MODIFIED := SYSDATE;
	INSERT INTO LABR_SETS_AUDIT
   		(SIT_ID, SET_NAME, SET_COUNTER, LAB_NAME, LAB_ID, FROM_DAY, FROM_MON, FROM_YEA, FROM_REL,
   		TO_DAY, TO_MON, TO_YEA, TO_REL, SIGNED, SIGNED_BY, MODIFIED, MODIFIED_BY)
	VALUES
   		(:old.SIT_ID, :old.SET_NAME, :old.SET_COUNTER, :old.LAB_NAME, :old.LAB_ID, :old.FROM_DAY, :old.FROM_MON, :old.FROM_YEA, :old.FROM_REL,
   		:old.TO_DAY, :old.TO_MON, :old.TO_YEA, :old.TO_REL, :old.SIGNED, :old.SIGNED_BY, :old.MODIFIED, :old.MODIFIED_BY);
END;
/


DROP TRIGGER MODIF_LOCK;

CREATE OR REPLACE TRIGGER MODIF_LOCK
before UPDATE ON LOCKING
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
    :new.MODIFIED := sysdate;
	SELECT USER INTO v_username FROM DUAL;  
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	insert into LOCKING_AUDIT
   		(PAT_ID, FORM, PAGE, MODULE, SET_ID, STATUS, MODIFIED, MODIFIED_BY, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.SET_ID, :old.STATUS, :old.MODIFIED, :old.MODIFIED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
end;
/


DROP TRIGGER MODIF_NOTIF_OPT_OUT;

CREATE OR REPLACE TRIGGER MODIF_NOTIF_OPT_OUT BEFORE UPDATE ON NOTIF_OPT_OUT REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO NOTIF_OPT_OUT_AUDIT (AUDIT_ID, ID,NOTIFICATION_ID,USR_ID,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NOTIFICATION_ID,:old.USR_ID,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER MODIF_NOTIF_ROLES;

CREATE OR REPLACE TRIGGER MODIF_NOTIF_ROLES BEFORE UPDATE ON NOTIF_ROLES REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO NOTIF_ROLES_AUDIT (AUDIT_ID, ID,NOTIFICATION_ID,ROLE_ID,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.NOTIFICATION_ID,:old.ROLE_ID,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER MODIF_OPT_CODE;

CREATE OR REPLACE TRIGGER MODIF_OPT_CODE
before UPDATE ON OPTCODES_LIST
referencing OLD as OLD New as New
for each row
BEGIN
  :new.MODIFIED := SYSDATE;
  begin
      INSERT INTO OPTCODES_LIST_AUDIT
         (	CODE,
         	DICT,  CODE1, CODE2, CODE3, CODE4, CODE5, CODE6,
         	DECODE1, DECODE2, DECODE3, DECODE4, DECODE5, DECODE6,
        	MODIFIED, MODIFIED_BY)
     	VALUES
      	( :old.CODE, :old.DICT,
    		:old.CODE1, :old.CODE2, :old.CODE3, :old.CODE4, :old.CODE5, :old.CODE6,
      		:old.DECODE1, :old.DECODE2, :old.DECODE3, :old.DECODE4, :old.DECODE5, :old.DECODE6,
      		:old.MODIFIED, :old.MODIFIED_BY);
    end;
  		update CODES set MODIFIED = SYSDATE, MODIFIED_BY=:new.MODIFIED_BY, REASON = 'Opt code modified', STATUS=4,
  			CODE1=to_char(:new.code), CODE2=:new.code2, CODE3=:new.code3, CODE4=:new.code4, CODE5=:new.CODE5,
  			DECODE1=:new.decode1, DECODE2=:new.decode2, DECODE3=:new.decode3, DECODE4=:new.decode4, DECODE5=:new.DECODE5
  			where  CODE1=to_char(:old.code) and decode(code_type, 4, 'med', 3, 'drug')=:old.dict;
END;
/


DROP TRIGGER MODIF_OPT_DATA;

CREATE OR REPLACE TRIGGER MODIF_OPT_DATA
before UPDATE of VALUE, COMMENTS, STATUS ON OPT_DATA
referencing OLD as OLD NEW as NEW
for each row
declare
	c_Module VARCHAR2(40):=NULL;
	v_username VARCHAR2 (128);  
BEGIN

	if (LOCKING_PKG.isFrozen(2, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
		Raise_application_error(-20200, 'Patient is locked or frozen (opt. data).');
	end if;

	if (:old.LOCKED=1) then
		Raise_application_error(-20211, 'Opt. field '||:new.FIELD||' is locked.');
	end if;

	:new.MODIFIED := SYSDATE;
	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED_BY_USER :=  v_username;
    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	INSERT INTO OPT_DATA_AUDIT
   		(REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE, MODIFIED, MODIFIED_BY, COMMENTS, ID, STATUS, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE, :old.MODIFIED, :old.MODIFIED_BY, :old.COMMENTS, :old.ID, :old.STATUS, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);

	if (:new.ID=2 AND :new.STATUS=3) then
		begin
			select MODULE into c_Module from FIELD_DESCR where FORM=:new.FORM and FIELD=:new.FIELD;
		exception
			when NO_DATA_FOUND then
				c_Module:=NULL;
		end;












	end if;

END;
/


DROP TRIGGER MODIF_QUERY;

CREATE OR REPLACE TRIGGER MODIF_QUERY
before UPDATE of QUESTION, ANSWER, STATUS ON QUERY
referencing OLD as OLD NEW as NEW
for each row
DECLARE
    v_username VARCHAR2 (128);
	n_CanUpdate NUMBER := 0;
	n_NewStatus NUMBER := 0;
BEGIN
	if (LOCKING_PKG.isFrozen(8, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
		Raise_application_error(-20200, 'Patient is locked or frozen.');
	end if;

	if (:new.STATUS=10 AND :new.ANSWER is NULL) then
		Raise_application_error(-20202, 'Answer to query is empty.');
	end if;

	if (:new.STATUS<>10 and :old.GROUP_ID is not NULL) then
		begin
			select CAN_UPDATE into n_CanUpdate from QUERY_GROUP_MEMBERS where
			WRK_ID = :new.MODIFIED_BY and GROUP_ID = :old.GROUP_ID;
			if (n_CanUpdate=0) then
				Raise_application_error(-20204, 'Query could not be modified: insufficient privilegy.');
			end if;
		exception
			WHEN NO_DATA_FOUND THEN
				Raise_application_error(-20205, 'Attempt to update the query that belongs to other group.');
		end;
	end if;

    if (:new.STATUS<>:old.STATUS) then
		begin
			select NEW_STATUS into n_NewStatus from ALLOWED_QS_MODIF where OLD_STATUS = :old.STATUS and NEW_STATUS = :new.STATUS;
		exception
			WHEN NO_DATA_FOUND THEN
				Raise_application_error(-20206, 'New status does not correspond to the allowed query flow.');
		end;
	end if;

    if (:new.QUESTION<>:old.QUESTION) then
		begin
			select STATUS into n_NewStatus from ALLOWED_Q_MODIF where FIELD = 'QUESTION' and STATUS = :new.STATUS;
		exception
			WHEN NO_DATA_FOUND THEN
				Raise_application_error(-20207, 'Question cannot be modified for query with status '||:new.STATUS);
		end;
	end if;

    if (:new.ANSWER<>:old.ANSWER) then
		begin
			select STATUS into n_NewStatus from ALLOWED_Q_MODIF where FIELD = 'ANSWER' and STATUS = :new.STATUS;
		exception
			WHEN NO_DATA_FOUND THEN
				Raise_application_error(-20209, 'Answer cannot be modified for query with status '||:new.STATUS);
		end;
	end if;

   	:new.MODIFIED := SYSDATE;
   	:new.CREATED := :old.CREATED;
	:new.GROUP_ID := :old.GROUP_ID;
	SELECT USER INTO v_username FROM DUAL;  
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	INSERT INTO QUERY_AUDIT
   		(QUERY_ID, RUN_ID, PAT_ID, FIELD, FORM, PAGE, COUNTER, QUESTION,
   	  	ANSWER, STATUS, CREATED, CREATED_BY, MODIFIED, MODIFIED_BY, QUERY_NO, GROUP_ID, VALUE, SPEC_VALUE, COMMENTS, REF_ID, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.QUERY_ID, :old.RUN_ID, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.QUESTION,
   		:old.ANSWER, :old.STATUS, :old.CREATED, :old.CREATED_BY, :old.MODIFIED, :old.MODIFIED_BY, :old.QUERY_NO, :old.GROUP_ID,
		:old.VALUE, :old.SPEC_VALUE, :old.COMMENTS, :old.REF_ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIF_REPORT_PERMISSION;

CREATE OR REPLACE TRIGGER MODIF_REPORT_PERMISSION BEFORE UPDATE ON REPORT_PERMISSION REFERENCING NEW AS NEW OLD AS OLD FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  BEGIN SELECT USER INTO v_username FROM DUAL;  :new.MODIFIED_BY_USER :=  v_username; :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP); INSERT INTO REPORT_PERMISSION_AUDIT (AUDIT_ID, ID,REPORT_OBJECT_PATH,VISIT_ID,ROLE_ID_READ_ACCESS,DISABLED,MODIFIED,MODIFIED_BY,CREATED,CREATED_BY,CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP, TRIGGER_ACTION, TRIGGER_DATE) VALUES(SYS_GUID(), :old.ID,:old.REPORT_OBJECT_PATH,:old.VISIT_ID,:old.ROLE_ID_READ_ACCESS,:old.DISABLED,:old.MODIFIED,:old.MODIFIED_BY,:old.CREATED,:old.CREATED_BY,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP, 'MODIFY', CURRENT_TIMESTAMP); END;
/


DROP TRIGGER MODIF_SCRIPT;

CREATE OR REPLACE TRIGGER MODIF_SCRIPT
before UPDATE ON SCRIPTS
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	if (UTIL.REGEXP.GET_MATCH(:new.NAME, '[\\\/,#$\s]', 2) is not NULL) then
		raise_application_error(-20701, 'Invalid script name "'||:new.NAME||'"');
	end if;

	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED := sysdate;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	insert into SCRIPTS_AUDIT
   		(NAME, VERSION, SOURCE, FLAGS, ALLOW, DENY, MODIFIED, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.NAME, :old.VERSION, :old.SOURCE, :old.FLAGS, :old.ALLOW, :old.DENY, :old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
end;
/


DROP TRIGGER MODIF_SCRIPT_QUERY;

CREATE OR REPLACE TRIGGER MODIF_SCRIPT_QUERY
before UPDATE ON SCRIPT_QUERY
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
    :new.MODIFIED := sysdate;
	SELECT USER INTO v_username FROM DUAL;  
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	insert into SCRIPT_QUERY_AUDIT
   		(QUERY_ID, RUN_ID, PAT_ID, FIELD, FORM, PAGE, COUNTER, QUESTION, STATUS, CREATED, CREATED_BY, MODIFIED_BY, MODIFIED, GROUP_ID, ORIG_QUESTION, VALUE, SPEC_VALUE, COMMENTS, REF_ID, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.QUERY_ID, :old.RUN_ID, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.QUESTION, :old.STATUS, :old.CREATED, :old.CREATED_BY, :old.MODIFIED_BY, :old.MODIFIED, :old.GROUP_ID, :old.ORIG_QUESTION, :old.VALUE, :old.SPEC_VALUE, :old.COMMENTS, :old.REF_ID,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
end;
/


DROP TRIGGER MODIF_SDV;

CREATE OR REPLACE TRIGGER MODIF_SDV
before UPDATE ON SDV
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
	:new.MODIFIED := SYSDATE;

	SELECT USER INTO v_username FROM DUAL;  
   	:new.MODIFIED_BY := v_username;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	INSERT INTO SDV_AUDIT
   		(PAT_ID, FORM, PAGE, MODULE, FIELD, COUNTER, FLAGS, GROUP_ID, MODIFIED_BY, MODIFIED, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.FIELD, :old.COUNTER, :old.FLAGS, :old.GROUP_ID, :old.MODIFIED_BY, :old.MODIFIED, :old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIF_SDV_ERRORS;

CREATE OR REPLACE TRIGGER MODIF_SDV_ERRORS
before UPDATE ON SDV_ERRORS
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED := sysdate;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	INSERT INTO SDV_ERRORS_AUDIT
   		(PAT_ID, FORM, PAGE, MODULE, ERRORS, FIELDS1, FIELDS2, FIELDS3, FIELDS4, GROUP_ID, MODIFIED_BY, MODIFIED, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FORM, :old.PAGE, :old.MODULE, :old.ERRORS, :old.FIELDS1, :old.FIELDS2, :old.FIELDS3, :old.FIELDS4, :old.GROUP_ID, :old.MODIFIED_BY, :old.MODIFIED,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIF_SDV_SESSION;

CREATE OR REPLACE TRIGGER MODIF_SDV_SESSION
before UPDATE of SESSION_ACCEPTED ON SDV_SESSION
referencing OLD as OLD NEW as NEW
for each row
BEGIN
      
    INSERT INTO SDV_SESSION_AUDIT
           (BATCH_ID, BATCH_NO, SIT_ID, BATCH_SIZE, SAMPLE_SIZE, CREATED_BY, CREATED, NUM_ACCEPT, TOTAL_ERRORS, SESSION_ACCEPTED, EVAL_DATE, SITE_REVIEW_BY, SITE_REVIEW_DATE, VSS_MODE)
    VALUES
           (:old.BATCH_ID, :old.BATCH_NO, :old.SIT_ID, :old.BATCH_SIZE, :old.SAMPLE_SIZE, :old.CREATED_BY, :old.CREATED, :old.NUM_ACCEPT, :old.TOTAL_ERRORS, :old.SESSION_ACCEPTED, :old.EVAL_DATE, :old.SITE_REVIEW_BY, :old.SITE_REVIEW_DATE, :old.VSS_MODE);
END;
/


DROP TRIGGER MODIF_SIGNED_MSG;

CREATE OR REPLACE TRIGGER MODIF_SIGNED_MSG
before UPDATE ON SIGNED_MSG
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
   	:new.MODIFIED := SYSDATE;
	SELECT USER INTO v_username FROM DUAL;  
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	INSERT INTO SIGNED_MSG_AUDIT
   		(MSG_ID, MSG_NAME, PAT_ID, MESSAGE, SIGNATURE, MODIFIED, MODIFIED_BY, KEY_ID, OBSOLETE, REASON, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.MSG_ID, :old.MSG_NAME, :old.PAT_ID, :old.MESSAGE, :old.SIGNATURE,
		:old.MODIFIED, :old.MODIFIED_BY, :old.KEY_ID, :old.OBSOLETE, :old.REASON,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER MODIF_SI_RECIPIENT;

CREATE OR REPLACE TRIGGER MODIF_SI_RECIPIENT
BEFORE UPDATE ON SI_RECIPIENT
REFERENCING OLD AS OLD NEW AS NEW
FOR EACH ROW
DECLARE v_username VARCHAR2 (128);
begin
    SELECT user INTO v_username  FROM dual;
    :new.modified := SYSTIMESTAMP;
    :new.modified_by := v_username ;
end;
/


DROP TRIGGER MODIF_USR_PAT;

CREATE OR REPLACE TRIGGER MODIF_USR_PAT
            before UPDATE of SIT_ID, SIGNED, INCLUDED, PAT_NO, SCR_NO, INITIALS, BIRTHDATE, WITHDRAWN_DATE, RANDOM_DATE, STATUS, STATUS_TS, PP_ID ON USR_PAT
            referencing OLD as OLD NEW as NEW
            for each row
            FOLLOWS WITHDRAW_USR_PAT
WHEN (
new.SIT_ID <> old.SIT_ID or (new.SIT_ID IS NULL AND old.SIT_ID IS NOT NULL) or (new.SIT_ID IS NOT NULL AND old.SIT_ID IS NULL) or
                new.SIGNED <> old.SIGNED or (new.SIGNED IS NULL AND old.SIGNED IS NOT NULL) or (new.SIGNED IS NOT NULL AND old.SIGNED IS NULL) or
                new.INCLUDED <> old.INCLUDED  or (new.INCLUDED IS NULL AND old.INCLUDED IS NOT NULL) or (new.INCLUDED IS NOT NULL AND old.INCLUDED IS NULL) or
                new.PAT_NO <> old.PAT_NO  or (new.PAT_NO IS NULL AND old.PAT_NO IS NOT NULL) or (new.PAT_NO IS NOT NULL AND old.PAT_NO IS NULL) or
                new.SCR_NO <> old.SCR_NO or (new.SCR_NO IS NULL AND old.SCR_NO IS NOT NULL) or (new.SCR_NO IS NOT NULL AND old.SCR_NO IS NULL) or
                new.INITIALS <> old.INITIALS or (new.INITIALS IS NULL AND old.INITIALS IS NOT NULL) or (new.INITIALS IS NOT NULL AND old.INITIALS IS NULL) or
                new.BIRTHDATE <> old.BIRTHDATE or (new.BIRTHDATE IS NULL AND old.BIRTHDATE IS NOT NULL) or (new.BIRTHDATE IS NOT NULL AND old.BIRTHDATE IS NULL) or
                new.WITHDRAWN_DATE <> old.WITHDRAWN_DATE or (new.WITHDRAWN_DATE IS NULL AND old.WITHDRAWN_DATE IS NOT NULL) or (new.WITHDRAWN_DATE IS NOT NULL AND old.WITHDRAWN_DATE IS NULL) or
                new.RANDOM_DATE <> old.RANDOM_DATE or (new.RANDOM_DATE IS NULL AND old.RANDOM_DATE IS NOT NULL) or (new.RANDOM_DATE IS NOT NULL AND old.RANDOM_DATE IS NULL) or
                new.STATUS <> old.STATUS or (new.STATUS IS NULL AND old.STATUS IS NOT NULL) or (new.STATUS IS NOT NULL AND old.STATUS IS NULL) or
                new.STATUS_TS <> old.STATUS_TS or (new.STATUS_TS IS NULL AND old.STATUS_TS IS NOT NULL) or (new.STATUS_TS IS NOT NULL AND old.STATUS_TS IS NULL) or
                new.PP_ID <> old.PP_ID or (new.PP_ID IS NULL AND old.PP_ID IS NOT NULL) or (new.PP_ID IS NOT NULL AND old.PP_ID IS NULL)
      )
DECLARE v_username VARCHAR2 (128);
            BEGIN
            	SELECT USER INTO v_username FROM DUAL;
                :new.CREATED := :old.CREATED;
                :new.CREATED_BY := :old.CREATED_BY;
                :new.MODIFIED := SYSDATE;
                if (:new.MODIFIED_BY IS NULL) then
                    :new.MODIFIED_BY := v_username;
                end if;
                :new.MODIFIED_BY_USER :=  v_username;
                :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

                INSERT INTO USR_PAT_AUDIT
                       (PAT_ID, SIT_ID, CREATED, CREATED_BY, MODIFIED, MODIFIED_BY, SIGNED, INCLUDED, SCR_NO, PAT_NO, INITIALS, BIRTHDATE, WITHDRAWN_DATE, RANDOM_DATE, STATUS, STATUS_TS,MODIFIED_TIMESTAMP,MODIFIED_BY_USER,CREATED_TIMESTAMP,CREATED_BY_USER, PP_ID)
                VALUES
                       (:old.PAT_ID, :old.SIT_ID, :old.CREATED, :old.CREATED_BY, :old.MODIFIED, :old.MODIFIED_BY,:old.SIGNED, :old.INCLUDED, :old.SCR_NO, :old.PAT_NO, :old.INITIALS, :old.BIRTHDATE, :old.WITHDRAWN_DATE, :old.RANDOM_DATE, :old.STATUS, :old.STATUS_TS,:old.MODIFIED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.CREATED_TIMESTAMP,:old.CREATED_BY_USER, :old.PP_ID);
            END;
/


DROP TRIGGER NEW_CODE;

CREATE OR REPLACE TRIGGER NEW_CODE
BEFORE INSERT ON CODES
REFERENCING OLD AS OLD NEW AS NEW
FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  
begin
	:new.MODIFIED := SYSDATE;
	SELECT USER INTO v_username FROM DUAL;

    :new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_BY_USER := v_username;

end;
/


DROP TRIGGER NEW_COMMENTS;

CREATE OR REPLACE TRIGGER NEW_COMMENTS
before INSERT ON COMMENTS
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
end;
/


DROP TRIGGER NEW_DATA;

CREATE OR REPLACE TRIGGER NEW_DATA
            before INSERT ON DATA
            referencing OLD as OLD NEW as NEW
            for each row
declare
                c_Wrk_Id VARCHAR(40):=NULL;
                c_Center_Id VARCHAR(40):=NULL;
                c_Answer VARCHAR(2000):=NULL;
                c_Reason VARCHAR(20000):=NULL;
                c_Module VARCHAR2(40):=NULL;
                n_Criticals NUMBER := 0;
                v_username VARCHAR2 (128);
                
            BEGIN
                if (LOCKING_PKG.isFrozen(1, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
                    Raise_application_error(-20200, 'Patient is locked or frozen.');
                end if;
                
                if (:new.VALUE is not NULL and :new.SPEC_VALUE<>0) then
                    Raise_application_error(-20208, :new.FIELD||': Both value and special value are not empty.');
                end if;

                SELECT USER INTO v_username FROM DUAL;
                :new.ID := SYS_GUID();
                :new.CREATED_BY_USER := v_username;
                :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
                :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
                :new.MODIFIED_BY_USER := v_username;
                
                :new.MODIFIED := SYSDATE;
                   c_Wrk_Id := :new.MODIFIED_BY;
            
                begin
                    select SIT_ID INTO c_Center_Id from USR_PAT where PAT_ID = :new.PAT_ID;
                    update USR_PAT set MODIFIED = :new.MODIFIED, MODIFIED_BY = :new.MODIFIED_BY where PAT_ID = :new.PAT_ID;
                exception
                    WHEN NO_DATA_FOUND THEN
                        begin
                            select SIT_ID into c_Center_Id from ADM_ICRF4_IF.WORKERS where WRK_ID=c_Wrk_Id;
                        exception
                            when NO_DATA_FOUND then
                                Raise_application_error(-20301, 'Unknown Center No. Please apply to project manager.');
                        end;
                        insert into USR_PAT (PAT_ID, SIT_ID, CREATED, CREATED_BY, MODIFIED_BY) values (:new.PAT_ID, c_Center_Id, :new.MODIFIED, c_Wrk_Id, c_Wrk_Id);
                        ON_NEW_RECORD(c_Center_Id, c_Wrk_Id, :new.PAT_ID);
                end;


                
                if (instr(:new.REASON, 'answer to query')>0) then
                
                    c_Reason := :new.REASON||': ';
                    for cur in (        
                        SELECT QUESTION FROM QUERY WHERE
                            STATUS in (1, 21) and
                            PAT_ID=:new.PAT_ID AND FIELD=:new.FIELD AND FORM=:new.FORM AND PAGE=:new.PAGE AND COUNTER=:new.COUNTER
                    ) loop
                        c_Reason := c_Reason||cur.QUESTION||CHR(13)||CHR(10);
                    end loop;
                
                    if (LENGTH(c_Reason)>4000) then
                        :new.REASON := SUBSTR(c_Reason, 1, 4000);
                    else
                        :new.REASON := c_Reason;
                    end if;    
                
                    update QUERY set STATUS=10, ANSWER='New data', MODIFIED_BY=:new.MODIFIED_BY where
                        STATUS in (1, 21) and
                        PAT_ID=:new.PAT_ID and FIELD=:new.FIELD and FORM=:new.FORM and PAGE=:new.PAGE and COUNTER=:new.COUNTER;
                end if;
                
                ON_UPDATE_RECORD(c_Wrk_id, :new.PAT_ID, :new.FORM, :new.PAGE, UPPER(:new.FIELD), :new.COUNTER, :new.VALUE);
                
                SDV_PKG.onDataUpdate(c_Wrk_Id, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD, :new.COUNTER);
                
                
                begin
                    select MODULE into c_Module from FIELD_DESCR where FORM=:new.FORM and FIELD=:new.FIELD;
                exception
                    when NO_DATA_FOUND then
                        c_Module:=NULL;
                end;
                    select count(*) into n_Criticals from FIELD_DESCR where BITAND(FLAGS,16)=16;
                    IF(n_Criticals > 0 and :new.COUNTER > 0) THEN
                      SDV_PKG.addCriticalLoopField(:new.PAT_ID, :new.FORM, :new.PAGE, :new.COUNTER, :new.FIELD);                
                    END IF;












                        
            END;
/


DROP TRIGGER NEW_DATA_VIS;

CREATE OR REPLACE TRIGGER NEW_DATA_VIS
before INSERT ON DATA
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

    if (:new.FIELD in ('VISDAY','VISMON','VISYEA') OR :new.FIELD like '%!_DATE' escape '!') then
        insert into VS_QUEUE (PAT_ID,FORM,INSTANCE,COUNTER) values(:new.PAT_ID,:new.FORM,:new.PAGE,:new.COUNTER);
    end if;
END;
/


DROP TRIGGER NEW_EXT_DATA;

CREATE OR REPLACE TRIGGER NEW_EXT_DATA
                before INSERT ON EXT_DATA
                referencing OLD as OLD NEW as NEW
                for each row
DECLARE
                v_username VARCHAR2 (128);
            BEGIN
                if (LOCKING_PKG.isFrozen(4, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
                    Raise_application_error(-20200, 'Patient is locked or frozen (ext. data).');
                end if;


                
                if (instr(:new.REASON, 'answer to query')>0) then
                    update QUERY set STATUS=10, ANSWER='New ext. data', MODIFIED_BY=:new.MODIFIED_BY where
                        STATUS=1 and
                        PAT_ID=:new.PAT_ID and FIELD=:new.FIELD and FORM=:new.FORM and PAGE=:new.PAGE and COUNTER=:new.COUNTER;
                end if;
                 
                SELECT USER INTO v_username FROM DUAL;  
                :new.CREATED_BY_USER :=  v_username ;
                :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
                :new.MODIFIED_BY_USER :=  v_username;
                :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
                ON_UPDATE_RECORD(:new.MODIFIED_BY, :new.PAT_ID, :new.FORM, :new.PAGE, UPPER(:new.FIELD), :new.COUNTER, :new.COMMENTS);
                
                SDV_PKG.onDataUpdate(:new.MODIFIED_BY, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD, :new.COUNTER);    
                
                
            END;
/


DROP TRIGGER NEW_FIELD;

CREATE OR REPLACE TRIGGER NEW_FIELD
before INSERT ON FIELD_DESCR
referencing OLD as OLD NEW as NEW
for each row
BEGIN
    :new.flow_number := GET_FLD_NUMBER();
    
    :new.ID := SYS_GUID();
    :new.CREATED_TIMESTAMP := SYSTIMESTAMP;
 	:new.CREATED_BY_USER := USER;
	:new.MODIFIED_TIMESTAMP := SYSTIMESTAMP;
 	:new.MODIFIED_BY_USER := USER;
END;
/


DROP TRIGGER NEW_FIELD_VALLAB;

CREATE OR REPLACE TRIGGER NEW_FIELD_VALLAB
before INSERT ON FIELD_VALLAB
referencing OLD as OLD NEW as NEW
for each row
BEGIN
    :new.ID := SYS_GUID();
    :new.CREATED_TIMESTAMP := SYSTIMESTAMP;
 	:new.CREATED_BY_USER := USER;
	:new.MODIFIED_TIMESTAMP := SYSTIMESTAMP;
 	:new.MODIFIED_BY_USER := USER;
END;
/


DROP TRIGGER NEW_LABR_DATA;

CREATE OR REPLACE TRIGGER NEW_LABR_DATA
before INSERT ON LABR_DATA
referencing OLD as OLD NEW as NEW
for each row
BEGIN
    :new.MODIFIED := SYSDATE;
END;
/


DROP TRIGGER NEW_LABR_SETS;

CREATE OR REPLACE TRIGGER NEW_LABR_SETS
before INSERT ON LABR_SETS
referencing OLD as OLD NEW as NEW
for each row
BEGIN
    :new.MODIFIED := SYSDATE;
END;
/


DROP TRIGGER NEW_LOCK;

CREATE OR REPLACE TRIGGER NEW_LOCK
before INSERT ON LOCKING
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
end;
/


DROP TRIGGER NEW_OPT_DATA;

CREATE OR REPLACE TRIGGER NEW_OPT_DATA
before INSERT ON OPT_DATA
referencing OLD as OLD NEW as NEW
for each row
declare
	c_Module VARCHAR2(40):=NULL;
	v_username VARCHAR2 (128);
BEGIN
	if (LOCKING_PKG.isFrozen(2, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
		Raise_application_error(-20200, 'Patient is locked or frozen (opt. data).');
	end if;

	if (:new.ID=2 AND :new.STATUS=3) then
		begin
			select MODULE into c_Module from FIELD_DESCR where FORM=:new.FORM and FIELD=:new.FIELD;
		exception
			when NO_DATA_FOUND then
				c_Module:=NULL;
		end;












	end if;
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

END;
/


DROP TRIGGER NEW_QUERY;

CREATE OR REPLACE TRIGGER NEW_QUERY
before INSERT ON QUERY
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
	if (LOCKING_PKG.isFrozen(8, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
		Raise_application_error(-20200, 'Patient is locked or frozen.');
	end if;

    :new.CREATED := SYSDATE;
    :new.MODIFIED := :new.CREATED;
	:new.CREATED_BY := :new.MODIFIED_BY;
	if (:new.ORIG_QUESTION is NULL) then
		:new.ORIG_QUESTION := :new.QUESTION;
	end if;

	select query_seq.nextval into :new.QUERY_NO from dual;

	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	if (:new.GROUP_ID is NULL) then
		begin
			select GROUP_ID into :new.GROUP_ID from QUERY_GROUP_MEMBERS where
			WRK_ID = :new.MODIFIED_BY and CAN_UPDATE = 1 and DEFAULT_GROUP = 1;
		exception
			when NO_DATA_FOUND then
				:new.GROUP_ID := NULL;
		end;
	end if;

END;
/


DROP TRIGGER NEW_SCRIPT;

CREATE OR REPLACE TRIGGER NEW_SCRIPT
before INSERT ON SCRIPTS
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	if (UTIL.REGEXP.GET_MATCH(:new.NAME, '[\\\/,#$\s]', 2) is not NULL) then
		raise_application_error(-20701, 'Invalid script name "'||:new.NAME||'"');
	end if;

	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

end;
/


DROP TRIGGER NEW_SCRIPT_QUERY;

CREATE OR REPLACE TRIGGER NEW_SCRIPT_QUERY
before INSERT ON SCRIPT_QUERY
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
end;
/


DROP TRIGGER NEW_SDV;

CREATE OR REPLACE TRIGGER NEW_SDV
before INSERT ON SDV
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
end;
/


DROP TRIGGER NEW_SDV_ERRORS;

CREATE OR REPLACE TRIGGER NEW_SDV_ERRORS
before INSERT ON SDV_ERRORS
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
end;
/


DROP TRIGGER NEW_SDV_GROUP_MEMBER;

CREATE OR REPLACE TRIGGER NEW_SDV_GROUP_MEMBER
before INSERT ON SDV_GROUP_MEMBERS
referencing OLD as OLD NEW as NEW
for each row
BEGIN
	if (:new.GROUP_ID='E_SIG' and :new.CAN_VERIFY=1) then
		raise_application_error(-20604, 'Could not add member '||:new.WRK_ID||' to reserved group E_SIG.');
	end if;
END;
/


DROP TRIGGER NEW_SETTING;

CREATE OR REPLACE TRIGGER NEW_SETTING
BEFORE INSERT ON SETTING
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
DECLARE
	v_username   VARCHAR2 (128);
BEGIN   
	SELECT USER INTO v_username FROM DUAL;

    :new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_BY_USER := v_username;

END;
/


DROP TRIGGER NEW_SETTING_PERMISSION;

CREATE OR REPLACE TRIGGER NEW_SETTING_PERMISSION
BEFORE INSERT ON SETTING_PERMISSION
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
DECLARE
	v_username   VARCHAR2 (128);
BEGIN   
	SELECT USER INTO v_username FROM DUAL;

    :new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_BY_USER := v_username;

END;
/


DROP TRIGGER NEW_SETTING_VALUE;

CREATE OR REPLACE TRIGGER NEW_SETTING_VALUE
BEFORE INSERT ON SETTING_VALUES
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
DECLARE
	v_username   VARCHAR2 (128);
BEGIN   
	SELECT USER INTO v_username FROM DUAL;

    :new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_BY_USER := v_username;

END;
/


DROP TRIGGER NEW_SIGNED_MSG;

CREATE OR REPLACE TRIGGER NEW_SIGNED_MSG
before INSERT ON SIGNED_MSG
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
begin
	SELECT USER INTO v_username FROM DUAL;  
	:new.CREATED_BY_USER :=  v_username ;
	:new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);
end;
/


DROP TRIGGER NEW_SI_RECIPIENT;

CREATE OR REPLACE TRIGGER NEW_SI_RECIPIENT
BEFORE INSERT ON SI_RECIPIENT
REFERENCING OLD AS OLD NEW AS NEW
FOR EACH ROW
DECLARE v_username VARCHAR2 (128);
begin
    SELECT user INTO v_username  FROM dual;
    :new.id := seq_si_recipient.nextval;
    :new.created := SYSTIMESTAMP;   
    :new.CREATED_BY := v_username;
end;
/


DROP TRIGGER NEW_USR_PAT;

CREATE OR REPLACE TRIGGER NEW_USR_PAT
            before INSERT ON USR_PAT
            referencing OLD as OLD NEW as NEW
            FOR EACH ROW
DECLARE v_username VARCHAR2 (128);  
            BEGIN
            	SELECT USER INTO v_username FROM DUAL;

            	:new.MODIFIED := SYSDATE;
            	:new.MODIFIED_BY :=  v_username;
            	:new.CREATED_BY_USER := v_username;
                :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
                :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
                :new.MODIFIED_BY_USER := v_username;

            END;
/


DROP TRIGGER NEW_WAIVER;

CREATE OR REPLACE TRIGGER NEW_WAIVER
BEFORE INSERT ON WAIVER
REFERENCING NEW AS NEW OLD AS OLD
FOR EACH ROW
DECLARE
    v_username   VARCHAR2 (128);
BEGIN   
    SELECT USER INTO v_username FROM DUAL;
    :new.CREATED_BY_USER := v_username;
    :new.CREATED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC (SYSTIMESTAMP);
    :new.MODIFIED_BY_USER := v_username;
END;
/


DROP TRIGGER POST_DELETE_DATA;

CREATE OR REPLACE TRIGGER POST_DELETE_DATA
            after DELETE ON DATA
BEGIN
              for cur in (SELECT distinct q.PAT_ID,i.VISIT_ID FROM VS_QUEUE q, V_PAT_SCHEDULE_IMPL p, VISIT_INFO i where p.PAT_ID=q.PAT_ID and i.FORM=q.FORM and i.INSTANCE=q.INSTANCE and i.COUNTER=q.COUNTER) loop
                update_pat_status(cur.PAT_ID);
                begin
                    update_visit_schedule(cur.PAT_ID);
                    delete from VS_QUEUE where PAT_ID=cur.PAT_ID;
                exception
                    when OTHERS then
                        NULL;
                end;
              end loop;
              for cur in (SELECT distinct q.PAT_ID,i.VISIT_ID FROM VS_QUEUE q, V_PAT_SCHEDULE_IMPL p, VISIT_INFO i, VISIT_UNSCHEDULED s where p.PAT_ID=q.PAT_ID and i.FORM=q.FORM and i.INSTANCE=q.INSTANCE and i.COUNTER=q.COUNTER and i.VISIT_ID=s.VISIT_ID) loop
                update_pat_status(cur.PAT_ID);
                delete from VS_QUEUE where PAT_ID=cur.PAT_ID;
              end loop;
            END;
/


DROP TRIGGER POST_MODIF_DATA;

CREATE OR REPLACE TRIGGER POST_MODIF_DATA
            after UPDATE of VALUE ON DATA
declare
              yea VARCHAR2(24):= 0;
              mon VARCHAR2(24):= 0;
              day VARCHAR2(24):= 0;
              dat DATE;
              specValue NUMBER := 0;
            BEGIN
              for cur in (SELECT distinct q.PAT_ID,i.VISIT_ID FROM VS_QUEUE q, V_PAT_SCHEDULE_IMPL p, VISIT_INFO i where p.PAT_ID=q.PAT_ID and i.FORM=q.FORM and i.INSTANCE=q.INSTANCE and i.COUNTER=q.COUNTER) loop
                specValue := 0;
                update_pat_status(cur.PAT_ID);
            
                begin
                    select SPEC_VALUE into specValue from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISYEA' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                exception
                    when no_data_found then
                        NULL;
                end;
                if (specValue=3) then
                    update_visit_schedule(cur.PAT_ID);
                    delete from VS_QUEUE where PAT_ID=cur.PAT_ID;
                else
                    begin
                        select VALUE into day from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISDAY' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                        select VALUE into mon from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISMON' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                        select VALUE into yea from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISYEA' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                    exception
                        when NO_DATA_FOUND then
                           if day=0 OR mon=0 OR yea=0 then NULL;
                            else DBMS_OUTPUT.PUT_LINE('error in trigger post_new_data - patId: ' || cur.PAT_ID || ' invalid date: ' || day||'.'||mon||'.'||yea);
                            end if;
                   end;
                    begin
                        select TO_DATE(day||'.'||mon||'.'||yea,'DD.MM.YYYY') into dat from DUAL;
                        update_visit_schedule(cur.PAT_ID);
                        delete from VS_QUEUE where PAT_ID=cur.PAT_ID;
                    exception
                        when OTHERS then
                           if day=0 OR mon=0 OR yea=0 then NULL;
                            else DBMS_OUTPUT.PUT_LINE('error in trigger post_new_data - patId: ' || cur.PAT_ID || ' invalid date: ' || day||'.'||mon||'.'||yea);
                            end if;
                    end;
                end if;
              end loop;
            END;
/


DROP TRIGGER POST_NEW_DATA;

CREATE OR REPLACE TRIGGER POST_NEW_DATA
            after INSERT ON DATA
declare
              yea VARCHAR2(24):= 0;
              mon VARCHAR2(24):= 0;
              day VARCHAR2(24):= 0;
              dat DATE;
              specValue NUMBER := 0;
            BEGIN
              for cur in (SELECT distinct q.PAT_ID,i.VISIT_ID FROM VS_QUEUE q, V_PAT_SCHEDULE_IMPL p, VISIT_INFO i where p.PAT_ID=q.PAT_ID and i.FORM=q.FORM and i.INSTANCE=q.INSTANCE and i.COUNTER=q.COUNTER) loop
                specValue := 0;
                update_pat_status(cur.PAT_ID);
                begin
                    select SPEC_VALUE into specValue from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISYEA' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                exception
                    when no_data_found then
                        NULL;
                end;
                if (specValue=3) then
                    update_visit_schedule(cur.PAT_ID);
                    delete from VS_QUEUE where PAT_ID=cur.PAT_ID;
                else
                    begin
                        select VALUE into day from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISDAY' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                        select VALUE into mon from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISMON' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                        select VALUE into yea from DATA d, V_PAT_SCHEDULE s where d.FIELD='VISYEA' and d.PAT_ID=cur.PAT_ID and d.PAT_ID=s.PAT_ID and d.FORM=s.FORM and d.PAGE=s.INSTANCE and d.COUNTER=s.COUNTER and s.VISIT_ID=cur.VISIT_ID;
                    exception
                       when no_data_found then
                            if day=0 OR mon=0 OR yea=0 then NULL;
                            else DBMS_OUTPUT.PUT_LINE('error in trigger post_new_data - patId: ' || cur.PAT_ID || ' invalid date: ' || day||'.'||mon||'.'||yea);
                            end if;
                    end;
                    begin
                        select TO_DATE(day||'.'||mon||'.'||yea,'DD.MM.YYYY') into dat from DUAL;
                        update_visit_schedule(cur.PAT_ID);
                        delete from VS_QUEUE where PAT_ID=cur.PAT_ID;
                    exception
                        when OTHERS then
                            if day=0 OR mon=0 OR yea=0 then NULL;
                            else DBMS_OUTPUT.PUT_LINE('error in trigger post_new_data - patId: ' || cur.PAT_ID || ' invalid date: ' || day||'.'||mon||'.'||yea);
                            end if;
                    end;
                end if;
              end loop;
            END;
/


DROP TRIGGER RELOCATE_CODES;

CREATE OR REPLACE TRIGGER RELOCATE_CODES
before UPDATE of COUNTER ON CODES
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);  
BEGIN
   	:new.MODIFIED := SYSDATE;
	if (:new.REASON is NULL or :new.REASON='new value') then
		:new.REASON := 'auto-relocation ('||TO_CHAR(:old.COUNTER)||'->'||TO_CHAR(:new.COUNTER)||')';
	end if;

	 SELECT USER INTO v_username FROM DUAL;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	INSERT INTO CODES_AUDIT
		(REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, CODE_ID,
		DICT,  CODE_TYPE, CODE1, CODE2, CODE3, CODE4, CODE5, CODE6,
		DECODE1, DECODE2, DECODE3, DECODE4, DECODE5, DECODE6,
		MODIFIED, MODIFIED_BY, STATUS, LOC, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
		(:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.CODE_ID,
		:old.DICT, :old.CODE_TYPE, :old.CODE1, :old.CODE2, :old.CODE3, :old.CODE4, :old.CODE5, :old.CODE6,
		:old.DECODE1, :old.DECODE2, :old.DECODE3, :old.DECODE4, :old.DECODE5, :old.DECODE6,
		:old.MODIFIED, :old.MODIFIED_BY, :old.STATUS, :old.LOC,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER RELOCATE_DATA;

CREATE OR REPLACE TRIGGER RELOCATE_DATA
before UPDATE of COUNTER ON DATA
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
	if (LOCKING_PKG.isFrozen(1, :new.PAT_ID, :new.FORM, :new.PAGE, :new.FIELD)) then
		Raise_application_error(-20200, 'Patient is locked or frozen.');
	end if;

	SELECT USER INTO v_username FROM DUAL;
   	:new.MODIFIED := SYSDATE;
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	if (:new.REASON is NULL or :new.REASON='new value') then
		:new.REASON := 'auto-relocation ('||TO_CHAR(:old.COUNTER)||'->'||TO_CHAR(:new.COUNTER)||')';
	end if;

	INSERT INTO DATA_AUDIT
   		(REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE, SPEC_VALUE, MODIFIED, MODIFIED_BY, COMMENTS, LOCKED, TRANSACT)
	VALUES
   		(:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE, :old.SPEC_VALUE, :old.MODIFIED, :old.MODIFIED_BY, :old.COMMENTS, :old.LOCKED, :old.TRANSACT);
END;
/


DROP TRIGGER RELOCATE_EXT_DATA;

CREATE OR REPLACE TRIGGER RELOCATE_EXT_DATA
before UPDATE of COUNTER ON EXT_DATA
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);  
BEGIN
    :new.MODIFIED := SYSDATE;
	SELECT USER INTO v_username FROM DUAL;  
    :new.MODIFIED_BY_USER :=  v_username;
    :new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	if (:new.REASON is NULL or :new.REASON='new value') then
		:new.REASON := 'auto-relocation ('||TO_CHAR(:old.COUNTER)||'->'||TO_CHAR(:new.COUNTER)||')';
	end if;

	INSERT INTO EXT_DATA_AUDIT
   		(PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE_TYPE,
		 ID, VALUE, COMMENTS, MODIFIED, MODIFIED_BY,
		 REASON, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE_TYPE,
		 :old.ID, :old.VALUE, :old.COMMENTS, :old.MODIFIED, :old.MODIFIED_BY,
		 :old.REASON, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER RELOCATE_OPT_DATA;

CREATE OR REPLACE TRIGGER RELOCATE_OPT_DATA
before UPDATE of COUNTER ON OPT_DATA
referencing OLD as OLD NEW as NEW
for each row
DECLARE v_username VARCHAR2 (128);
BEGIN
    :new.MODIFIED := SYSDATE;
	SELECT USER INTO v_username FROM DUAL;  
	:new.MODIFIED_BY_USER :=  v_username;
	:new.MODIFIED_TIMESTAMP := SYS_EXTRACT_UTC(SYSTIMESTAMP);

	if (:new.REASON is NULL) then
		:new.REASON := 'auto-relocation ('||TO_CHAR(:old.COUNTER)||'->'||TO_CHAR(:new.COUNTER)||')';
	end if;

	INSERT INTO OPT_DATA_AUDIT
   		(REASON, PAT_ID, FIELD, FORM, PAGE, COUNTER, VALUE, MODIFIED, MODIFIED_BY, COMMENTS, ID, STATUS, LOCKED, TRANSACT, CREATED_BY_USER,CREATED_TIMESTAMP,MODIFIED_BY_USER,MODIFIED_TIMESTAMP)
	VALUES
   		(:old.REASON, :old.PAT_ID, :old.FIELD, :old.FORM, :old.PAGE, :old.COUNTER, :old.VALUE, :old.MODIFIED, :old.MODIFIED_BY, :old.COMMENTS, :old.ID, :old.STATUS, :old.LOCKED, :old.TRANSACT,:old.CREATED_BY_USER,:old.CREATED_TIMESTAMP,:old.MODIFIED_BY_USER,:old.MODIFIED_TIMESTAMP);
END;
/


DROP TRIGGER TRG_AUDIT_TRAIL_INSERT;

CREATE OR REPLACE TRIGGER TRG_AUDIT_TRAIL_INSERT
before INSERT ON AUDIT_TRAIL
referencing OLD as OLD NEW as NEW
for each row
declare   
    v_username varchar2(40);
BEGIN
   SELECT user INTO v_username  FROM dual;
   
     :new.audit_id := audit_trail_seq.nextval;
     :new.created := SYS_EXTRACT_UTC(SYSTIMESTAMP);   
     :new.CREATED_BY := v_username;    
END;
/


DROP TRIGGER TRG_AUDIT_TRAIL_UPDATE;

CREATE OR REPLACE TRIGGER TRG_AUDIT_TRAIL_UPDATE
before UPDATE ON AUDIT_TRAIL
referencing OLD as OLD NEW as NEW
for each row
DECLARE
   v_username varchar2(40);    
  BEGIN
  SELECT user INTO v_username  FROM dual;
  :new.modified := SYS_EXTRACT_UTC(SYSTIMESTAMP);
  :new.modified_by := v_username ;
END;
/


DROP TRIGGER TR_D_AT_DATA;

CREATE OR REPLACE TRIGGER TR_D_AT_DATA AFTER DELETE ON DATA FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'PAT_ID', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.PAT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'FIELD', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.FIELD); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'FORM', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.FORM); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'PAGE', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.PAGE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'COUNTER', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.COUNTER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'VALUE', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'SPEC_VALUE', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.SPEC_VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'COMMENTS', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.COMMENTS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'MODIFIED', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host, to_char (:old.MODIFIED,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'MODIFIED_BY', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.MODIFIED_BY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'REASON', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.REASON); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'LOCKED', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.LOCKED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'TRANSACT', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.TRANSACT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'ID', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'MODIFIED_TIMESTAMP', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.MODIFIED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'MODIFIED_BY_USER', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'CREATED_TIMESTAMP', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('DATA','d', :old.id, 'CREATED_BY_USER', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.CREATED_BY_USER);  END;
/


DROP TRIGGER TR_D_AT_DRUG_BATCH;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_BATCH AFTER DELETE ON DRUG_BATCH FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'BLOCKED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCKED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'WAREHOUSE_BATCH_NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WAREHOUSE_BATCH_NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_BATCH_COUNTRY;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_BATCH_COUNTRY AFTER DELETE ON DRUG_BATCH_COUNTRY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'COUNTRY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNTRY_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'BATCH_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BATCH_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_BATCH_COUNTRY','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_COHORT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_COHORT AFTER DELETE ON DRUG_COHORT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_COHORT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_COHORT_ATTRIB AFTER DELETE ON DRUG_COHORT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'COHORT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COHORT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_COHORT_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_CONFIGURATION;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_CONFIGURATION AFTER DELETE ON DRUG_CONFIGURATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_CONFIGURATION','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_DOSE;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_DOSE AFTER DELETE ON DRUG_DOSE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_DOSE','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_ENROLMENT_CAT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_ENROLMENT_CAT AFTER DELETE ON DRUG_ENROLMENT_CAT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'IS_DEFAULT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.IS_DEFAULT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_CAT','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_ENROLMENT_LEVEL;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_ENROLMENT_LEVEL AFTER DELETE ON DRUG_ENROLMENT_LEVEL FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_EXPECTED_QTY;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_EXPECTED_QTY AFTER DELETE ON DRUG_EXPECTED_QTY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'REQUESTED_QTY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REQUESTED_QTY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'UNBLINDED_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBLINDED_KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_EXPECTED_QTY','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_FACTOR;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_FACTOR AFTER DELETE ON DRUG_FACTOR FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'SHORT_NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHORT_NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'FACTOR_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FACTOR_TYPE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_FACTOR_VALUE AFTER DELETE ON DRUG_FACTOR_VALUE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'RATIO', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RATIO); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_FACTOR_VALUE','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_KIT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_KIT AFTER DELETE ON DRUG_KIT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'REPLACES_KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REPLACES_KIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'BLOCK_NUMBER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_NUMBER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'EXPIRATION_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.EXPIRATION_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'DISPENSE_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.DISPENSE_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'PAT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'BATCH_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BATCH_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'VERIFICATION_CODE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VERIFICATION_CODE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'UNBL_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'VISIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VISIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'QUARANTINED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUARANTINED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'PREVIOUS_STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PREVIOUS_STATUS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'VERIFICATION_STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VERIFICATION_STATUS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'COUNT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'RECALL_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.RECALL_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'SORT_KEY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SORT_KEY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'MANUAL_DISPENSED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MANUAL_DISPENSED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'MANUFACTURING_LOT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MANUFACTURING_LOT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_KIT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_KIT_ATTRIB AFTER DELETE ON DRUG_KIT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_KIT_FILEITEM;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_KIT_FILEITEM AFTER DELETE ON DRUG_KIT_FILEITEM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'LINES', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LINES); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'WORKER_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WORKER_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'IMPORT_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.IMPORT_TIMESTAMP,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_KIT_FILEITEM_ATTR;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_KIT_FILEITEM_ATTR AFTER DELETE ON DRUG_KIT_FILEITEM_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_KIT_PROJECT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_KIT_PROJECT AFTER DELETE ON DRUG_KIT_PROJECT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'PROJECT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PROJECT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_PROJECT','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_KIT_TYPE;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_KIT_TYPE AFTER DELETE ON DRUG_KIT_TYPE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'LABEL_MODE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LABEL_MODE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'DOSE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DOSE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'ENROLMENT_LEVEL_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_LEVEL_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'SUBCLASS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBCLASS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'BLINDED_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLINDED_KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'ADDRESS_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ADDRESS_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'TEMP_CONTROLLED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEMP_CONTROLLED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'RANGE_LOW_TEMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANGE_LOW_TEMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'RANGE_HIGH_TEMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANGE_HIGH_TEMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'EXTRA_KIT_LOGIC', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.EXTRA_KIT_LOGIC); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'NON_DRUG', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NON_DRUG); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'BLOCK_SHIPMENT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_SHIPMENT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'BLOCK_SIZE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_SIZE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_KIT_TYPE','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_LOCATION;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_LOCATION AFTER DELETE ON DRUG_LOCATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'ENROLMENT_CATEGORY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_CATEGORY_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION','d', :old.id, 'LOCATION_SEQ_NUMBER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_SEQ_NUMBER);  END;
/


DROP TRIGGER TR_D_AT_DRUG_LOCATION_ATTRIB;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_LOCATION_ATTRIB AFTER DELETE ON DRUG_LOCATION_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_NOTIFICATION;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_NOTIFICATION AFTER DELETE ON DRUG_NOTIFICATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'LOCATION_QUERY_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_QUERY_TYPE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'LOCATION_QUERY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_QUERY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'REPLACER_QUERY_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REPLACER_QUERY_TYPE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'REPLACER_QUERY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REPLACER_QUERY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'TEXT_QUERY_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEXT_QUERY_TYPE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'TEXT_QUERY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEXT_QUERY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'SUBJECT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBJECT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'PREAMBLE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PREAMBLE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'TEXT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEXT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'INTERVAL', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.INTERVAL); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'INTERVAL_UNIT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.INTERVAL_UNIT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'LASTACTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.LASTACTION,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'STARTDATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.STARTDATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'DISABLE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DISABLE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'IS_BLINDED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.IS_BLINDED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'ATTACHMENT_URL', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ATTACHMENT_URL); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'CLASS_NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CLASS_NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'TRIGGER_ON_INSERT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TRIGGER_ON_INSERT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'TRIGGER_ON_UPDATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TRIGGER_ON_UPDATE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'PARAMETER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PARAMETER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'PARAMETER_ACCESS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PARAMETER_ACCESS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_NOTIFICATION','d', :old.id, 'NOTIFIER_OBJECT_PATH', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NOTIFIER_OBJECT_PATH);  END;
/


DROP TRIGGER TR_D_AT_DRUG_PAT_VST_QTY_REQ;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_PAT_VST_QTY_REQ AFTER DELETE ON DRUG_PAT_VST_QTY_REQ FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'PAT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'UNBL_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'QUANTITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUANTITY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'VISIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VISIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_PRIORITY_ASSIGNME;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_PRIORITY_ASSIGNME AFTER DELETE ON DRUG_PRIORITY_ASSIGNMENT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'PRIORITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PRIORITY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'UNBLINDED_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBLINDED_KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'QUANTITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUANTITY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'DOSE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DOSE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_PRIORITY_SUPPLY;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_PRIORITY_SUPPLY AFTER DELETE ON DRUG_PRIORITY_SUPPLY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'PRIORITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PRIORITY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'UNBL_KIT_TYPE_ID_FROM', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID_FROM); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'UNBL_KIT_TYPE_ID_TO', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID_TO); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'QUANTITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUANTITY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'COUNTRY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNTRY_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'ENABLED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENABLED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_RNDNO;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_RNDNO AFTER DELETE ON DRUG_RNDNO FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'RANDOM_SOURCE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANDOM_SOURCE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'SORT_KEY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SORT_KEY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'RANDOMIZATION_ORDER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANDOMIZATION_ORDER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'BLOCK', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'BLOCK_ASSIGNED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_ASSIGNED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'COHORT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COHORT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'RANDOM_WEIGHT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANDOM_WEIGHT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'PAT_NO', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_NO); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'TREATMENTARM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TREATMENTARM_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'PAT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'VISIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VISIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'UNBLINDED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.UNBLINDED_AT,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO','d', :old.id, 'REASON_UNBLINDING', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REASON_UNBLINDING);  END;
/


DROP TRIGGER TR_D_AT_DRUG_RNDNO_ATTRIB;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_RNDNO_ATTRIB AFTER DELETE ON DRUG_RNDNO_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'RNDNO_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RNDNO_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_RNDNO_FACTOR_VALU;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_RNDNO_FACTOR_VALU AFTER DELETE ON DRUG_RNDNO_FACTOR_VALUE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'RNDNO_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RNDNO_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_RNDNO_FILEITEM;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_RNDNO_FILEITEM AFTER DELETE ON DRUG_RNDNO_FILEITEM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'LINES', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LINES); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'WORKER_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WORKER_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'IMPORT_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.IMPORT_TIMESTAMP,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_RNDNO_FILEITEM_AT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_RNDNO_FILEITEM_AT AFTER DELETE ON DRUG_RNDNO_FILEITEM_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_SERVICE;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_SERVICE AFTER DELETE ON DRUG_SERVICE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SERVICE','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER);  END;
/


DROP TRIGGER TR_D_AT_DRUG_SHIPMENT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_SHIPMENT AFTER DELETE ON DRUG_SHIPMENT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'SENDER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SENDER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'FROM_LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FROM_LOCATION_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'TO_LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TO_LOCATION_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'SERVICE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SERVICE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'SHIP_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.SHIP_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'REQUEST_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.REQUEST_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'ACKNOWLEDGEMENT_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.ACKNOWLEDGEMENT_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'LOCATION_ADDRESS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_ADDRESS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'TEMP_CONTROLLED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEMP_CONTROLLED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'TEMP_OUT_OF_RANGE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEMP_OUT_OF_RANGE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_SHIPMENT_ADDRESS;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_SHIPMENT_ADDRESS AFTER DELETE ON DRUG_SHIPMENT_ADDRESS FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'FNAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FNAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'MNAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MNAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'LNAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LNAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'TITLE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TITLE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'ORGANISATION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ORGANISATION); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'STREET', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STREET); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'CITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CITY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'STATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'ZIP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ZIP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'COUNTRY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNTRY_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'PHONE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PHONE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'MPHONE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MPHONE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'FAX', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FAX); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'EMAIL', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.EMAIL); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'NOTES', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NOTES); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_SHIPMENT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_SHIPMENT_ATTRIB AFTER DELETE ON DRUG_SHIPMENT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_STATUS;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_STATUS AFTER DELETE ON DRUG_STATUS FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'SUBCLASS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBCLASS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'LOCATION_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_TYPE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_STATUS','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_SUPPLY_CONF;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_SUPPLY_CONF AFTER DELETE ON DRUG_SUPPLY_CONF FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'ENROLMENT_LEVEL_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_LEVEL_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'ENROLMENT_CATEGORY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_CATEGORY_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'WAREHOUSE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WAREHOUSE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'INITIAL_VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.INITIAL_VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'MIN_VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MIN_VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'MAX_VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MAX_VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'SUBCLASS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBCLASS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_SUPPLY_CONF','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_TREATMENTARM;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_TREATMENTARM AFTER DELETE ON DRUG_TREATMENTARM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'DOSE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DOSE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'TREATMENTARM_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TREATMENTARM_TYPE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_TREATMENTARM','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_UNBLINDING_LOG;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_UNBLINDING_LOG AFTER DELETE ON DRUG_UNBLINDING_LOG FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'WORKER_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WORKER_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'RNDNO_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RNDNO_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'REASON_UNBLINDING', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REASON_UNBLINDING); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TYPE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'UNBLINDED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.UNBLINDED_AT,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNBLINDING_LOG','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_UNPACKAGED_KIT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_UNPACKAGED_KIT AFTER DELETE ON DRUG_UNPACKAGED_KIT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'VERIFICATION_CODE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VERIFICATION_CODE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'SORT_KEY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SORT_KEY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'COUNT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'UNBL_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'BLOCK_NUMBER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_NUMBER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_DRUG_UNPACKAGED_KIT_AT;

CREATE OR REPLACE TRIGGER TR_D_AT_DRUG_UNPACKAGED_KIT_AT AFTER DELETE ON DRUG_UNPACKAGED_KIT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'UNPACKAGED_KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNPACKAGED_KIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','d', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_SETTING_VALUES;

CREATE OR REPLACE TRIGGER TR_D_AT_SETTING_VALUES AFTER DELETE ON SETTING_VALUES FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'SETTING_ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.SETTING_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'LOCATION_ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.LOCATION_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'PARAMETER_ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.PARAMETER_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'VALUE', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.VALUE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'UNIT', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.UNIT); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'MODIFIED', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.MODIFIED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'MODIFIED_BY', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.MODIFIED_BY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'CREATED', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.CREATED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'CREATED_BY', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.CREATED_BY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'CREATED_BY_USER', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'CREATED_TIMESTAMP', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'MODIFIED_BY_USER', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('SETTING_VALUES','d', :old.id, 'MODIFIED_TIMESTAMP', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host,:old.MODIFIED_TIMESTAMP);  END;
/


DROP TRIGGER TR_D_AT_USR_PAT;

CREATE OR REPLACE TRIGGER TR_D_AT_USR_PAT AFTER DELETE ON USR_PAT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'PAT_ID', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.PAT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'SIT_ID', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.SIT_ID); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'CREATED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host, to_char (:old.CREATED,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'CREATED_BY', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.CREATED_BY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'MODIFIED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host, to_char (:old.MODIFIED,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'MODIFIED_BY', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.MODIFIED_BY); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'SIGNED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.SIGNED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'INCLUDED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.INCLUDED); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'SCR_NO', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.SCR_NO); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'PAT_NO', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.PAT_NO); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'INITIALS', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.INITIALS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'BIRTHDATE', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.BIRTHDATE); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'WITHDRAWN_DATE', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host, to_char (:old.WITHDRAWN_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'RANDOM_DATE', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host, to_char (:old.RANDOM_DATE,'DD-MON-YYYY HH24:MI:SS')); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'STATUS', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.STATUS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'STATUS_TS', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.STATUS_TS); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'MODIFIED_TIMESTAMP', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.MODIFIED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'MODIFIED_BY_USER', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.MODIFIED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'CREATED_TIMESTAMP', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.CREATED_TIMESTAMP); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'CREATED_BY_USER', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.CREATED_BY_USER); INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host,old_value)
                 VALUES ('USR_PAT','d', :old.pat_id, 'PP_ID', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host,:old.PP_ID);  END;
/


DROP TRIGGER TR_I_AT_DATA;

CREATE OR REPLACE TRIGGER TR_I_AT_DATA after insert ON DATA FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host) values ('DATA','i',:new.id, :new.id, 'ID', SYSTIMESTAMP, v_username,:new.modified_by,v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_BATCH;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_BATCH AFTER INSERT ON DRUG_BATCH FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_BATCH','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_BATCH_COUNTRY;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_BATCH_COUNTRY AFTER INSERT ON DRUG_BATCH_COUNTRY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_BATCH_COUNTRY','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_COHORT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_COHORT AFTER INSERT ON DRUG_COHORT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_COHORT','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_COHORT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_COHORT_ATTRIB AFTER INSERT ON DRUG_COHORT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_COHORT_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_CONFIGURATION;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_CONFIGURATION AFTER INSERT ON DRUG_CONFIGURATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_CONFIGURATION','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_DOSE;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_DOSE AFTER INSERT ON DRUG_DOSE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_DOSE','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_ENROLMENT_CAT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_ENROLMENT_CAT AFTER INSERT ON DRUG_ENROLMENT_CAT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_ENROLMENT_CAT','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_ENROLMENT_LEVEL;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_ENROLMENT_LEVEL AFTER INSERT ON DRUG_ENROLMENT_LEVEL FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_ENROLMENT_LEVEL','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_EXPECTED_QTY;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_EXPECTED_QTY AFTER INSERT ON DRUG_EXPECTED_QTY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_EXPECTED_QTY','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_FACTOR;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_FACTOR AFTER INSERT ON DRUG_FACTOR FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_FACTOR','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_FACTOR_VALUE AFTER INSERT ON DRUG_FACTOR_VALUE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_FACTOR_VALUE','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_KIT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_KIT AFTER INSERT ON DRUG_KIT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_KIT','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_KIT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_KIT_ATTRIB AFTER INSERT ON DRUG_KIT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_KIT_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_KIT_FILEITEM;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_KIT_FILEITEM AFTER INSERT ON DRUG_KIT_FILEITEM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_KIT_FILEITEM','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_KIT_FILEITEM_ATTR;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_KIT_FILEITEM_ATTR AFTER INSERT ON DRUG_KIT_FILEITEM_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_KIT_FILEITEM_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_KIT_PROJECT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_KIT_PROJECT AFTER INSERT ON DRUG_KIT_PROJECT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_KIT_PROJECT','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_KIT_TYPE;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_KIT_TYPE AFTER INSERT ON DRUG_KIT_TYPE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_KIT_TYPE','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_LOCATION;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_LOCATION AFTER INSERT ON DRUG_LOCATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_LOCATION','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_LOCATION_ATTRIB;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_LOCATION_ATTRIB AFTER INSERT ON DRUG_LOCATION_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_LOCATION_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_NOTIFICATION;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_NOTIFICATION AFTER INSERT ON DRUG_NOTIFICATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_NOTIFICATION','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_PAT_VST_QTY_REQ;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_PAT_VST_QTY_REQ AFTER INSERT ON DRUG_PAT_VST_QTY_REQ FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_PAT_VST_QTY_REQ','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_PRIORITY_ASSIGNME;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_PRIORITY_ASSIGNME AFTER INSERT ON DRUG_PRIORITY_ASSIGNMENT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_PRIORITY_ASSIGNMENT','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_PRIORITY_SUPPLY;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_PRIORITY_SUPPLY AFTER INSERT ON DRUG_PRIORITY_SUPPLY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_PRIORITY_SUPPLY','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_RNDNO;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_RNDNO AFTER INSERT ON DRUG_RNDNO FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_RNDNO','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_RNDNO_ATTRIB;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_RNDNO_ATTRIB AFTER INSERT ON DRUG_RNDNO_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_RNDNO_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_RNDNO_FACTOR_VALU;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_RNDNO_FACTOR_VALU AFTER INSERT ON DRUG_RNDNO_FACTOR_VALUE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_RNDNO_FACTOR_VALUE','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_RNDNO_FILEITEM;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_RNDNO_FILEITEM AFTER INSERT ON DRUG_RNDNO_FILEITEM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_RNDNO_FILEITEM','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_RNDNO_FILEITEM_AT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_RNDNO_FILEITEM_AT AFTER INSERT ON DRUG_RNDNO_FILEITEM_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_RNDNO_FILEITEM_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_SERVICE;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_SERVICE AFTER INSERT ON DRUG_SERVICE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_SERVICE','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_SHIPMENT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_SHIPMENT AFTER INSERT ON DRUG_SHIPMENT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_SHIPMENT','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_SHIPMENT_ADDRESS;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_SHIPMENT_ADDRESS AFTER INSERT ON DRUG_SHIPMENT_ADDRESS FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_SHIPMENT_ADDRESS','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_SHIPMENT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_SHIPMENT_ATTRIB AFTER INSERT ON DRUG_SHIPMENT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_SHIPMENT_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_STATUS;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_STATUS AFTER INSERT ON DRUG_STATUS FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_STATUS','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_SUPPLY_CONF;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_SUPPLY_CONF AFTER INSERT ON DRUG_SUPPLY_CONF FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_SUPPLY_CONF','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_TREATMENTARM;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_TREATMENTARM AFTER INSERT ON DRUG_TREATMENTARM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_TREATMENTARM','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_UNBLINDING_LOG;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_UNBLINDING_LOG AFTER INSERT ON DRUG_UNBLINDING_LOG FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_UNBLINDING_LOG','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_UNPACKAGED_KIT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_UNPACKAGED_KIT AFTER INSERT ON DRUG_UNPACKAGED_KIT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_UNPACKAGED_KIT','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_DRUG_UNPACKAGED_KIT_AT;

CREATE OR REPLACE TRIGGER TR_I_AT_DRUG_UNPACKAGED_KIT_AT AFTER INSERT ON DRUG_UNPACKAGED_KIT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('DRUG_UNPACKAGED_KIT_ATTRIB','i',:new.id, :new.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_SETTING_VALUES;

CREATE OR REPLACE TRIGGER TR_I_AT_SETTING_VALUES after insert ON SETTING_VALUES FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host) values ('SETTING_VALUES','i',:new.id, :new.id, 'ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END, v_application, v_host); END;
/


DROP TRIGGER TR_I_AT_USR_PAT;

CREATE OR REPLACE TRIGGER TR_I_AT_USR_PAT after insert ON USR_PAT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual; INSERT INTO  audit_trail  (table_name, operation, primary_key, new_value, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host) values ('USR_PAT','i',:new.pat_id, :new.pat_id, 'PAT_ID', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host); END;
/


DROP TRIGGER TR_U_AT_DATA;

CREATE OR REPLACE TRIGGER TR_U_AT_DATA after update ON DATA FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128);v_application varchar2 (128);  BEGIN  SELECT user INTO v_username  FROM dual;SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('PAT_ID')
                                      and nvl(:old.PAT_ID,'NULL')
                                           <> nvl (:new.PAT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'PAT_ID', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.PAT_ID,:new.PAT_ID); end if; if updating('FIELD')
                                      and nvl(:old.FIELD,'NULL')
                                           <> nvl (:new.FIELD,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'FIELD', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.FIELD,:new.FIELD); end if; if updating('FORM')
                                      and nvl(:old.FORM,'-9999')
                                           <> nvl (:new.FORM,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'FORM', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.FORM,:new.FORM); end if; if updating('PAGE')
                                      and nvl(:old.PAGE,'-9999')
                                           <> nvl (:new.PAGE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'PAGE', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.PAGE,:new.PAGE); end if; if updating('COUNTER')
                                      and nvl(:old.COUNTER,'-9999')
                                           <> nvl (:new.COUNTER,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'COUNTER', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.COUNTER,:new.COUNTER); end if; if updating('VALUE')
                                      and nvl(:old.VALUE,'NULL')
                                           <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'VALUE', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.VALUE,:new.VALUE); end if; if updating('SPEC_VALUE')
                                      and nvl(:old.SPEC_VALUE,'-9999')
                                           <> nvl (:new.SPEC_VALUE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'SPEC_VALUE', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.SPEC_VALUE,:new.SPEC_VALUE); end if; if updating('COMMENTS')
                                      and nvl(:old.COMMENTS,'NULL')
                                           <> nvl (:new.COMMENTS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'COMMENTS', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.COMMENTS,:new.COMMENTS); end if; if updating('MODIFIED')
                                      and nvl(:old.MODIFIED, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.MODIFIED, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'MODIFIED', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host, to_char (:old.MODIFIED,'DD-MON-YYYY HH24:MI:SS'),
                                                                       to_char (:new.MODIFIED,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('MODIFIED_BY')
                                      and nvl(:old.MODIFIED_BY,'NULL')
                                           <> nvl (:new.MODIFIED_BY,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'MODIFIED_BY', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.MODIFIED_BY,:new.MODIFIED_BY); end if; if updating('REASON')
                                      and nvl(:old.REASON,'NULL')
                                           <> nvl (:new.REASON,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'REASON', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.REASON,:new.REASON); end if; if updating('LOCKED')
                                      and nvl(:old.LOCKED,'-9999')
                                           <> nvl (:new.LOCKED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'LOCKED', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.LOCKED,:new.LOCKED); end if; if updating('TRANSACT')
                                      and nvl(:old.TRANSACT, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.TRANSACT, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'TRANSACT', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.TRANSACT,:new.TRANSACT); end if; if updating('ID')
                                      and nvl(:old.ID,'NULL')
                                           <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'ID', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.ID,:new.ID); end if; if updating('MODIFIED_TIMESTAMP')
                                      and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'MODIFIED_TIMESTAMP', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                      and nvl(:old.MODIFIED_BY_USER,'NULL')
                                           <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'MODIFIED_BY_USER', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                      and nvl(:old.CREATED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.CREATED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'CREATED_TIMESTAMP', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('CREATED_BY_USER')
                                      and nvl(:old.CREATED_BY_USER,'NULL')
                                           <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('DATA','u', :old.id, 'CREATED_BY_USER', SYSTIMESTAMP, v_username,:new.modified_by,v_application,v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_BATCH;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_BATCH AFTER UPDATE ON DRUG_BATCH FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('BLOCKED')
                                          and nvl(:old.BLOCKED,'-9999')
                                               <> nvl (:new.BLOCKED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'BLOCKED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCKED,:new.BLOCKED); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('WAREHOUSE_BATCH_NAME')
                                          and nvl(:old.WAREHOUSE_BATCH_NAME,'NULL')
                                               <> nvl (:new.WAREHOUSE_BATCH_NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'WAREHOUSE_BATCH_NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WAREHOUSE_BATCH_NAME,:new.WAREHOUSE_BATCH_NAME); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_BATCH_COUNTRY;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_BATCH_COUNTRY AFTER UPDATE ON DRUG_BATCH_COUNTRY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('COUNTRY_ID')
                                          and nvl(:old.COUNTRY_ID,'NULL')
                                               <> nvl (:new.COUNTRY_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'COUNTRY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNTRY_ID,:new.COUNTRY_ID); end if; if updating('BATCH_ID')
                                          and nvl(:old.BATCH_ID,'NULL')
                                               <> nvl (:new.BATCH_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'BATCH_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BATCH_ID,:new.BATCH_ID); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_BATCH_COUNTRY','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_COHORT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_COHORT AFTER UPDATE ON DRUG_COHORT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('DESCRIPTION')
                                          and nvl(:old.DESCRIPTION,'NULL')
                                               <> nvl (:new.DESCRIPTION,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION,:new.DESCRIPTION); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_COHORT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_COHORT_ATTRIB AFTER UPDATE ON DRUG_COHORT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('COHORT_ID')
                                          and nvl(:old.COHORT_ID,'NULL')
                                               <> nvl (:new.COHORT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'COHORT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COHORT_ID,:new.COHORT_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_COHORT_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_CONFIGURATION;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_CONFIGURATION AFTER UPDATE ON DRUG_CONFIGURATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_CONFIGURATION','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_DOSE;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_DOSE AFTER UPDATE ON DRUG_DOSE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_DOSE','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_ENROLMENT_CAT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_ENROLMENT_CAT AFTER UPDATE ON DRUG_ENROLMENT_CAT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('IS_DEFAULT')
                                          and nvl(:old.IS_DEFAULT,'-9999')
                                               <> nvl (:new.IS_DEFAULT,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'IS_DEFAULT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.IS_DEFAULT,:new.IS_DEFAULT); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_CAT','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_ENROLMENT_LEVEL;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_ENROLMENT_LEVEL AFTER UPDATE ON DRUG_ENROLMENT_LEVEL FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_ENROLMENT_LEVEL','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_EXPECTED_QTY;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_EXPECTED_QTY AFTER UPDATE ON DRUG_EXPECTED_QTY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('REQUESTED_QTY')
                                          and nvl(:old.REQUESTED_QTY,'-9999')
                                               <> nvl (:new.REQUESTED_QTY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'REQUESTED_QTY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REQUESTED_QTY,:new.REQUESTED_QTY); end if; if updating('UNBLINDED_KIT_TYPE_ID')
                                          and nvl(:old.UNBLINDED_KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.UNBLINDED_KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'UNBLINDED_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBLINDED_KIT_TYPE_ID,:new.UNBLINDED_KIT_TYPE_ID); end if; if updating('SHIPMENT_ID')
                                          and nvl(:old.SHIPMENT_ID,'NULL')
                                               <> nvl (:new.SHIPMENT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID,:new.SHIPMENT_ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_EXPECTED_QTY','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_FACTOR;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_FACTOR AFTER UPDATE ON DRUG_FACTOR FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('SHORT_NAME')
                                          and nvl(:old.SHORT_NAME,'NULL')
                                               <> nvl (:new.SHORT_NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'SHORT_NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHORT_NAME,:new.SHORT_NAME); end if; if updating('FACTOR_TYPE')
                                          and nvl(:old.FACTOR_TYPE,'NULL')
                                               <> nvl (:new.FACTOR_TYPE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'FACTOR_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FACTOR_TYPE,:new.FACTOR_TYPE); end if; if updating('DESCRIPTION')
                                          and nvl(:old.DESCRIPTION,'NULL')
                                               <> nvl (:new.DESCRIPTION,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION,:new.DESCRIPTION); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_FACTOR_VALUE;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_FACTOR_VALUE AFTER UPDATE ON DRUG_FACTOR_VALUE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('DESCRIPTION')
                                          and nvl(:old.DESCRIPTION,'NULL')
                                               <> nvl (:new.DESCRIPTION,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION,:new.DESCRIPTION); end if; if updating('RATIO')
                                          and nvl(:old.RATIO,'-9999')
                                               <> nvl (:new.RATIO,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'RATIO', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RATIO,:new.RATIO); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_FACTOR_VALUE','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_KIT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_KIT AFTER UPDATE ON DRUG_KIT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if; if updating('REPLACES_KIT_ID')
                                          and nvl(:old.REPLACES_KIT_ID,'NULL')
                                               <> nvl (:new.REPLACES_KIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'REPLACES_KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REPLACES_KIT_ID,:new.REPLACES_KIT_ID); end if; if updating('BLOCK_NUMBER')
                                          and nvl(:old.BLOCK_NUMBER,'-9999')
                                               <> nvl (:new.BLOCK_NUMBER,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'BLOCK_NUMBER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_NUMBER,:new.BLOCK_NUMBER); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('SHIPMENT_ID')
                                          and nvl(:old.SHIPMENT_ID,'NULL')
                                               <> nvl (:new.SHIPMENT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID,:new.SHIPMENT_ID); end if; if updating('STATUS_ID')
                                          and nvl(:old.STATUS_ID,'NULL')
                                               <> nvl (:new.STATUS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID,:new.STATUS_ID); end if; if updating('EXPIRATION_DATE')
                                          and nvl(:old.EXPIRATION_DATE, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.EXPIRATION_DATE, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'EXPIRATION_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.EXPIRATION_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.EXPIRATION_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('DISPENSE_DATE')
                                          and nvl(:old.DISPENSE_DATE, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.DISPENSE_DATE, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'DISPENSE_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.DISPENSE_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.DISPENSE_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('PAT_ID')
                                          and nvl(:old.PAT_ID,'NULL')
                                               <> nvl (:new.PAT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'PAT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_ID,:new.PAT_ID); end if; if updating('LOCATION_ID')
                                          and nvl(:old.LOCATION_ID,'NULL')
                                               <> nvl (:new.LOCATION_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_ID,:new.LOCATION_ID); end if; if updating('BATCH_ID')
                                          and nvl(:old.BATCH_ID,'NULL')
                                               <> nvl (:new.BATCH_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'BATCH_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BATCH_ID,:new.BATCH_ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VERIFICATION_CODE')
                                          and nvl(:old.VERIFICATION_CODE,'NULL')
                                               <> nvl (:new.VERIFICATION_CODE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'VERIFICATION_CODE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VERIFICATION_CODE,:new.VERIFICATION_CODE); end if; if updating('UNBL_KIT_TYPE_ID')
                                          and nvl(:old.UNBL_KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.UNBL_KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'UNBL_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID,:new.UNBL_KIT_TYPE_ID); end if; if updating('VISIT_ID')
                                          and nvl(:old.VISIT_ID,'NULL')
                                               <> nvl (:new.VISIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'VISIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VISIT_ID,:new.VISIT_ID); end if; if updating('KIT_TYPE_ID')
                                          and nvl(:old.KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_TYPE_ID,:new.KIT_TYPE_ID); end if; if updating('QUARANTINED')
                                          and nvl(:old.QUARANTINED,'-9999')
                                               <> nvl (:new.QUARANTINED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'QUARANTINED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUARANTINED,:new.QUARANTINED); end if; if updating('PREVIOUS_STATUS_ID')
                                          and nvl(:old.PREVIOUS_STATUS_ID,'NULL')
                                               <> nvl (:new.PREVIOUS_STATUS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'PREVIOUS_STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PREVIOUS_STATUS_ID,:new.PREVIOUS_STATUS_ID); end if; if updating('VERIFICATION_STATUS_ID')
                                          and nvl(:old.VERIFICATION_STATUS_ID,'NULL')
                                               <> nvl (:new.VERIFICATION_STATUS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'VERIFICATION_STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VERIFICATION_STATUS_ID,:new.VERIFICATION_STATUS_ID); end if; if updating('COUNT')
                                          and nvl(:old.COUNT,'-9999')
                                               <> nvl (:new.COUNT,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'COUNT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNT,:new.COUNT); end if; if updating('RECALL_DATE')
                                          and nvl(:old.RECALL_DATE, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.RECALL_DATE, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'RECALL_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.RECALL_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.RECALL_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('SORT_KEY')
                                          and nvl(:old.SORT_KEY,'-9999')
                                               <> nvl (:new.SORT_KEY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'SORT_KEY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SORT_KEY,:new.SORT_KEY); end if; if updating('FILEITEM_ID')
                                          and nvl(:old.FILEITEM_ID,'NULL')
                                               <> nvl (:new.FILEITEM_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID,:new.FILEITEM_ID); end if; if updating('MANUAL_DISPENSED')
                                          and nvl(:old.MANUAL_DISPENSED,'-9999')
                                               <> nvl (:new.MANUAL_DISPENSED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'MANUAL_DISPENSED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MANUAL_DISPENSED,:new.MANUAL_DISPENSED); end if; if updating('MANUFACTURING_LOT')
                                          and nvl(:old.MANUFACTURING_LOT,'NULL')
                                               <> nvl (:new.MANUFACTURING_LOT,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'MANUFACTURING_LOT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MANUFACTURING_LOT,:new.MANUFACTURING_LOT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_KIT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_KIT_ATTRIB AFTER UPDATE ON DRUG_KIT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('KIT_ID')
                                          and nvl(:old.KIT_ID,'NULL')
                                               <> nvl (:new.KIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_ID,:new.KIT_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_KIT_FILEITEM;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_KIT_FILEITEM AFTER UPDATE ON DRUG_KIT_FILEITEM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('LINES')
                                          and nvl(:old.LINES,'-9999')
                                               <> nvl (:new.LINES,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'LINES', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LINES,:new.LINES); end if; if updating('WORKER_ID')
                                          and nvl(:old.WORKER_ID,'NULL')
                                               <> nvl (:new.WORKER_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'WORKER_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WORKER_ID,:new.WORKER_ID); end if; if updating('IMPORT_TIMESTAMP')
                                          and nvl(:old.IMPORT_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.IMPORT_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'IMPORT_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.IMPORT_TIMESTAMP,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.IMPORT_TIMESTAMP,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_KIT_FILEITEM_ATTR;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_KIT_FILEITEM_ATTR AFTER UPDATE ON DRUG_KIT_FILEITEM_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('FILEITEM_ID')
                                          and nvl(:old.FILEITEM_ID,'NULL')
                                               <> nvl (:new.FILEITEM_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID,:new.FILEITEM_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_KIT_PROJECT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_KIT_PROJECT AFTER UPDATE ON DRUG_KIT_PROJECT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('KIT_ID')
                                          and nvl(:old.KIT_ID,'NULL')
                                               <> nvl (:new.KIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_ID,:new.KIT_ID); end if; if updating('PROJECT_ID')
                                          and nvl(:old.PROJECT_ID,'NULL')
                                               <> nvl (:new.PROJECT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'PROJECT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PROJECT_ID,:new.PROJECT_ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_PROJECT','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_KIT_TYPE;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_KIT_TYPE AFTER UPDATE ON DRUG_KIT_TYPE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('LABEL_MODE')
                                          and nvl(:old.LABEL_MODE,'NULL')
                                               <> nvl (:new.LABEL_MODE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'LABEL_MODE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LABEL_MODE,:new.LABEL_MODE); end if; if updating('DOSE')
                                          and nvl(:old.DOSE,'-9999')
                                               <> nvl (:new.DOSE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'DOSE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DOSE,:new.DOSE); end if; if updating('ENROLMENT_LEVEL_ID')
                                          and nvl(:old.ENROLMENT_LEVEL_ID,'NULL')
                                               <> nvl (:new.ENROLMENT_LEVEL_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'ENROLMENT_LEVEL_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_LEVEL_ID,:new.ENROLMENT_LEVEL_ID); end if; if updating('SUBCLASS')
                                          and nvl(:old.SUBCLASS,'NULL')
                                               <> nvl (:new.SUBCLASS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'SUBCLASS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBCLASS,:new.SUBCLASS); end if; if updating('BLINDED_KIT_TYPE_ID')
                                          and nvl(:old.BLINDED_KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.BLINDED_KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'BLINDED_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLINDED_KIT_TYPE_ID,:new.BLINDED_KIT_TYPE_ID); end if; if updating('DESCRIPTION')
                                          and nvl(:old.DESCRIPTION,'NULL')
                                               <> nvl (:new.DESCRIPTION,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION,:new.DESCRIPTION); end if; if updating('ADDRESS_TYPE_ID')
                                          and nvl(:old.ADDRESS_TYPE_ID,'NULL')
                                               <> nvl (:new.ADDRESS_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'ADDRESS_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ADDRESS_TYPE_ID,:new.ADDRESS_TYPE_ID); end if; if updating('TEMP_CONTROLLED')
                                          and nvl(:old.TEMP_CONTROLLED,'-9999')
                                               <> nvl (:new.TEMP_CONTROLLED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'TEMP_CONTROLLED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEMP_CONTROLLED,:new.TEMP_CONTROLLED); end if; if updating('RANGE_LOW_TEMP')
                                          and nvl(:old.RANGE_LOW_TEMP,'-9999')
                                               <> nvl (:new.RANGE_LOW_TEMP,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'RANGE_LOW_TEMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANGE_LOW_TEMP,:new.RANGE_LOW_TEMP); end if; if updating('RANGE_HIGH_TEMP')
                                          and nvl(:old.RANGE_HIGH_TEMP,'-9999')
                                               <> nvl (:new.RANGE_HIGH_TEMP,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'RANGE_HIGH_TEMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANGE_HIGH_TEMP,:new.RANGE_HIGH_TEMP); end if; if updating('EXTRA_KIT_LOGIC')
                                          and nvl(:old.EXTRA_KIT_LOGIC,'NULL')
                                               <> nvl (:new.EXTRA_KIT_LOGIC,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'EXTRA_KIT_LOGIC', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.EXTRA_KIT_LOGIC,:new.EXTRA_KIT_LOGIC); end if; if updating('NON_DRUG')
                                          and nvl(:old.NON_DRUG,'-9999')
                                               <> nvl (:new.NON_DRUG,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'NON_DRUG', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NON_DRUG,:new.NON_DRUG); end if; if updating('BLOCK_SHIPMENT')
                                          and nvl(:old.BLOCK_SHIPMENT,'-9999')
                                               <> nvl (:new.BLOCK_SHIPMENT,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'BLOCK_SHIPMENT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_SHIPMENT,:new.BLOCK_SHIPMENT); end if; if updating('BLOCK_SIZE')
                                          and nvl(:old.BLOCK_SIZE,'-9999')
                                               <> nvl (:new.BLOCK_SIZE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'BLOCK_SIZE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_SIZE,:new.BLOCK_SIZE); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_KIT_TYPE','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_LOCATION;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_LOCATION AFTER UPDATE ON DRUG_LOCATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('STATUS_ID')
                                          and nvl(:old.STATUS_ID,'NULL')
                                               <> nvl (:new.STATUS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID,:new.STATUS_ID); end if; if updating('ENROLMENT_CATEGORY_ID')
                                          and nvl(:old.ENROLMENT_CATEGORY_ID,'NULL')
                                               <> nvl (:new.ENROLMENT_CATEGORY_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'ENROLMENT_CATEGORY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_CATEGORY_ID,:new.ENROLMENT_CATEGORY_ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if; if updating('LOCATION_SEQ_NUMBER')
                                          and nvl(:old.LOCATION_SEQ_NUMBER,'-9999')
                                               <> nvl (:new.LOCATION_SEQ_NUMBER,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION','u', :old.id, 'LOCATION_SEQ_NUMBER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_SEQ_NUMBER,:new.LOCATION_SEQ_NUMBER); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_LOCATION_ATTRIB;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_LOCATION_ATTRIB AFTER UPDATE ON DRUG_LOCATION_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('LOCATION_ID')
                                          and nvl(:old.LOCATION_ID,'NULL')
                                               <> nvl (:new.LOCATION_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_ID,:new.LOCATION_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_LOCATION_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_NOTIFICATION;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_NOTIFICATION AFTER UPDATE ON DRUG_NOTIFICATION FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('LOCATION_QUERY_TYPE')
                                          and nvl(:old.LOCATION_QUERY_TYPE,'NULL')
                                               <> nvl (:new.LOCATION_QUERY_TYPE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'LOCATION_QUERY_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_QUERY_TYPE,:new.LOCATION_QUERY_TYPE); end if; if updating('LOCATION_QUERY')
                                          and nvl(:old.LOCATION_QUERY,'NULL')
                                               <> nvl (:new.LOCATION_QUERY,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'LOCATION_QUERY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_QUERY,:new.LOCATION_QUERY); end if; if updating('REPLACER_QUERY_TYPE')
                                          and nvl(:old.REPLACER_QUERY_TYPE,'NULL')
                                               <> nvl (:new.REPLACER_QUERY_TYPE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'REPLACER_QUERY_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REPLACER_QUERY_TYPE,:new.REPLACER_QUERY_TYPE); end if; if updating('REPLACER_QUERY')
                                          and nvl(:old.REPLACER_QUERY,'NULL')
                                               <> nvl (:new.REPLACER_QUERY,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'REPLACER_QUERY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REPLACER_QUERY,:new.REPLACER_QUERY); end if; if updating('TEXT_QUERY_TYPE')
                                          and nvl(:old.TEXT_QUERY_TYPE,'NULL')
                                               <> nvl (:new.TEXT_QUERY_TYPE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'TEXT_QUERY_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEXT_QUERY_TYPE,:new.TEXT_QUERY_TYPE); end if; if updating('TEXT_QUERY')
                                          and nvl(:old.TEXT_QUERY,'NULL')
                                               <> nvl (:new.TEXT_QUERY,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'TEXT_QUERY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEXT_QUERY,:new.TEXT_QUERY); end if; if updating('SUBJECT')
                                          and nvl(:old.SUBJECT,'NULL')
                                               <> nvl (:new.SUBJECT,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'SUBJECT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBJECT,:new.SUBJECT); end if; if updating('PREAMBLE')
                                          and nvl(:old.PREAMBLE,'NULL')
                                               <> nvl (:new.PREAMBLE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'PREAMBLE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PREAMBLE,:new.PREAMBLE); end if; if updating('TEXT')
                                          and nvl(:old.TEXT,'NULL')
                                               <> nvl (:new.TEXT,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'TEXT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEXT,:new.TEXT); end if; if updating('INTERVAL')
                                          and nvl(:old.INTERVAL,'-9999')
                                               <> nvl (:new.INTERVAL,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'INTERVAL', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.INTERVAL,:new.INTERVAL); end if; if updating('INTERVAL_UNIT')
                                          and nvl(:old.INTERVAL_UNIT,'NULL')
                                               <> nvl (:new.INTERVAL_UNIT,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'INTERVAL_UNIT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.INTERVAL_UNIT,:new.INTERVAL_UNIT); end if; if updating('LASTACTION')
                                          and nvl(:old.LASTACTION, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.LASTACTION, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'LASTACTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.LASTACTION,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.LASTACTION,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('STARTDATE')
                                          and nvl(:old.STARTDATE, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.STARTDATE, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'STARTDATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.STARTDATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.STARTDATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('DISABLE')
                                          and nvl(:old.DISABLE,'-9999')
                                               <> nvl (:new.DISABLE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'DISABLE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DISABLE,:new.DISABLE); end if; if updating('IS_BLINDED')
                                          and nvl(:old.IS_BLINDED,'-9999')
                                               <> nvl (:new.IS_BLINDED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'IS_BLINDED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.IS_BLINDED,:new.IS_BLINDED); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if; if updating('ATTACHMENT_URL')
                                          and nvl(:old.ATTACHMENT_URL,'NULL')
                                               <> nvl (:new.ATTACHMENT_URL,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'ATTACHMENT_URL', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ATTACHMENT_URL,:new.ATTACHMENT_URL); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('CLASS_NAME')
                                          and nvl(:old.CLASS_NAME,'NULL')
                                               <> nvl (:new.CLASS_NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'CLASS_NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CLASS_NAME,:new.CLASS_NAME); end if; if updating('TRIGGER_ON_INSERT')
                                          and nvl(:old.TRIGGER_ON_INSERT,'-9999')
                                               <> nvl (:new.TRIGGER_ON_INSERT,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'TRIGGER_ON_INSERT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TRIGGER_ON_INSERT,:new.TRIGGER_ON_INSERT); end if; if updating('TRIGGER_ON_UPDATE')
                                          and nvl(:old.TRIGGER_ON_UPDATE,'-9999')
                                               <> nvl (:new.TRIGGER_ON_UPDATE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'TRIGGER_ON_UPDATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TRIGGER_ON_UPDATE,:new.TRIGGER_ON_UPDATE); end if; if updating('PARAMETER')
                                          and nvl(:old.PARAMETER,'NULL')
                                               <> nvl (:new.PARAMETER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'PARAMETER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PARAMETER,:new.PARAMETER); end if; if updating('PARAMETER_ACCESS')
                                          and nvl(:old.PARAMETER_ACCESS,'NULL')
                                               <> nvl (:new.PARAMETER_ACCESS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'PARAMETER_ACCESS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PARAMETER_ACCESS,:new.PARAMETER_ACCESS); end if; if updating('NOTIFIER_OBJECT_PATH')
                                          and nvl(:old.NOTIFIER_OBJECT_PATH,'NULL')
                                               <> nvl (:new.NOTIFIER_OBJECT_PATH,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_NOTIFICATION','u', :old.id, 'NOTIFIER_OBJECT_PATH', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NOTIFIER_OBJECT_PATH,:new.NOTIFIER_OBJECT_PATH); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_PAT_VST_QTY_REQ;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_PAT_VST_QTY_REQ AFTER UPDATE ON DRUG_PAT_VST_QTY_REQ FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('PAT_ID')
                                          and nvl(:old.PAT_ID,'NULL')
                                               <> nvl (:new.PAT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'PAT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_ID,:new.PAT_ID); end if; if updating('UNBL_KIT_TYPE_ID')
                                          and nvl(:old.UNBL_KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.UNBL_KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'UNBL_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID,:new.UNBL_KIT_TYPE_ID); end if; if updating('QUANTITY')
                                          and nvl(:old.QUANTITY,'-9999')
                                               <> nvl (:new.QUANTITY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'QUANTITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUANTITY,:new.QUANTITY); end if; if updating('VISIT_ID')
                                          and nvl(:old.VISIT_ID,'NULL')
                                               <> nvl (:new.VISIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'VISIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VISIT_ID,:new.VISIT_ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PAT_VST_QTY_REQ','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_PRIORITY_ASSIGNME;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_PRIORITY_ASSIGNME AFTER UPDATE ON DRUG_PRIORITY_ASSIGNMENT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('PRIORITY')
                                          and nvl(:old.PRIORITY,'-9999')
                                               <> nvl (:new.PRIORITY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'PRIORITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PRIORITY,:new.PRIORITY); end if; if updating('UNBLINDED_KIT_TYPE_ID')
                                          and nvl(:old.UNBLINDED_KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.UNBLINDED_KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'UNBLINDED_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBLINDED_KIT_TYPE_ID,:new.UNBLINDED_KIT_TYPE_ID); end if; if updating('QUANTITY')
                                          and nvl(:old.QUANTITY,'-9999')
                                               <> nvl (:new.QUANTITY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'QUANTITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUANTITY,:new.QUANTITY); end if; if updating('DOSE_ID')
                                          and nvl(:old.DOSE_ID,'NULL')
                                               <> nvl (:new.DOSE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'DOSE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DOSE_ID,:new.DOSE_ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_ASSIGNMENT','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_PRIORITY_SUPPLY;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_PRIORITY_SUPPLY AFTER UPDATE ON DRUG_PRIORITY_SUPPLY FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('PRIORITY')
                                          and nvl(:old.PRIORITY,'-9999')
                                               <> nvl (:new.PRIORITY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'PRIORITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PRIORITY,:new.PRIORITY); end if; if updating('UNBL_KIT_TYPE_ID_FROM')
                                          and nvl(:old.UNBL_KIT_TYPE_ID_FROM,'NULL')
                                               <> nvl (:new.UNBL_KIT_TYPE_ID_FROM,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'UNBL_KIT_TYPE_ID_FROM', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID_FROM,:new.UNBL_KIT_TYPE_ID_FROM); end if; if updating('UNBL_KIT_TYPE_ID_TO')
                                          and nvl(:old.UNBL_KIT_TYPE_ID_TO,'NULL')
                                               <> nvl (:new.UNBL_KIT_TYPE_ID_TO,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'UNBL_KIT_TYPE_ID_TO', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID_TO,:new.UNBL_KIT_TYPE_ID_TO); end if; if updating('QUANTITY')
                                          and nvl(:old.QUANTITY,'-9999')
                                               <> nvl (:new.QUANTITY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'QUANTITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.QUANTITY,:new.QUANTITY); end if; if updating('COUNTRY_ID')
                                          and nvl(:old.COUNTRY_ID,'NULL')
                                               <> nvl (:new.COUNTRY_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'COUNTRY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNTRY_ID,:new.COUNTRY_ID); end if; if updating('ENABLED')
                                          and nvl(:old.ENABLED,'-9999')
                                               <> nvl (:new.ENABLED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'ENABLED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENABLED,:new.ENABLED); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_PRIORITY_SUPPLY','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_RNDNO;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_RNDNO AFTER UPDATE ON DRUG_RNDNO FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('RANDOM_SOURCE')
                                          and nvl(:old.RANDOM_SOURCE,'NULL')
                                               <> nvl (:new.RANDOM_SOURCE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'RANDOM_SOURCE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANDOM_SOURCE,:new.RANDOM_SOURCE); end if; if updating('SORT_KEY')
                                          and nvl(:old.SORT_KEY,'-9999')
                                               <> nvl (:new.SORT_KEY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'SORT_KEY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SORT_KEY,:new.SORT_KEY); end if; if updating('RANDOMIZATION_ORDER')
                                          and nvl(:old.RANDOMIZATION_ORDER,'-9999')
                                               <> nvl (:new.RANDOMIZATION_ORDER,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'RANDOMIZATION_ORDER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANDOMIZATION_ORDER,:new.RANDOMIZATION_ORDER); end if; if updating('FILEITEM_ID')
                                          and nvl(:old.FILEITEM_ID,'NULL')
                                               <> nvl (:new.FILEITEM_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID,:new.FILEITEM_ID); end if; if updating('BLOCK')
                                          and nvl(:old.BLOCK,'-9999')
                                               <> nvl (:new.BLOCK,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'BLOCK', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK,:new.BLOCK); end if; if updating('BLOCK_ASSIGNED')
                                          and nvl(:old.BLOCK_ASSIGNED,'-9999')
                                               <> nvl (:new.BLOCK_ASSIGNED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'BLOCK_ASSIGNED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_ASSIGNED,:new.BLOCK_ASSIGNED); end if; if updating('COHORT_ID')
                                          and nvl(:old.COHORT_ID,'NULL')
                                               <> nvl (:new.COHORT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'COHORT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COHORT_ID,:new.COHORT_ID); end if; if updating('RANDOM_WEIGHT')
                                          and nvl(:old.RANDOM_WEIGHT,'-9999')
                                               <> nvl (:new.RANDOM_WEIGHT,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'RANDOM_WEIGHT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RANDOM_WEIGHT,:new.RANDOM_WEIGHT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('PAT_NO')
                                          and nvl(:old.PAT_NO,'-9999')
                                               <> nvl (:new.PAT_NO,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'PAT_NO', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_NO,:new.PAT_NO); end if; if updating('STATUS_ID')
                                          and nvl(:old.STATUS_ID,'NULL')
                                               <> nvl (:new.STATUS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID,:new.STATUS_ID); end if; if updating('TREATMENTARM_ID')
                                          and nvl(:old.TREATMENTARM_ID,'NULL')
                                               <> nvl (:new.TREATMENTARM_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'TREATMENTARM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TREATMENTARM_ID,:new.TREATMENTARM_ID); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('PAT_ID')
                                          and nvl(:old.PAT_ID,'NULL')
                                               <> nvl (:new.PAT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'PAT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PAT_ID,:new.PAT_ID); end if; if updating('VISIT_ID')
                                          and nvl(:old.VISIT_ID,'NULL')
                                               <> nvl (:new.VISIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'VISIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VISIT_ID,:new.VISIT_ID); end if; if updating('UNBLINDED_AT')
                                          and nvl(:old.UNBLINDED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.UNBLINDED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'UNBLINDED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.UNBLINDED_AT,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.UNBLINDED_AT,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('REASON_UNBLINDING')
                                          and nvl(:old.REASON_UNBLINDING,'NULL')
                                               <> nvl (:new.REASON_UNBLINDING,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO','u', :old.id, 'REASON_UNBLINDING', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REASON_UNBLINDING,:new.REASON_UNBLINDING); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_RNDNO_ATTRIB;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_RNDNO_ATTRIB AFTER UPDATE ON DRUG_RNDNO_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('RNDNO_ID')
                                          and nvl(:old.RNDNO_ID,'NULL')
                                               <> nvl (:new.RNDNO_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'RNDNO_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RNDNO_ID,:new.RNDNO_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_RNDNO_FACTOR_VALU;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_RNDNO_FACTOR_VALU AFTER UPDATE ON DRUG_RNDNO_FACTOR_VALUE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('RNDNO_ID')
                                          and nvl(:old.RNDNO_ID,'NULL')
                                               <> nvl (:new.RNDNO_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'RNDNO_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RNDNO_ID,:new.RNDNO_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FACTOR_VALUE','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_RNDNO_FILEITEM;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_RNDNO_FILEITEM AFTER UPDATE ON DRUG_RNDNO_FILEITEM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('LINES')
                                          and nvl(:old.LINES,'-9999')
                                               <> nvl (:new.LINES,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'LINES', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LINES,:new.LINES); end if; if updating('WORKER_ID')
                                          and nvl(:old.WORKER_ID,'NULL')
                                               <> nvl (:new.WORKER_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'WORKER_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WORKER_ID,:new.WORKER_ID); end if; if updating('IMPORT_TIMESTAMP')
                                          and nvl(:old.IMPORT_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.IMPORT_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'IMPORT_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.IMPORT_TIMESTAMP,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.IMPORT_TIMESTAMP,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_RNDNO_FILEITEM_AT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_RNDNO_FILEITEM_AT AFTER UPDATE ON DRUG_RNDNO_FILEITEM_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('FILEITEM_ID')
                                          and nvl(:old.FILEITEM_ID,'NULL')
                                               <> nvl (:new.FILEITEM_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID,:new.FILEITEM_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_RNDNO_FILEITEM_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_SERVICE;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_SERVICE AFTER UPDATE ON DRUG_SERVICE FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SERVICE','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_SHIPMENT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_SHIPMENT AFTER UPDATE ON DRUG_SHIPMENT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('STATUS_ID')
                                          and nvl(:old.STATUS_ID,'NULL')
                                               <> nvl (:new.STATUS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID,:new.STATUS_ID); end if; if updating('SENDER')
                                          and nvl(:old.SENDER,'NULL')
                                               <> nvl (:new.SENDER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'SENDER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SENDER,:new.SENDER); end if; if updating('FROM_LOCATION_ID')
                                          and nvl(:old.FROM_LOCATION_ID,'NULL')
                                               <> nvl (:new.FROM_LOCATION_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'FROM_LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FROM_LOCATION_ID,:new.FROM_LOCATION_ID); end if; if updating('TO_LOCATION_ID')
                                          and nvl(:old.TO_LOCATION_ID,'NULL')
                                               <> nvl (:new.TO_LOCATION_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'TO_LOCATION_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TO_LOCATION_ID,:new.TO_LOCATION_ID); end if; if updating('SERVICE_ID')
                                          and nvl(:old.SERVICE_ID,'NULL')
                                               <> nvl (:new.SERVICE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'SERVICE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SERVICE_ID,:new.SERVICE_ID); end if; if updating('SHIP_DATE')
                                          and nvl(:old.SHIP_DATE, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.SHIP_DATE, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'SHIP_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.SHIP_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.SHIP_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('REQUEST_DATE')
                                          and nvl(:old.REQUEST_DATE, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.REQUEST_DATE, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'REQUEST_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.REQUEST_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.REQUEST_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('ACKNOWLEDGEMENT_DATE')
                                          and nvl(:old.ACKNOWLEDGEMENT_DATE, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.ACKNOWLEDGEMENT_DATE, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'ACKNOWLEDGEMENT_DATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.ACKNOWLEDGEMENT_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.ACKNOWLEDGEMENT_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('LOCATION_ADDRESS_ID')
                                          and nvl(:old.LOCATION_ADDRESS_ID,'NULL')
                                               <> nvl (:new.LOCATION_ADDRESS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'LOCATION_ADDRESS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_ADDRESS_ID,:new.LOCATION_ADDRESS_ID); end if; if updating('TEMP_CONTROLLED')
                                          and nvl(:old.TEMP_CONTROLLED,'-9999')
                                               <> nvl (:new.TEMP_CONTROLLED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'TEMP_CONTROLLED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEMP_CONTROLLED,:new.TEMP_CONTROLLED); end if; if updating('TEMP_OUT_OF_RANGE')
                                          and nvl(:old.TEMP_OUT_OF_RANGE,'-9999')
                                               <> nvl (:new.TEMP_OUT_OF_RANGE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'TEMP_OUT_OF_RANGE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TEMP_OUT_OF_RANGE,:new.TEMP_OUT_OF_RANGE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_SHIPMENT_ADDRESS;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_SHIPMENT_ADDRESS AFTER UPDATE ON DRUG_SHIPMENT_ADDRESS FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('SHIPMENT_ID')
                                          and nvl(:old.SHIPMENT_ID,'NULL')
                                               <> nvl (:new.SHIPMENT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID,:new.SHIPMENT_ID); end if; if updating('FNAME')
                                          and nvl(:old.FNAME,'NULL')
                                               <> nvl (:new.FNAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'FNAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FNAME,:new.FNAME); end if; if updating('MNAME')
                                          and nvl(:old.MNAME,'NULL')
                                               <> nvl (:new.MNAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'MNAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MNAME,:new.MNAME); end if; if updating('LNAME')
                                          and nvl(:old.LNAME,'NULL')
                                               <> nvl (:new.LNAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'LNAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LNAME,:new.LNAME); end if; if updating('TITLE')
                                          and nvl(:old.TITLE,'NULL')
                                               <> nvl (:new.TITLE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'TITLE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TITLE,:new.TITLE); end if; if updating('ORGANISATION')
                                          and nvl(:old.ORGANISATION,'NULL')
                                               <> nvl (:new.ORGANISATION,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'ORGANISATION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ORGANISATION,:new.ORGANISATION); end if; if updating('STREET')
                                          and nvl(:old.STREET,'NULL')
                                               <> nvl (:new.STREET,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'STREET', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STREET,:new.STREET); end if; if updating('CITY')
                                          and nvl(:old.CITY,'NULL')
                                               <> nvl (:new.CITY,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'CITY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CITY,:new.CITY); end if; if updating('STATE')
                                          and nvl(:old.STATE,'NULL')
                                               <> nvl (:new.STATE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'STATE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATE,:new.STATE); end if; if updating('ZIP')
                                          and nvl(:old.ZIP,'NULL')
                                               <> nvl (:new.ZIP,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'ZIP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ZIP,:new.ZIP); end if; if updating('COUNTRY_ID')
                                          and nvl(:old.COUNTRY_ID,'NULL')
                                               <> nvl (:new.COUNTRY_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'COUNTRY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNTRY_ID,:new.COUNTRY_ID); end if; if updating('PHONE')
                                          and nvl(:old.PHONE,'NULL')
                                               <> nvl (:new.PHONE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'PHONE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.PHONE,:new.PHONE); end if; if updating('MPHONE')
                                          and nvl(:old.MPHONE,'NULL')
                                               <> nvl (:new.MPHONE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'MPHONE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MPHONE,:new.MPHONE); end if; if updating('FAX')
                                          and nvl(:old.FAX,'NULL')
                                               <> nvl (:new.FAX,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'FAX', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FAX,:new.FAX); end if; if updating('EMAIL')
                                          and nvl(:old.EMAIL,'NULL')
                                               <> nvl (:new.EMAIL,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'EMAIL', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.EMAIL,:new.EMAIL); end if; if updating('NOTES')
                                          and nvl(:old.NOTES,'NULL')
                                               <> nvl (:new.NOTES,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'NOTES', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NOTES,:new.NOTES); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ADDRESS','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_SHIPMENT_ATTRIB;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_SHIPMENT_ATTRIB AFTER UPDATE ON DRUG_SHIPMENT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('SHIPMENT_ID')
                                          and nvl(:old.SHIPMENT_ID,'NULL')
                                               <> nvl (:new.SHIPMENT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'SHIPMENT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SHIPMENT_ID,:new.SHIPMENT_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SHIPMENT_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_STATUS;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_STATUS AFTER UPDATE ON DRUG_STATUS FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('SUBCLASS')
                                          and nvl(:old.SUBCLASS,'NULL')
                                               <> nvl (:new.SUBCLASS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'SUBCLASS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBCLASS,:new.SUBCLASS); end if; if updating('LOCATION_TYPE')
                                          and nvl(:old.LOCATION_TYPE,'NULL')
                                               <> nvl (:new.LOCATION_TYPE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'LOCATION_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.LOCATION_TYPE,:new.LOCATION_TYPE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_STATUS','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_SUPPLY_CONF;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_SUPPLY_CONF AFTER UPDATE ON DRUG_SUPPLY_CONF FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('ENROLMENT_LEVEL_ID')
                                          and nvl(:old.ENROLMENT_LEVEL_ID,'NULL')
                                               <> nvl (:new.ENROLMENT_LEVEL_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'ENROLMENT_LEVEL_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_LEVEL_ID,:new.ENROLMENT_LEVEL_ID); end if; if updating('ENROLMENT_CATEGORY_ID')
                                          and nvl(:old.ENROLMENT_CATEGORY_ID,'NULL')
                                               <> nvl (:new.ENROLMENT_CATEGORY_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'ENROLMENT_CATEGORY_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ENROLMENT_CATEGORY_ID,:new.ENROLMENT_CATEGORY_ID); end if; if updating('WAREHOUSE_ID')
                                          and nvl(:old.WAREHOUSE_ID,'NULL')
                                               <> nvl (:new.WAREHOUSE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'WAREHOUSE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WAREHOUSE_ID,:new.WAREHOUSE_ID); end if; if updating('INITIAL_VALUE')
                                          and nvl(:old.INITIAL_VALUE,'-9999')
                                               <> nvl (:new.INITIAL_VALUE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'INITIAL_VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.INITIAL_VALUE,:new.INITIAL_VALUE); end if; if updating('MIN_VALUE')
                                          and nvl(:old.MIN_VALUE,'-9999')
                                               <> nvl (:new.MIN_VALUE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'MIN_VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MIN_VALUE,:new.MIN_VALUE); end if; if updating('MAX_VALUE')
                                          and nvl(:old.MAX_VALUE,'-9999')
                                               <> nvl (:new.MAX_VALUE,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'MAX_VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MAX_VALUE,:new.MAX_VALUE); end if; if updating('SUBCLASS')
                                          and nvl(:old.SUBCLASS,'NULL')
                                               <> nvl (:new.SUBCLASS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'SUBCLASS', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SUBCLASS,:new.SUBCLASS); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_SUPPLY_CONF','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_TREATMENTARM;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_TREATMENTARM AFTER UPDATE ON DRUG_TREATMENTARM FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('DESCRIPTION')
                                          and nvl(:old.DESCRIPTION,'NULL')
                                               <> nvl (:new.DESCRIPTION,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'DESCRIPTION', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DESCRIPTION,:new.DESCRIPTION); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('STATUS_ID')
                                          and nvl(:old.STATUS_ID,'NULL')
                                               <> nvl (:new.STATUS_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'STATUS_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.STATUS_ID,:new.STATUS_ID); end if; if updating('DOSE')
                                          and nvl(:old.DOSE,'NULL')
                                               <> nvl (:new.DOSE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'DOSE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.DOSE,:new.DOSE); end if; if updating('TREATMENTARM_TYPE')
                                          and nvl(:old.TREATMENTARM_TYPE,'NULL')
                                               <> nvl (:new.TREATMENTARM_TYPE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'TREATMENTARM_TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TREATMENTARM_TYPE,:new.TREATMENTARM_TYPE); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_TREATMENTARM','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_UNBLINDING_LOG;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_UNBLINDING_LOG AFTER UPDATE ON DRUG_UNBLINDING_LOG FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('WORKER_ID')
                                          and nvl(:old.WORKER_ID,'NULL')
                                               <> nvl (:new.WORKER_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'WORKER_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.WORKER_ID,:new.WORKER_ID); end if; if updating('RNDNO_ID')
                                          and nvl(:old.RNDNO_ID,'NULL')
                                               <> nvl (:new.RNDNO_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'RNDNO_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.RNDNO_ID,:new.RNDNO_ID); end if; if updating('REASON_UNBLINDING')
                                          and nvl(:old.REASON_UNBLINDING,'NULL')
                                               <> nvl (:new.REASON_UNBLINDING,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'REASON_UNBLINDING', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.REASON_UNBLINDING,:new.REASON_UNBLINDING); end if; if updating('TYPE')
                                          and nvl(:old.TYPE,'NULL')
                                               <> nvl (:new.TYPE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'TYPE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.TYPE,:new.TYPE); end if; if updating('UNBLINDED_AT')
                                          and nvl(:old.UNBLINDED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.UNBLINDED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'UNBLINDED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host, to_char (:old.UNBLINDED_AT,'DD-MON-YYYY HH24:MI:SS'),
                                                                           to_char (:new.UNBLINDED_AT,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNBLINDING_LOG','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_UNPACKAGED_KIT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_UNPACKAGED_KIT AFTER UPDATE ON DRUG_UNPACKAGED_KIT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VERIFICATION_CODE')
                                          and nvl(:old.VERIFICATION_CODE,'NULL')
                                               <> nvl (:new.VERIFICATION_CODE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'VERIFICATION_CODE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VERIFICATION_CODE,:new.VERIFICATION_CODE); end if; if updating('SORT_KEY')
                                          and nvl(:old.SORT_KEY,'-9999')
                                               <> nvl (:new.SORT_KEY,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'SORT_KEY', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.SORT_KEY,:new.SORT_KEY); end if; if updating('COUNT')
                                          and nvl(:old.COUNT,'-9999')
                                               <> nvl (:new.COUNT,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'COUNT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.COUNT,:new.COUNT); end if; if updating('KIT_TYPE_ID')
                                          and nvl(:old.KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.KIT_TYPE_ID,:new.KIT_TYPE_ID); end if; if updating('UNBL_KIT_TYPE_ID')
                                          and nvl(:old.UNBL_KIT_TYPE_ID,'NULL')
                                               <> nvl (:new.UNBL_KIT_TYPE_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'UNBL_KIT_TYPE_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNBL_KIT_TYPE_ID,:new.UNBL_KIT_TYPE_ID); end if; if updating('FILEITEM_ID')
                                          and nvl(:old.FILEITEM_ID,'NULL')
                                               <> nvl (:new.FILEITEM_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'FILEITEM_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.FILEITEM_ID,:new.FILEITEM_ID); end if; if updating('BLOCK_NUMBER')
                                          and nvl(:old.BLOCK_NUMBER,'-9999')
                                               <> nvl (:new.BLOCK_NUMBER,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'BLOCK_NUMBER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.BLOCK_NUMBER,:new.BLOCK_NUMBER); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_DRUG_UNPACKAGED_KIT_AT;

CREATE OR REPLACE TRIGGER TR_U_AT_DRUG_UNPACKAGED_KIT_AT AFTER UPDATE ON DRUG_UNPACKAGED_KIT_ATTRIB FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('UNPACKAGED_KIT_ID')
                                          and nvl(:old.UNPACKAGED_KIT_ID,'NULL')
                                               <> nvl (:new.UNPACKAGED_KIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'UNPACKAGED_KIT_ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.UNPACKAGED_KIT_ID,:new.UNPACKAGED_KIT_ID); end if; if updating('NAME')
                                          and nvl(:old.NAME,'NULL')
                                               <> nvl (:new.NAME,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'NAME', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.NAME,:new.NAME); end if; if updating('VALUE')
                                          and nvl(:old.VALUE,'NULL')
                                               <> nvl (:new.VALUE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'VALUE', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.VALUE,:new.VALUE); end if; if updating('MODIFIED')
                                          and nvl(:old.MODIFIED,'NULL')
                                               <> nvl (:new.MODIFIED,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'MODIFIED', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED,:new.MODIFIED); end if; if updating('MODIFIED_AT')
                                          and nvl(:old.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_AT, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'MODIFIED_AT', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_AT,:new.MODIFIED_AT); end if; if updating('ID')
                                          and nvl(:old.ID,'NULL')
                                               <> nvl (:new.ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'ID', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.ID,:new.ID); end if; if updating('CREATED_BY_USER')
                                          and nvl(:old.CREATED_BY_USER,'NULL')
                                               <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'CREATED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                          and nvl(:old.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.CREATED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'CREATED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                          and nvl(:old.MODIFIED_BY_USER,'NULL')
                                               <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'MODIFIED_BY_USER', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('MODIFIED_TIMESTAMP')
                                          and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy'))
                                               <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01-01-1600', 'dd-mm-yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by,audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                     VALUES ('DRUG_UNPACKAGED_KIT_ATTRIB','u', :old.id, 'MODIFIED_TIMESTAMP', SYS_EXTRACT_UTC(SYSTIMESTAMP), v_username, :new.modified, v_application, v_host,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if;  END;
/


DROP TRIGGER TR_U_AT_SETTING_VALUES;

CREATE OR REPLACE TRIGGER TR_U_AT_SETTING_VALUES after update ON SETTING_VALUES FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('ID') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.ID, :new.ID ); end if; if updating('SETTING_ID') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'SETTING_ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.SETTING_ID, :new.SETTING_ID ); end if; if updating('LOCATION_ID') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'LOCATION_ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.LOCATION_ID, :new.LOCATION_ID ); end if; if updating('PARAMETER_ID') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'PARAMETER_ID', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.PARAMETER_ID, :new.PARAMETER_ID ); end if; if updating('VALUE') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'VALUE', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.VALUE, :new.VALUE ); end if; if updating('UNIT') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'UNIT', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.UNIT, :new.UNIT ); end if; if updating('MODIFIED') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'MODIFIED', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.MODIFIED, :new.MODIFIED ); end if; if updating('MODIFIED_BY') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'MODIFIED_BY', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.MODIFIED_BY, :new.MODIFIED_BY ); end if; if updating('CREATED') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'CREATED', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.CREATED, :new.CREATED ); end if; if updating('CREATED_BY') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'CREATED_BY', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.CREATED_BY, :new.CREATED_BY ); end if; if updating('CREATED_BY_USER') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'CREATED_BY_USER', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.CREATED_BY_USER, :new.CREATED_BY_USER ); end if; if updating('CREATED_TIMESTAMP') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'CREATED_TIMESTAMP', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.CREATED_TIMESTAMP, :new.CREATED_TIMESTAMP ); end if; if updating('MODIFIED_BY_USER') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'MODIFIED_BY_USER', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.MODIFIED_BY_USER, :new.MODIFIED_BY_USER ); end if; if updating('MODIFIED_TIMESTAMP') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('SETTING_VALUES','u', :old.id, 'MODIFIED_TIMESTAMP', SYSTIMESTAMP, v_username, CASE WHEN v_username IS NULL OR v_username = 'P90020903' THEN :new.modified_by ELSE NULL END,v_application, v_host ,:old.MODIFIED_TIMESTAMP, :new.MODIFIED_TIMESTAMP ); end if;  END;
/


DROP TRIGGER TR_U_AT_USR_PAT;

CREATE OR REPLACE TRIGGER TR_U_AT_USR_PAT after update ON USR_PAT FOR EACH ROW
declare v_username varchar2(40); v_host varchar2 (128); v_application varchar2 (128); BEGIN  SELECT user INTO v_username  FROM dual; SELECT program INTO v_application FROM v$session WHERE audsid=userenv('sessionid'); SELECT SYS_CONTEXT ('USERENV', 'SERVER_HOST') into v_host  FROM dual;if updating('PAT_ID')
                                      and nvl(:old.PAT_ID,'NULL')
                                           <> nvl (:new.PAT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'PAT_ID', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.PAT_ID,:new.PAT_ID); end if; if updating('SIT_ID')
                                      and nvl(:old.SIT_ID,'NULL')
                                           <> nvl (:new.SIT_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'SIT_ID', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.SIT_ID,:new.SIT_ID); end if; if updating('CREATED')
                                      and nvl(:old.CREATED, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.CREATED, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'CREATED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host , to_char (:old.CREATED,'DD-MON-YYYY HH24:MI:SS'),
                                                                       to_char (:new.CREATED,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('CREATED_BY')
                                      and nvl(:old.CREATED_BY,'NULL')
                                           <> nvl (:new.CREATED_BY,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'CREATED_BY', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.CREATED_BY,:new.CREATED_BY); end if; if updating('MODIFIED')
                                      and nvl(:old.MODIFIED, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.MODIFIED, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'MODIFIED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host , to_char (:old.MODIFIED,'DD-MON-YYYY HH24:MI:SS'),
                                                                       to_char (:new.MODIFIED,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('MODIFIED_BY')
                                      and nvl(:old.MODIFIED_BY,'NULL')
                                           <> nvl (:new.MODIFIED_BY,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'MODIFIED_BY', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.MODIFIED_BY,:new.MODIFIED_BY); end if; if updating('SIGNED')
                                      and nvl(:old.SIGNED,'-9999')
                                           <> nvl (:new.SIGNED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'SIGNED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.SIGNED,:new.SIGNED); end if; if updating('INCLUDED')
                                      and nvl(:old.INCLUDED,'-9999')
                                           <> nvl (:new.INCLUDED,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'INCLUDED', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.INCLUDED,:new.INCLUDED); end if; if updating('SCR_NO')
                                      and nvl(:old.SCR_NO,'NULL')
                                           <> nvl (:new.SCR_NO,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'SCR_NO', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.SCR_NO,:new.SCR_NO); end if; if updating('PAT_NO')
                                      and nvl(:old.PAT_NO,'-9999')
                                           <> nvl (:new.PAT_NO,'-9999') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'PAT_NO', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.PAT_NO,:new.PAT_NO); end if; if updating('INITIALS')
                                      and nvl(:old.INITIALS,'NULL')
                                           <> nvl (:new.INITIALS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'INITIALS', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.INITIALS,:new.INITIALS); end if; if updating('BIRTHDATE')
                                      and nvl(:old.BIRTHDATE,'NULL')
                                           <> nvl (:new.BIRTHDATE,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'BIRTHDATE', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.BIRTHDATE,:new.BIRTHDATE); end if; if updating('WITHDRAWN_DATE')
                                      and nvl(:old.WITHDRAWN_DATE, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.WITHDRAWN_DATE, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'WITHDRAWN_DATE', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host , to_char (:old.WITHDRAWN_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                       to_char (:new.WITHDRAWN_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('RANDOM_DATE')
                                      and nvl(:old.RANDOM_DATE, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.RANDOM_DATE, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'RANDOM_DATE', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host , to_char (:old.RANDOM_DATE,'DD-MON-YYYY HH24:MI:SS'),
                                                                       to_char (:new.RANDOM_DATE,'DD-MON-YYYY HH24:MI:SS')); end if; if updating('STATUS')
                                      and nvl(:old.STATUS,'NULL')
                                           <> nvl (:new.STATUS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'STATUS', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.STATUS,:new.STATUS); end if; if updating('STATUS_TS')
                                      and nvl(:old.STATUS_TS,'NULL')
                                           <> nvl (:new.STATUS_TS,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'STATUS_TS', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.STATUS_TS,:new.STATUS_TS); end if; if updating('MODIFIED_TIMESTAMP')
                                      and nvl(:old.MODIFIED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.MODIFIED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'MODIFIED_TIMESTAMP', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.MODIFIED_TIMESTAMP,:new.MODIFIED_TIMESTAMP); end if; if updating('MODIFIED_BY_USER')
                                      and nvl(:old.MODIFIED_BY_USER,'NULL')
                                           <> nvl (:new.MODIFIED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'MODIFIED_BY_USER', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.MODIFIED_BY_USER,:new.MODIFIED_BY_USER); end if; if updating('CREATED_TIMESTAMP')
                                      and nvl(:old.CREATED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy'))
                                           <> nvl (:new.CREATED_TIMESTAMP, to_date ('01/01/1600', 'dd/mm/yyyy')) then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'CREATED_TIMESTAMP', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.CREATED_TIMESTAMP,:new.CREATED_TIMESTAMP); end if; if updating('CREATED_BY_USER')
                                      and nvl(:old.CREATED_BY_USER,'NULL')
                                           <> nvl (:new.CREATED_BY_USER,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'CREATED_BY_USER', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.CREATED_BY_USER,:new.CREATED_BY_USER); end if; if updating('PP_ID')
                                      and nvl(:old.PP_ID,'NULL')
                                           <> nvl (:new.PP_ID,'NULL') then INSERT INTO audit_trail (table_name, operation, primary_key, column_name,  audit_date, audit_by, audit_by_worker, audit_by_application, audit_by_host, old_value, new_value)
                 VALUES ('USR_PAT','u', :old.pat_id, 'PP_ID', SYSTIMESTAMP, v_username, :new.modified_by,v_application, v_host ,:old.PP_ID,:new.PP_ID); end if;  END;
/


DROP TRIGGER WITHDRAW_USR_PAT;

CREATE OR REPLACE TRIGGER WITHDRAW_USR_PAT
            BEFORE UPDATE
            OF WITHDRAWN_DATE
            ON USR_PAT
            referencing OLD as old NEW as new
            FOR EACH ROW
WHEN (
(new.WITHDRAWN_DATE <> old.WITHDRAWN_DATE) or (new.WITHDRAWN_DATE IS NOT NULL AND old.WITHDRAWN_DATE IS NULL)
      )
DECLARE
                strcount int := 0;
                wdstatusKey varchar2(40) := 'PAT_STATUS_WITHDRAWN';
                wdstatusDefault varchar2(40) := 'unset-string-' || wdstatusKey;
                wdstatus varchar2(40) := wdstatusDefault;
            BEGIN
                select count(*) into strcount from strings where str_id = wdstatusKey;
                if (strcount = 1) then
                    select nvl(en,wdstatusDefault) into wdstatus from strings where str_id = wdstatusKey;
                end if;
                
                :new.status := wdstatus;
                :new.status_ts := TO_CHAR (:new.WITHDRAWN_DATE, 'dd-Mon-yyyy HH24:MI:SS');
                 
            END WITHDRAW_USR_PAT;
/
